In [15]:
import pandas as pd
import pickle
data = pd.read_pickle('processed_10k_articles.pkl')


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
tfidf=TfidfVectorizer(min_df=.03, stop_words='english') #later, try PCA on full Tfidf matrix
tfidf_matrix=tfidf.fit_transform(data['process'])

In [17]:
import pandas as pd
import numpy, scipy
from sklearn.cluster import KMeans

class Iterative_KMeans:

    def __init__(self, dataframe, num_clusters, feature_matrix, min_cluster_size, max_depth): #note feature_matrix and dataframe must have same num of rows; also f_m should be sparse
        self.dataframe = dataframe
        self.num_clusters = num_clusters
        self.feature_matrix=feature_matrix
        self.n_samples=dataframe.shape[0]
        self.min_cluster_size=min_cluster_size #minimum size clsuter that algo will split at each iteration
        self.max_depth=max_depth #max depth of tree


    def get_cluster_labels(self, feature_matrix, num_clusters): #performs kmeans and returns vector of labels
        kmeans=KMeans(n_clusters=num_clusters)
        labels=kmeans.fit_predict(feature_matrix)
        return labels

    def remove_rows(self, sparse_matrix, index): #if index=[2,4,6] removes rows 2 4 6 from matrix
        return numpy.delete(sparse_matrix.toarray(), index, 0)


    @staticmethod
    def tuple_append(t, entry): #add entry to end of tuple t, like list append
        x=list(t)
        x.append(entry)
        return(tuple(x))

    def get_final_labels(self, df): #returns vector of final cluster labels, after Level data has been added in Run(); labels is a vector of tuples
        labels=[]
        for i in df.index.values:
            j=1
            while df['Level_'+str(j)][i]==None:
                j+=1
            labels.append(df['Level_'+str(j)][i])
        return labels

    def get_cluster_titles(self, cluster_label, final_labels):  # return list of all elements whose cluster label starts with cluster_label; note c_l is a tuple and f_l is vector of tuples
        titles=[]
        cluster_depth=len(cluster_label)
        for i in range(self.dataframe.shape[0]):
            if final_labels[i][0:cluster_depth]==cluster_label: #check if beginning of final label matches given label
                titles.append(self.dataframe['title'][i])
        return titles

    def get_cm(self, label, final_labels): #return cm in feature space of all points in cluter labeled by label
        cm=[0]*self.feature_matrix.shape[1]
        count=0
        for i in range(len(final_labels)):
            if final_labels[i]==label:
                cm+=self.feature_matrix[i, :].toarray() #if matrix is sparse
                count+=1
        if count==0:
            print('Warning: Cluster '+str(label)+' is empty')
            return 0
        else:
            return cm/count


    def get_cm_dict(self, final_labels): #returns a dictionary with entries of the form {(3,4): cm of cluster (3,4)}
        cm={}
        for label in set(final_labels): #iterate over distinct labels
            cm[label]=self.get_cm(label, final_labels)
        return cm

    def get_nearest_cm(self,feature_vector, cm_dict): #given feature vector not in data set, finds closest cluster center
        min_dist=float('inf')
        for label in cm_dict.keys():
            if scipy.spatial.distance.euclidean(feature_vector, cm_dict[label])<min_dist:
                min_dist=scipy.spatial.distance.euclidean(feature_vector, cm_dict[label])
                min_label=label
        return min_label



    def Run(self): #returns vector of cluster lables; each entry is a tuple; note each tuple begins with 0
        level=0
        df=self.dataframe
        df['Level_0']=[(0,)]*self.n_samples #need comma for single element tuple
        big_clusters=[(0,)]
        index=df.index.values
        while len(big_clusters)>0 and level<self.max_depth:
            sizes={} #records size of each newly created cluster
            level+=1
            df['Level_'+str(level)]=df['Level_'+str(level-1)] #create new column for next level
            for label in big_clusters: #loop though cluster with >=min_cluster_size elements
                cluster_df=df[df['Level_'+str(level)]==label] #get rows in current cluster
                cluster_df['new_label']=[[]]*cluster_df.shape[0]
                cluster_index=cluster_df.index.values
                cluster_fm=self.remove_rows(self.feature_matrix, list(set(index).difference(set(cluster_index)))) #keep only rows from feature matrix corresponding to these articles
                labels=self.get_cluster_labels(cluster_fm, self.num_clusters) #perform clustering
                cluster_df['new_label']=labels
                count=[0]*self.num_clusters #initialize array to store length of each new cluster
                for i in cluster_index: #loop through indices of original cluster
                    a=df['Level_'+str(level)][i]
                    b=cluster_df['new_label'][i] #for ease of reading
                    df['Level_'+str(level)][i]=self.tuple_append(a,b) #go to entry in original df and update
                    count[cluster_df['new_label'][i]]+=1 #compute size of each new cluster

                for i in range(self.num_clusters): #loop thorugh new clusters
                    sizes[self.tuple_append(label, i)]=count[i] #record size, using total address as key
                    #sizes records size of all new clusters, not just those created from a given label; the above step ensures sizes has separate entries e..g (0,1,2) and (0,2,2) (here label is the first two elements)
            for i in range(self.n_samples): #loop through dataframe and remove redundant entries
                if df['Level_'+str(level)][i]==df['Level_'+str(level-1)][i] and df['Level_'+str(level-1)][i]!=None:
                    df['Level_'+str(level)][i]=None #delete duplicate labels for clusters that weren't further divided
                else:
                    df['Level_'+str(level-1)][i]=None #keep only most recent label for clusters that were further divided


            big_clusters=[label for label in list(sizes.keys()) if sizes[label]>=self.min_cluster_size] #update with new clusters
        return self.get_final_labels(df) #return vector of tuples




In [18]:
Iter_KM=Iterative_KMeans(dataframe=data, num_clusters=10, feature_matrix=tfidf_matrix, min_cluster_size=20, max_depth=7)

I just copypasted the output below. This is what you would get by running Iter_KM.Run().

The label (0,1,0,3) means the article is in the 3rd cluster of the 0th cluster of the first cluster of the 0th cluster. You'll notice every tuple begins with a zero-this was just to make the program easier to write, and can be very easily removed from the output. i

In [19]:
labels=[(0, 4, 8, 5, 2, 8), (0, 1, 0, 3), (0, 1, 5, 1), (0, 4, 8, 5, 7, 7, 6), (0, 1, 4, 2), (0, 1, 4, 6, 2), (0, 1, 2, 7, 2), (0, 1, 4, 6, 1), (0, 4, 8, 5, 7, 0, 0, 8), (0, 1, 4, 6, 3), (0, 6, 8, 0), (0, 1, 4, 6, 6), (0, 1, 6, 6, 4), (0, 4, 8, 8, 6), (0, 1, 5, 2, 0), (0, 1, 3, 9, 0), (0, 1, 2, 3), (0, 1, 6, 5, 0), (0, 5, 4, 8), (0, 1, 6, 2), (0, 1, 6, 2), (0, 1, 2, 7, 2), (0, 4, 2, 7, 1, 1), (0, 4, 8, 0, 3), (0, 4, 4, 9), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 4, 7), (0, 4, 0, 5, 9), (0, 4, 8, 5, 3, 8), (0, 4, 8, 2, 5, 5), (0, 6, 0, 2), (0, 1, 6, 6, 8), (0, 1, 4, 1), (0, 2, 0, 6), (0, 1, 6, 3, 7), (0, 4, 8, 5, 0, 9), (0, 5, 8, 5), (0, 1, 2, 8, 5), (0, 1, 0, 0), (0, 1, 6, 9, 7), (0, 4, 0, 5, 2), (0, 4, 8, 5, 1, 3), (0, 4, 8, 4, 2), (0, 1, 0, 9), (0, 1, 5, 7), (0, 1, 6, 8), (0, 1, 4, 5), (0, 4, 8, 5, 1, 6), (0, 4, 0, 1, 2), (0, 4, 5, 2, 0), (0, 2, 3, 2, 5), (0, 4, 8, 1, 7), (0, 4, 0, 0, 7), (0, 2, 2, 5), (0, 4, 8, 2, 2), (0, 2, 9, 0), (0, 2, 9, 0), (0, 2, 1, 6), (0, 4, 6, 9), (0, 4, 9, 2, 4), (0, 1, 6, 3, 1, 8), (0, 4, 4, 3), (0, 1, 5, 8, 3), (0, 1, 3, 5), (0, 1, 2, 4, 3), (0, 1, 6, 6, 5), (0, 6, 9, 8, 4), (0, 4, 0, 4, 4), (0, 1, 4, 3, 1), (0, 4, 8, 5, 1, 0, 1), (0, 1, 0, 2, 7), (0, 1, 5, 1), (0, 4, 8, 5, 1, 3), (0, 1, 0, 3), (0, 4, 8, 5, 5, 8), (0, 2, 7, 7), (0, 4, 0, 5, 1), (0, 1, 7, 7, 9), (0, 4, 8, 5, 1, 8), (0, 1, 7, 3), (0, 1, 5, 4), (0, 1, 4, 9), (0, 1, 0, 2, 5), (0, 1, 6, 3, 5), (0, 1, 5, 8, 5), (0, 4, 8, 0, 2), (0, 1, 2, 4, 3), (0, 4, 8, 5, 1, 2), (0, 1, 0, 2, 1), (0, 2, 1, 6), (0, 6, 5, 1), (0, 4, 8, 5, 8, 0), (0, 5, 7, 1), (0, 1, 4, 4), (0, 2, 3, 8, 8), (0, 4, 8, 5, 7, 0, 3, 3), (0, 1, 7, 7, 1), (0, 1, 3, 9, 0), (0, 4, 0, 5, 1), (0, 1, 6, 6, 0), (0, 4, 8, 1, 2), (0, 4, 9, 2, 4), (0, 1, 4, 4), (0, 1, 5, 2, 6), (0, 1, 9, 8), (0, 4, 0, 5, 9), (0, 4, 8, 5, 7, 0, 1), (0, 1, 8, 1, 2), (0, 6, 0, 6), (0, 4, 5, 4, 7), (0, 1, 7, 7, 1), (0, 4, 2, 7, 4), (0, 6, 8, 1), (0, 6, 0, 7, 6), (0, 1, 6, 3, 7), (0, 4, 4, 1, 3), (0, 4, 6, 6, 4), (0, 1, 9, 0, 9), (0, 0, 7, 3), (0, 1, 7, 2), (0, 4, 6, 9), (0, 4, 8, 5, 7, 0, 2, 8), (0, 4, 8, 1, 5), (0, 1, 6, 2), (0, 1, 6, 4, 5), (0, 1, 5, 8, 0), (0, 1, 3, 2), (0, 1, 3, 8), (0, 4, 6, 4), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 2, 7, 2), (0, 4, 8, 5, 2, 1), (0, 4, 0, 5, 1), (0, 4, 0, 6, 3, 1), (0, 4, 8, 5, 1, 3), (0, 6, 4, 2), (0, 6, 4, 2), (0, 4, 8, 6, 3), (0, 6, 0, 9, 0), (0, 6, 4, 2), (0, 4, 0, 6, 1), (0, 9, 2, 5), (0, 4, 8, 2, 4, 7), (0, 4, 6, 2, 2), (0, 6, 7, 8), (0, 1, 4, 5), (0, 4, 8, 2, 5, 2), (0, 4, 8, 2, 5, 2), (0, 4, 8, 2, 1, 8), (0, 4, 8, 7, 3), (0, 4, 8, 2, 1, 2), (0, 4, 8, 2, 1, 1), (0, 9, 2, 3), (0, 9, 6, 2), (0, 9, 6, 2), (0, 1, 5, 5, 4), (0, 1, 9, 0, 1), (0, 1, 6, 6, 3), (0, 1, 6, 6, 3), (0, 1, 5, 8, 4), (0, 4, 5, 9), (0, 4, 0, 0, 2), (0, 7, 1, 6, 8), (0, 1, 7, 0), (0, 1, 5, 2, 2), (0, 4, 8, 8, 8, 1), (0, 1, 0, 6), (0, 6, 9, 8, 4), (0, 1, 5, 8, 4), (0, 4, 5, 4, 2), (0, 1, 2, 7, 2), (0, 1, 8, 3), (0, 4, 8, 5, 7, 2, 8), (0, 4, 8, 5, 7, 0, 2, 0), (0, 6, 5, 3), (0, 5, 4, 5), (0, 1, 4, 7), (0, 4, 3, 1, 4), (0, 4, 6, 2, 2), (0, 4, 0, 5, 6), (0, 4, 2, 7, 5), (0, 5, 7, 8), (0, 4, 8, 5, 1, 0, 1), (0, 1, 9, 0, 6), (0, 1, 3, 0), (0, 1, 6, 1, 2), (0, 4, 8, 5, 1, 1, 3), (0, 4, 7, 8, 8), (0, 4, 8, 5, 8, 3, 2), (0, 4, 8, 8, 0, 1), (0, 5, 0, 2, 1, 1), (0, 1, 1, 0), (0, 1, 6, 5, 1), (0, 4, 6, 2, 1), (0, 1, 2, 6), (0, 4, 8, 5, 1, 0, 1), (0, 1, 0, 5), (0, 4, 5, 4, 1, 1), (0, 4, 0, 0, 7), (0, 6, 0, 3, 6), (0, 0, 0, 0), (0, 4, 0, 5, 6), (0, 1, 0, 7), (0, 1, 6, 6, 1), (0, 1, 4, 6, 4), (0, 1, 2, 4, 5), (0, 6, 9, 8, 4), (0, 4, 8, 5, 2, 6), (0, 1, 8, 5), (0, 2, 3, 2, 1), (0, 4, 6, 7), (0, 6, 0, 7, 3), (0, 6, 1, 7), (0, 4, 7, 8, 9), (0, 4, 8, 5, 5, 8), (0, 1, 6, 7, 5, 0), (0, 4, 7, 8, 5), (0, 4, 8, 5, 8, 4, 3), (0, 4, 6, 9), (0, 6, 4, 3), (0, 5, 4, 9), (0, 6, 3, 9), (0, 4, 8, 5, 7, 8, 3), (0, 6, 9, 0), (0, 0, 1, 5), (0, 4, 8, 5, 7, 9, 9), (0, 4, 0, 9), (0, 6, 0, 6), (0, 6, 4, 5), (0, 9, 6, 0), (0, 1, 9, 4), (0, 4, 8, 5, 4, 5), (0, 6, 7, 8), (0, 9, 8, 4), (0, 0, 3, 1), (0, 6, 4, 2), (0, 1, 2, 4, 3), (0, 6, 4, 0, 6), (0, 1, 3, 9, 6), (0, 6, 4, 0, 9), (0, 6, 0, 7, 3), (0, 4, 9, 1), (0, 0, 2, 6), (0, 4, 8, 0, 9, 1, 9), (0, 6, 2, 3), (0, 1, 4, 3, 1), (0, 4, 8, 0, 5, 3), (0, 4, 0, 1, 5, 2), (0, 4, 0, 3, 0), (0, 4, 0, 0, 4), (0, 1, 0, 8), (0, 9, 3, 2, 1), (0, 6, 4, 0, 3), (0, 1, 4, 1), (0, 6, 4, 2), (0, 4, 0, 5, 7), (0, 1, 8, 3), (0, 4, 6, 5, 0), (0, 1, 2, 4, 3), (0, 1, 1, 4), (0, 4, 8, 5, 2, 6), (0, 4, 8, 5, 3, 9), (0, 4, 0, 5, 4), (0, 4, 8, 5, 7, 4, 8), (0, 6, 4, 3), (0, 6, 4, 5), (0, 6, 4, 0, 4), (0, 4, 8, 5, 7, 9, 0), (0, 6, 4, 0, 4), (0, 6, 4, 0, 1), (0, 4, 8, 5, 7, 6, 1), (0, 6, 4, 5), (0, 0, 0, 4, 7), (0, 4, 8, 5, 7, 4, 8), (0, 1, 2, 7, 1), (0, 4, 4, 3), (0, 6, 4, 2), (0, 1, 4, 3, 0), (0, 9, 2, 3), (0, 1, 5, 6, 9), (0, 6, 8, 0), (0, 4, 8, 7, 8), (0, 6, 4, 8), (0, 4, 0, 6, 4, 1), (0, 1, 4, 4), (0, 4, 6, 6, 6), (0, 1, 5, 7), (0, 1, 7, 7, 3), (0, 4, 4, 0, 1), (0, 1, 5, 8, 3), (0, 1, 0, 4), (0, 6, 0, 6), (0, 4, 8, 5, 8, 6, 4), (0, 4, 6, 4), (0, 6, 4, 0, 0), (0, 6, 4, 0, 7), (0, 4, 8, 1, 2), (0, 4, 6, 5, 8), (0, 4, 8, 5, 8, 3, 4), (0, 6, 4, 0, 3), (0, 6, 4, 0, 0), (0, 6, 4, 2), (0, 4, 8, 5, 5, 8), (0, 6, 4, 2), (0, 6, 4, 0, 8), (0, 6, 4, 0, 2), (0, 6, 4, 0, 0), (0, 6, 4, 1), (0, 6, 4, 2), (0, 6, 4, 2), (0, 6, 4, 0, 4), (0, 6, 4, 5), (0, 6, 4, 0, 2), (0, 6, 4, 0, 3), (0, 6, 4, 0, 3), (0, 6, 4, 2), (0, 6, 4, 2), (0, 6, 4, 2), (0, 6, 4, 0, 5), (0, 2, 3, 7, 1), (0, 1, 0, 5), (0, 6, 4, 0, 4), (0, 6, 4, 2), (0, 6, 4, 0, 3), (0, 6, 4, 0, 9), (0, 6, 4, 0, 3), (0, 6, 4, 0, 8), (0, 6, 4, 2), (0, 6, 4, 0, 3), (0, 6, 4, 0, 2), (0, 6, 4, 2), (0, 6, 0, 4), (0, 4, 8, 8, 6), (0, 1, 2, 5, 9), (0, 4, 6, 2, 9), (0, 0, 0, 4, 3), (0, 4, 6, 0, 3), (0, 4, 4, 0, 1), (0, 2, 9, 1), (0, 4, 4, 6), (0, 4, 6, 2, 2), (0, 6, 0, 1, 1), (0, 3, 9), (0, 6, 0, 7, 4), (0, 4, 8, 1, 5), (0, 1, 2, 2, 8), (0, 4, 0, 6, 3, 1), (0, 4, 8, 4, 4, 4), (0, 2, 3, 7, 2), (0, 5, 4, 0), (0, 1, 5, 1), (0, 4, 2, 7, 1, 2), (0, 4, 6, 1, 3), (0, 4, 6, 5, 2), (0, 4, 8, 5, 7, 0, 4), (0, 6, 0, 4), (0, 0, 9, 6), (0, 0, 0, 4, 3), (0, 6, 4, 8), (0, 4, 8, 5, 3, 4, 8), (0, 0, 0, 1), (0, 6, 7, 3, 2), (0, 4, 5, 2, 3), (0, 1, 4, 3, 3), (0, 6, 7, 4, 5), (0, 4, 6, 6, 1), (0, 1, 6, 5, 8), (0, 6, 7, 6), (0, 1, 6, 7, 3), (0, 1, 6, 5, 5), (0, 4, 3, 3), (0, 5, 0, 2, 1, 1), (0, 0, 0, 4, 8), (0, 1, 4, 3, 4), (0, 6, 9, 9), (0, 9, 8, 5), (0, 0, 4, 6), (0, 1, 6, 8), (0, 0, 2, 1), (0, 6, 7, 4, 2), (0, 1, 0, 3), (0, 1, 5, 1), (0, 4, 0, 6, 3, 1), (0, 4, 8, 5, 7, 0, 4), (0, 0, 9, 3), (0, 4, 8, 1, 8), (0, 1, 4, 6, 7), (0, 1, 6, 7, 5, 9), (0, 4, 8, 5, 9, 3), (0, 0, 2, 6), (0, 1, 5, 3), (0, 1, 2, 5, 7, 6), (0, 0, 3, 1), (0, 6, 8, 2), (0, 2, 3, 2, 6), (0, 1, 5, 9), (0, 4, 8, 5, 8, 2), (0, 4, 6, 1, 0), (0, 9, 3, 1), (0, 4, 8, 5, 7, 8, 3), (0, 4, 8, 5, 8, 4, 4), (0, 1, 6, 7, 0, 9), (0, 9, 2, 1), (0, 4, 8, 5, 9, 6), (0, 1, 5, 0, 1), (0, 6, 0, 5, 8), (0, 2, 0, 6), (0, 4, 0, 1, 7), (0, 6, 0, 6), (0, 6, 7, 4, 1), (0, 4, 8, 7, 6), (0, 1, 5, 6, 3), (0, 4, 9, 2, 4), (0, 4, 0, 1, 5, 9), (0, 1, 6, 6, 8), (0, 9, 3, 9), (0, 1, 4, 3, 9), (0, 1, 6, 4, 4), (0, 1, 4, 6, 4), (0, 4, 8, 5, 7, 5, 8), (0, 4, 6, 2, 8), (0, 0, 5), (0, 4, 0, 6, 1), (0, 4, 8, 5, 5, 1), (0, 4, 8, 9, 0), (0, 1, 6, 9, 3), (0, 0, 9, 2, 9), (0, 4, 6, 2, 2), (0, 4, 0, 5, 6), (0, 9, 6, 5), (0, 1, 2, 1), (0, 0, 2, 2), (0, 1, 1, 0), (0, 2, 1, 8), (0, 4, 8, 5, 7, 3, 0), (0, 6, 0, 9, 2), (0, 4, 0, 6, 4, 2), (0, 4, 0, 5, 5), (0, 4, 8, 2, 1, 2), (0, 4, 8, 5, 7, 8, 9), (0, 4, 8, 5, 8, 3, 1), (0, 1, 6, 1, 2), (0, 0, 7, 9), (0, 4, 6, 2, 8), (0, 1, 6, 6, 8), (0, 6, 9, 8, 7), (0, 6, 9, 8, 4), (0, 4, 0, 5, 5), (0, 1, 3, 2), (0, 7, 7, 2), (0, 1, 5, 0, 1), (0, 4, 8, 8, 8, 1), (0, 1, 5, 1), (0, 4, 8, 5, 7, 0, 3, 1), (0, 4, 2, 1), (0, 4, 8, 1, 5), (0, 4, 2, 4, 3, 7), (0, 6, 6, 4, 3), (0, 4, 8, 5, 8, 5, 4), (0, 4, 8, 5, 7, 0, 8), (0, 7, 1, 6, 9), (0, 1, 4, 1), (0, 1, 0, 2, 8), (0, 4, 8, 8, 4), (0, 6, 8, 1), (0, 6, 9, 8, 4), (0, 4, 8, 2, 9), (0, 1, 6, 5, 1), (0, 1, 1, 3), (0, 1, 6, 7, 5, 2), (0, 6, 2, 8), (0, 1, 9, 2), (0, 4, 8, 5, 1, 2), (0, 0, 3, 2), (0, 4, 8, 8, 8, 7), (0, 5, 0, 5), (0, 1, 5, 8, 9), (0, 0, 8, 1), (0, 1, 6, 2), (0, 2, 9, 1), (0, 6, 2, 3), (0, 1, 4, 1), (0, 2, 3, 2, 8), (0, 4, 8, 5, 3, 4, 1), (0, 4, 6, 5, 7), (0, 1, 2, 4, 1), (0, 4, 8, 5, 5, 1), (0, 4, 8, 2, 1, 9), (0, 6, 7, 3, 3), (0, 6, 7, 3, 3), (0, 6, 0, 9, 0), (0, 4, 8, 5, 7, 7, 7), (0, 4, 6, 6, 8), (0, 6, 0, 3, 2), (0, 1, 2, 6), (0, 4, 5, 7, 1), (0, 1, 2, 5, 6), (0, 1, 6, 8), (0, 1, 8, 3), (0, 1, 6, 7, 0, 2), (0, 4, 5, 9), (0, 4, 4, 7), (0, 1, 4, 7), (0, 4, 6, 1, 1, 4), (0, 4, 2, 7, 1, 1), (0, 4, 2, 7, 1, 1), (0, 4, 6, 3), (0, 4, 8, 8, 4), (0, 4, 6, 8, 7), (0, 0, 6, 8), (0, 9, 3, 2, 1), (0, 1, 7, 7, 7), (0, 4, 9, 5, 0, 5), (0, 4, 8, 4, 5), (0, 7, 3, 7), (0, 6, 0, 3, 3), (0, 1, 2, 7, 2), (0, 1, 2, 4, 3), (0, 1, 5, 5, 6), (0, 4, 8, 0, 6), (0, 4, 8, 0, 0, 4), (0, 1, 8, 0), (0, 4, 8, 5, 7, 7, 3), (0, 9, 2, 2), (0, 1, 3, 8), (0, 4, 4, 3), (0, 4, 6, 1, 1, 1), (0, 4, 6, 6, 9), (0, 6, 2, 8), (0, 9, 0, 9), (0, 4, 8, 5, 5, 8), (0, 4, 7, 6, 6), (0, 4, 8, 2, 1, 0), (0, 4, 6, 0, 6), (0, 4, 6, 5, 5), (0, 7, 5, 1, 4), (0, 2, 3, 6, 3), (0, 4, 2, 7, 1, 1), (0, 4, 0, 1, 5, 4), (0, 4, 8, 5, 7, 6, 2), (0, 4, 8, 5, 1, 3), (0, 4, 7, 6, 5), (0, 6, 1, 8), (0, 4, 0, 5, 6), (0, 9, 2, 9), (0, 9, 3, 2, 7, 4), (0, 4, 0, 5, 1), (0, 4, 2, 7, 1, 7), (0, 6, 0, 9, 9), (0, 6, 0, 9, 9), (0, 6, 1, 8), (0, 0, 0, 1), (0, 0, 3, 3), (0, 9, 3, 2, 1), (0, 5, 8, 5), (0, 0, 0, 5, 5), (0, 4, 2, 7, 1, 9), (0, 4, 9, 9), (0, 0, 9, 2, 7), (0, 4, 5, 2, 1), (0, 1, 5, 4), (0, 4, 0, 8, 2), (0, 4, 0, 1, 7), (0, 1, 6, 0), (0, 6, 6, 2), (0, 6, 8, 9), (0, 1, 6, 6, 6), (0, 4, 8, 2, 8), (0, 1, 0, 2, 4), (0, 6, 0, 1, 6), (0, 1, 2, 7, 9), (0, 1, 7, 5), (0, 1, 8, 3), (0, 1, 2, 2, 8), (0, 1, 5, 4), (0, 1, 6, 7, 6), (0, 1, 4, 6, 1), (0, 4, 8, 3, 4, 6), (0, 2, 0, 6), (0, 0, 1, 9, 3), (0, 4, 8, 2, 2), (0, 6, 2, 8), (0, 4, 8, 2, 9), (0, 4, 8, 5, 7, 0, 1), (0, 1, 2, 4, 1), (0, 6, 0, 4), (0, 4, 2, 1), (0, 0, 7, 3), (0, 1, 0, 7), (0, 4, 6, 7), (0, 1, 2, 5, 7, 5), (0, 4, 6, 5, 8), (0, 4, 8, 5, 7, 0, 0, 7), (0, 4, 8, 2, 1, 1), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 7, 0, 1), (0, 7, 1, 7), (0, 4, 2, 7, 5), (0, 4, 2, 7, 1, 3), (0, 1, 3, 2), (0, 4, 6, 5, 1), (0, 4, 8, 5, 8, 0), (0, 9, 0, 1, 1), (0, 4, 8, 5, 7, 4, 0), (0, 4, 6, 2, 1), (0, 2, 3, 7, 1), (0, 0, 5), (0, 4, 8, 0, 8, 4), (0, 1, 4, 5), (0, 4, 8, 5, 1, 3), (0, 1, 5, 7), (0, 1, 4, 6, 8), (0, 4, 7, 2), (0, 4, 0, 1, 5, 2), (0, 1, 3, 7), (0, 4, 5, 4, 1, 2), (0, 1, 6, 3, 3), (0, 4, 8, 5, 8, 9), (0, 4, 8, 5, 8, 2), (0, 4, 5, 4, 1, 2), (0, 4, 0, 5, 1), (0, 6, 0, 7, 3), (0, 4, 0, 6, 3, 3), (0, 4, 6, 1, 1, 1), (0, 4, 0, 1, 1), (0, 4, 0, 7, 5), (0, 4, 6, 6, 3), (0, 4, 6, 2, 2), (0, 4, 8, 1, 6), (0, 4, 8, 5, 7, 0, 5), (0, 4, 8, 9, 4, 8), (0, 1, 6, 9, 2), (0, 4, 8, 0, 6), (0, 1, 5, 8, 1), (0, 2, 5, 1), (0, 1, 1, 2), (0, 4, 8, 5, 1, 3), (0, 4, 8, 9, 4, 1), (0, 1, 2, 0), (0, 1, 6, 5, 8), (0, 1, 4, 6, 4), (0, 4, 8, 2, 0), (0, 4, 8, 8, 8, 1), (0, 1, 3, 3), (0, 4, 6, 6, 2), (0, 1, 0, 2, 4), (0, 4, 8, 7, 1), (0, 1, 4, 8), (0, 6, 0, 6), (0, 4, 7, 6, 3), (0, 1, 4, 6, 4), (0, 4, 0, 1, 2), (0, 4, 8, 2, 9), (0, 6, 0, 7, 6), (0, 1, 7, 5), (0, 1, 4, 6, 0), (0, 4, 0, 6, 3, 6), (0, 4, 8, 2, 0), (0, 4, 3, 8), (0, 4, 6, 9), (0, 6, 1, 8), (0, 4, 2, 4, 3, 2), (0, 2, 8, 3), (0, 1, 4, 6, 6), (0, 4, 8, 0, 0, 9), (0, 1, 6, 7, 2), (0, 6, 9, 8, 4), (0, 9, 3, 2, 0), (0, 1, 6, 3, 0), (0, 1, 6, 0), (0, 6, 9, 5), (0, 1, 4, 2), (0, 4, 8, 8, 4), (0, 2, 3, 8, 8), (0, 1, 1, 2), (0, 6, 3, 6), (0, 4, 8, 9, 3), (0, 6, 2, 6), (0, 4, 8, 2, 0), (0, 6, 0, 7, 6), (0, 1, 4, 5), (0, 1, 4, 6, 7), (0, 1, 6, 7, 0, 1), (0, 4, 8, 5, 2, 4), (0, 1, 4, 4), (0, 4, 2, 2, 7), (0, 1, 2, 7, 1), (0, 6, 0, 8, 2), (0, 1, 5, 3), (0, 1, 2, 4, 9), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 7, 7, 3), (0, 1, 2, 2, 0), (0, 9, 3, 2, 3), (0, 1, 5, 8, 5), (0, 6, 0, 6), (0, 4, 8, 9, 4, 1), (0, 1, 3, 3), (0, 4, 8, 5, 2, 1), (0, 0, 8, 5), (0, 4, 8, 2, 4, 2), (0, 4, 3, 8), (0, 6, 3, 7), (0, 4, 0, 2, 0), (0, 4, 8, 5, 8, 2), (0, 9, 6, 6), (0, 4, 8, 5, 7, 7, 1), (0, 3, 8), (0, 4, 6, 9), (0, 1, 5, 8, 4), (0, 4, 2, 1), (0, 1, 2, 3), (0, 2, 3, 7, 1), (0, 6, 2, 8), (0, 4, 0, 5, 1), (0, 4, 8, 5, 7, 8, 4), (0, 1, 4, 3, 9), (0, 6, 4, 7), (0, 6, 7, 9), (0, 1, 9, 7), (0, 4, 2, 7, 5), (0, 4, 8, 5, 0, 3), (0, 4, 7, 4), (0, 4, 8, 5, 7, 8, 4), (0, 9, 1, 2), (0, 9, 2, 3), (0, 1, 4, 4), (0, 4, 8, 5, 7, 8, 4), (0, 1, 1, 7), (0, 6, 0, 1, 9), (0, 6, 9, 9), (0, 1, 6, 1, 3), (0, 4, 8, 5, 8, 4, 4), (0, 4, 5, 4, 1, 3), (0, 4, 8, 5, 7, 2, 2), (0, 4, 6, 1, 6), (0, 1, 3, 7), (0, 1, 3, 1), (0, 1, 4, 6, 1), (0, 4, 8, 5, 8, 3, 5), (0, 4, 8, 5, 8, 6, 4), (0, 4, 7, 9), (0, 4, 8, 0, 0, 2), (0, 4, 8, 5, 8, 3, 0), (0, 4, 6, 1, 0), (0, 1, 5, 9), (0, 4, 8, 8, 7), (0, 4, 6, 5, 8), (0, 4, 0, 5, 0), (0, 1, 4, 5), (0, 1, 6, 4, 1), (0, 4, 8, 9, 4, 1), (0, 6, 7, 0, 9), (0, 4, 0, 1, 1), (0, 4, 0, 6, 3, 2), (0, 1, 4, 6, 3), (0, 5, 4, 3), (0, 1, 6, 8), (0, 4, 0, 1, 5, 2), (0, 1, 2, 2, 8), (0, 1, 8, 1, 0), (0, 9, 3, 1), (0, 1, 1, 3), (0, 1, 8, 1, 1), (0, 1, 5, 6, 8), (0, 1, 5, 8, 0), (0, 9, 3, 5), (0, 1, 6, 4, 2), (0, 2, 9, 9), (0, 2, 9, 9), (0, 2, 9, 1), (0, 4, 8, 8, 0, 7), (0, 1, 6, 1, 4), (0, 4, 8, 0, 3), (0, 4, 5, 2, 2), (0, 4, 2, 7, 2), (0, 1, 5, 0, 4), (0, 4, 8, 5, 2, 3), (0, 9, 3, 2, 7, 2), (0, 4, 8, 0, 0, 2), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 2, 8), (0, 4, 8, 1, 2), (0, 1, 1, 0), (0, 1, 4, 4), (0, 1, 8, 1, 5), (0, 4, 8, 5, 8, 6, 4), (0, 1, 0, 2, 7), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 2, 5, 1), (0, 4, 6, 2, 7), (0, 1, 0, 4), (0, 2, 3, 8, 7), (0, 1, 1, 2), (0, 4, 7, 4), (0, 4, 7, 1), (0, 4, 5, 1), (0, 4, 8, 5, 2, 1), (0, 1, 1, 0), (0, 4, 8, 5, 1, 3), (0, 4, 0, 5, 6), (0, 4, 8, 5, 8, 8), (0, 1, 1, 3), (0, 4, 8, 2, 3), (0, 4, 8, 5, 8, 8), (0, 4, 8, 5, 9, 8), (0, 4, 0, 4, 3), (0, 1, 1, 3), (0, 1, 0, 7), (0, 1, 1, 2), (0, 4, 8, 5, 3, 2), (0, 1, 5, 4), (0, 4, 0, 1, 1), (0, 4, 2, 7, 1, 1), (0, 1, 5, 0, 7), (0, 4, 8, 5, 8, 5, 8), (0, 6, 0, 9, 3), (0, 6, 1, 1, 1), (0, 9, 3, 6), (0, 4, 6, 5, 7), (0, 4, 8, 5, 8, 7), (0, 6, 9, 2), (0, 1, 3, 8), (0, 4, 8, 5, 8, 7), (0, 4, 0, 5, 0), (0, 4, 8, 5, 8, 3, 0), (0, 4, 8, 5, 7, 2, 5), (0, 1, 5, 6, 4), (0, 6, 0, 6), (0, 4, 0, 6, 1), (0, 4, 0, 5, 4), (0, 2, 7, 0, 5), (0, 1, 0, 9), (0, 1, 0, 7), (0, 4, 8, 5, 3, 5), (0, 4, 0, 5, 6), (0, 4, 0, 5, 0), (0, 1, 7, 7, 2), (0, 6, 7, 2), (0, 4, 0, 5, 5), (0, 4, 0, 5, 2), (0, 4, 0, 5, 0), (0, 4, 5, 4, 1, 0), (0, 4, 6, 2, 4), (0, 4, 0, 6, 4, 1), (0, 1, 4, 1), (0, 4, 6, 3), (0, 1, 2, 5, 2), (0, 6, 2, 3), (0, 4, 6, 9), (0, 4, 2, 2, 4), (0, 4, 8, 1, 2), (0, 2, 1, 6), (0, 4, 8, 8, 8, 6), (0, 0, 8, 3), (0, 4, 0, 1, 8), (0, 1, 0, 0), (0, 4, 6, 1, 1, 1), (0, 1, 4, 1), (0, 4, 6, 9), (0, 4, 6, 2, 8), (0, 4, 6, 6, 3), (0, 5, 1, 3), (0, 5, 4, 0), (0, 4, 6, 6, 9), (0, 4, 5, 4, 0), (0, 1, 1, 0), (0, 1, 5, 8, 4), (0, 1, 7, 0), (0, 4, 8, 9, 4, 3), (0, 1, 6, 6, 8), (0, 4, 8, 5, 7, 7, 3), (0, 1, 4, 3, 9), (0, 4, 6, 5, 7), (0, 4, 8, 1, 2), (0, 4, 8, 5, 7, 3, 7), (0, 4, 4, 3), (0, 4, 8, 0, 2), (0, 4, 6, 2, 2), (0, 1, 0, 3), (0, 4, 0, 1, 7), (0, 4, 6, 5, 7), (0, 4, 6, 1, 8), (0, 4, 6, 1, 3), (0, 4, 8, 4, 4, 4), (0, 4, 6, 9), (0, 1, 6, 6, 4), (0, 3, 3, 8), (0, 1, 2, 5, 7, 3), (0, 4, 8, 0, 2), (0, 4, 8, 2, 2), (0, 5, 3, 0), (0, 1, 2, 4, 0), (0, 1, 2, 7, 7), (0, 1, 3, 0), (0, 1, 2, 7, 6), (0, 6, 5, 5), (0, 6, 0, 6), (0, 6, 0, 6), (0, 4, 5, 2, 2), (0, 1, 6, 2), (0, 1, 4, 5), (0, 6, 0, 3, 7), (0, 4, 6, 8, 6), (0, 5, 0, 2, 2), (0, 6, 7, 3, 0), (0, 1, 0, 3), (0, 4, 6, 5, 7), (0, 1, 5, 6, 9), (0, 4, 8, 5, 7, 7, 6), (0, 4, 8, 6, 1), (0, 1, 1, 3), (0, 2, 1, 1), (0, 4, 8, 5, 7, 0, 7), (0, 1, 4, 3, 0), (0, 1, 4, 6, 3), (0, 9, 3, 2, 1), (0, 4, 6, 6, 1), (0, 2, 3, 8, 8), (0, 4, 0, 4, 4), (0, 1, 2, 5, 3), (0, 6, 0, 9, 2), (0, 4, 8, 2, 4, 2), (0, 4, 0, 8, 4), (0, 4, 0, 1, 5, 6), (0, 1, 6, 1, 2), (0, 1, 6, 7, 0, 1), (0, 4, 0, 7, 6), (0, 1, 5, 8, 6), (0, 4, 8, 5, 3, 3), (0, 1, 3, 1), (0, 1, 4, 6, 3), (0, 0, 4, 5), (0, 4, 8, 5, 7, 9, 8), (0, 0, 9, 2, 0), (0, 4, 7, 8, 4), (0, 2, 1, 1), (0, 1, 2, 4, 2), (0, 2, 7, 0, 3), (0, 4, 8, 5, 7, 7, 4), (0, 4, 8, 9, 4, 0), (0, 4, 8, 0, 8, 1), (0, 4, 8, 5, 7, 7, 3), (0, 4, 8, 5, 7, 4, 2), (0, 4, 2, 3), (0, 4, 0, 7, 7), (0, 6, 5, 1), (0, 4, 8, 5, 7, 1, 3), (0, 2, 3, 7, 9), (0, 4, 8, 5, 1, 8), (0, 6, 8, 1), (0, 5, 8, 1), (0, 1, 4, 6, 2), (0, 6, 9, 9), (0, 2, 7, 5), (0, 4, 8, 5, 7, 9, 8), (0, 1, 2, 5, 2), (0, 4, 8, 0, 9, 3), (0, 4, 6, 0, 0), (0, 9, 3, 7), (0, 2, 9, 3), (0, 4, 3, 1, 6), (0, 1, 8, 3), (0, 4, 6, 5, 8), (0, 1, 5, 2, 6), (0, 0, 9, 6), (0, 6, 3, 1), (0, 4, 8, 5, 7, 2, 7), (0, 4, 8, 5, 8, 3, 9), (0, 4, 8, 7, 5), (0, 0, 4, 2), (0, 4, 8, 3, 2), (0, 4, 8, 5, 8, 4, 3), (0, 6, 0, 9, 0), (0, 4, 8, 2, 5, 4), (0, 4, 8, 5, 2, 8), (0, 1, 6, 1, 2), (0, 4, 8, 5, 5, 3), (0, 4, 8, 8, 5), (0, 1, 5, 0, 2), (0, 9, 3, 2, 3), (0, 1, 3, 1), (0, 4, 5, 9), (0, 1, 2, 5, 1), (0, 4, 8, 3, 4, 1), (0, 4, 2, 7, 1, 8), (0, 4, 0, 3, 0), (0, 4, 8, 5, 4, 3), (0, 1, 3, 6), (0, 1, 9, 2), (0, 4, 7, 6, 6), (0, 6, 7, 4, 8), (0, 1, 7, 0), (0, 1, 6, 7, 0, 0), (0, 4, 8, 5, 8, 8), (0, 4, 8, 8, 8, 1), (0, 1, 3, 1), (0, 4, 8, 5, 3, 1), (0, 6, 1, 4), (0, 4, 8, 6, 3), (0, 4, 8, 5, 8, 7), (0, 4, 8, 5, 7, 9, 2), (0, 4, 2, 2, 7), (0, 4, 5, 2, 8), (0, 4, 8, 8, 8, 1), (0, 4, 7, 6, 9), (0, 6, 9, 8, 4), (0, 4, 8, 5, 7, 0, 3, 0), (0, 4, 8, 5, 4, 9), (0, 4, 6, 5, 7), (0, 1, 6, 6, 7), (0, 1, 2, 2, 4), (0, 4, 8, 5, 7, 4, 6), (0, 1, 5, 2, 7), (0, 4, 8, 5, 2, 2), (0, 4, 8, 5, 0, 7), (0, 2, 1, 1), (0, 6, 7, 3, 7), (0, 4, 5, 9), (0, 4, 8, 5, 8, 3, 6), (0, 1, 2, 7, 9), (0, 1, 6, 5, 6), (0, 6, 0, 8, 6), (0, 4, 8, 5, 8, 6, 3), (0, 4, 8, 5, 7, 0, 9, 0), (0, 1, 6, 3, 1, 4), (0, 1, 5, 8, 4), (0, 2, 3, 7, 2), (0, 4, 8, 5, 8, 3, 5), (0, 4, 8, 5, 7, 4, 6), (0, 4, 8, 5, 4, 8, 1), (0, 4, 6, 2, 2), (0, 4, 8, 5, 7, 7, 5), (0, 9, 9, 2), (0, 4, 8, 5, 7, 7, 5), (0, 1, 5, 0, 1), (0, 2, 1, 6), (0, 1, 6, 9, 3), (0, 4, 8, 5, 7, 7, 5), (0, 1, 1, 3), (0, 4, 8, 5, 7, 0, 5), (0, 4, 8, 5, 5, 0), (0, 1, 5, 5, 3), (0, 4, 2, 1), (0, 4, 8, 5, 0, 3), (0, 4, 8, 4, 4, 4), (0, 4, 8, 3, 4, 0), (0, 5, 9), (0, 6, 7, 0, 1), (0, 4, 8, 5, 7, 9, 1), (0, 1, 6, 4, 3), (0, 4, 0, 6, 2), (0, 1, 6, 7, 3), (0, 1, 3, 1), (0, 0, 6, 6), (0, 8, 0), (0, 4, 8, 5, 8, 0), (0, 1, 6, 1, 4), (0, 1, 2, 6), (0, 1, 6, 7, 0, 6), (0, 1, 0, 2, 7), (0, 1, 2, 7, 6), (0, 1, 3, 9, 1), (0, 4, 8, 7, 8), (0, 1, 2, 2, 8), (0, 4, 8, 5, 7, 5, 7), (0, 1, 2, 7, 4), (0, 4, 8, 5, 7, 0, 9, 5), (0, 4, 5, 4, 1, 2), (0, 1, 6, 5, 3), (0, 1, 2, 5, 3), (0, 4, 8, 5, 7, 8, 8), (0, 1, 5, 7), (0, 1, 9, 5), (0, 1, 5, 8, 0), (0, 1, 5, 5, 4), (0, 1, 0, 7), (0, 4, 0, 9), (0, 1, 5, 0, 2), (0, 1, 4, 1), (0, 1, 9, 3), (0, 4, 8, 3, 2), (0, 4, 8, 5, 7, 7, 5), (0, 2, 8, 4), (0, 4, 8, 5, 7, 6, 2), (0, 6, 9, 2), (0, 6, 0, 7, 3), (0, 6, 1, 0), (0, 2, 8, 4), (0, 4, 8, 2, 8), (0, 4, 8, 5, 7, 6, 1), (0, 6, 8, 2), (0, 4, 8, 5, 7, 8, 3), (0, 6, 3, 3, 7), (0, 2, 2, 0), (0, 4, 8, 0, 1), (0, 4, 6, 8, 7), (0, 1, 0, 2, 7), (0, 1, 6, 9, 9), (0, 6, 0, 3, 6), (0, 1, 2, 3), (0, 4, 8, 5, 7, 4, 3), (0, 1, 4, 8), (0, 4, 7, 5), (0, 6, 5, 3), (0, 4, 8, 8, 7), (0, 1, 3, 9, 0), (0, 4, 8, 5, 1, 1, 6), (0, 4, 4, 1, 1, 9), (0, 1, 4, 9), (0, 1, 2, 5, 7, 0), (0, 9, 3, 2, 0), (0, 4, 6, 8, 2), (0, 6, 7, 9), (0, 4, 8, 0, 9, 1, 1), (0, 1, 3, 5), (0, 0, 0, 8), (0, 6, 7, 8), (0, 4, 8, 8, 1), (0, 6, 0, 6), (0, 1, 6, 3, 7), (0, 4, 6, 3), (0, 5, 3, 8), (0, 4, 5, 7, 1), (0, 4, 8, 3, 8), (0, 6, 1, 3), (0, 4, 8, 5, 1, 6), (0, 4, 8, 2, 4, 3), (0, 4, 6, 9), (0, 2, 8, 3), (0, 1, 5, 5, 7), (0, 9, 3, 2, 1), (0, 4, 8, 8, 5), (0, 4, 8, 9, 4, 2), (0, 4, 2, 4, 3, 7), (0, 1, 2, 5, 3), (0, 4, 8, 5, 7, 1, 3), (0, 1, 0, 6), (0, 4, 8, 4, 2), (0, 4, 0, 3, 4), (0, 1, 1, 0), (0, 1, 4, 3, 2), (0, 4, 0, 1, 9), (0, 4, 8, 0, 0, 1), (0, 1, 2, 5, 8), (0, 2, 9, 9), (0, 1, 9, 3), (0, 1, 2, 5, 2), (0, 5, 1, 1), (0, 2, 3, 8, 8), (0, 0, 7, 1), (0, 4, 8, 5, 7, 7, 1), (0, 1, 2, 5, 6), (0, 1, 9, 4), (0, 1, 6, 4, 2), (0, 5, 8, 0), (0, 1, 6, 3, 5), (0, 6, 1, 4), (0, 1, 6, 4, 4), (0, 1, 2, 7, 6), (0, 4, 6, 5, 2), (0, 5, 7, 6), (0, 4, 8, 5, 7, 4, 5), (0, 1, 0, 4), (0, 1, 5, 3), (0, 9, 8, 7), (0, 1, 9, 6), (0, 4, 2, 0), (0, 6, 5, 2), (0, 1, 4, 6, 6), (0, 4, 8, 5, 7, 0, 2, 2), (0, 4, 3, 1, 0), (0, 4, 8, 8, 8, 1), (0, 7, 5, 1, 5), (0, 1, 1, 3), (0, 4, 0, 5, 4), (0, 1, 4, 3, 1), (0, 1, 7, 2), (0, 4, 8, 1, 5), (0, 1, 5, 8, 3), (0, 4, 8, 5, 7, 0, 6), (0, 1, 6, 7, 5, 1), (0, 4, 8, 7, 5), (0, 6, 8, 2), (0, 1, 5, 7), (0, 4, 2, 1), (0, 6, 3, 8), (0, 4, 2, 2, 1), (0, 1, 5, 7), (0, 4, 8, 3, 2), (0, 1, 0, 0), (0, 4, 8, 5, 7, 9, 8), (0, 4, 7, 3), (0, 1, 3, 9, 0), (0, 4, 8, 4, 6), (0, 4, 8, 2, 5, 9), (0, 4, 0, 1, 5, 0), (0, 6, 3, 3, 0), (0, 9, 1, 3), (0, 6, 0, 9, 2), (0, 4, 8, 2, 9), (0, 1, 7, 7, 7), (0, 2, 6, 9), (0, 1, 6, 2), (0, 4, 8, 2, 4, 2), (0, 1, 5, 8, 5), (0, 4, 4, 3), (0, 4, 6, 6, 3), (0, 0, 9, 6), (0, 4, 8, 2, 8), (0, 5, 3, 5), (0, 1, 0, 2, 5), (0, 1, 2, 5, 4), (0, 1, 1, 0), (0, 4, 6, 2, 5), (0, 4, 8, 5, 8, 3, 7), (0, 4, 8, 5, 7, 0, 2, 8), (0, 9, 3, 4), (0, 4, 0, 3, 2), (0, 1, 1, 4), (0, 1, 0, 3), (0, 4, 0, 1, 9), (0, 4, 4, 9), (0, 1, 6, 9, 8), (0, 1, 2, 7, 6), (0, 3, 3, 5), (0, 4, 8, 0, 1), (0, 4, 5, 9), (0, 4, 9, 8), (0, 4, 9, 4), (0, 6, 0, 5, 4), (0, 4, 8, 2, 2), (0, 4, 8, 2, 5, 7), (0, 6, 5, 8, 7), (0, 4, 8, 5, 7, 2, 6), (0, 6, 3, 4), (0, 5, 7, 0), (0, 1, 7, 4), (0, 4, 7, 8, 4), (0, 1, 7, 7, 1), (0, 4, 0, 1, 0), (0, 1, 5, 5, 0), (0, 1, 0, 2, 4), (0, 4, 5, 4, 1, 4, 1), (0, 4, 5, 4, 1, 2), (0, 1, 4, 5), (0, 7, 1, 0, 1), (0, 4, 8, 0, 0, 3), (0, 4, 8, 5, 8, 5, 7), (0, 1, 1, 7), (0, 4, 5, 9), (0, 4, 4, 0, 4), (0, 1, 4, 1), (0, 4, 9, 3), (0, 4, 9, 5, 0, 0, 9), (0, 4, 8, 8, 0, 3), (0, 4, 8, 5, 7, 0, 5), (0, 4, 8, 3, 2), (0, 5, 1, 6), (0, 4, 8, 0, 5, 6), (0, 1, 5, 8, 3), (0, 4, 4, 4, 8), (0, 4, 2, 4, 2), (0, 7, 1, 5, 3), (0, 4, 2, 4, 2), (0, 4, 9, 2, 4), (0, 4, 8, 5, 3, 4, 1), (0, 4, 8, 5, 7, 0, 0, 1), (0, 1, 7, 7, 1), (0, 1, 2, 5, 4), (0, 1, 5, 8, 2), (0, 4, 8, 6, 3), (0, 0, 3, 1), (0, 4, 5, 4, 2), (0, 4, 8, 1, 5), (0, 1, 6, 7, 4), (0, 4, 6, 2, 4), (0, 3, 7), (0, 0, 3, 7), (0, 0, 3, 7), (0, 6, 2, 3), (0, 1, 6, 3, 9), (0, 0, 3, 0), (0, 2, 3, 6, 5), (0, 0, 0, 4, 2), (0, 1, 5, 9), (0, 0, 3, 7), (0, 1, 2, 4, 1), (0, 4, 0, 1, 4), (0, 2, 2, 0), (0, 4, 8, 5, 8, 2), (0, 4, 8, 5, 1, 5), (0, 1, 2, 2, 2), (0, 1, 0, 2, 3), (0, 1, 4, 1), (0, 4, 8, 5, 7, 0, 3, 8), (0, 4, 8, 5, 5, 2), (0, 4, 6, 2, 3), (0, 4, 8, 5, 1, 1, 3), (0, 1, 0, 2, 9), (0, 0, 1, 1), (0, 1, 4, 0), (0, 4, 8, 9, 7), (0, 9, 7, 0), (0, 4, 8, 4, 4, 8), (0, 9, 5, 5), (0, 4, 8, 9, 0), (0, 4, 8, 5, 7, 0, 2, 3), (0, 4, 2, 5), (0, 4, 8, 5, 0, 1), (0, 4, 2, 4, 3, 0), (0, 6, 5, 7), (0, 1, 5, 9), (0, 6, 1, 8), (0, 4, 8, 0, 9, 1, 1), (0, 4, 8, 8, 5), (0, 4, 8, 5, 8, 7), (0, 4, 2, 4, 3, 2), (0, 1, 3, 9, 2), (0, 4, 8, 4, 4, 4), (0, 1, 0, 2, 7), (0, 2, 6, 1), (0, 4, 8, 5, 7, 6, 0), (0, 4, 0, 1, 4), (0, 6, 0, 8, 7), (0, 1, 5, 4), (0, 9, 3, 5), (0, 1, 6, 7, 3), (0, 0, 9, 3), (0, 4, 8, 5, 2, 4), (0, 2, 3, 8, 8), (0, 0, 6, 1), (0, 4, 0, 1, 7), (0, 4, 8, 5, 9, 1), (0, 4, 8, 5, 7, 0, 9, 5), (0, 1, 4, 0), (0, 4, 8, 5, 7, 1, 1), (0, 4, 8, 5, 1, 0, 9), (0, 4, 8, 0, 9, 6), (0, 1, 4, 5), (0, 6, 1, 3), (0, 1, 9, 8), (0, 1, 1, 6), (0, 4, 8, 0, 9, 1, 4), (0, 1, 8, 1, 3), (0, 1, 6, 7, 2), (0, 1, 2, 3), (0, 6, 9, 0), (0, 4, 8, 2, 3), (0, 1, 2, 5, 7, 4), (0, 0, 6, 8), (0, 4, 8, 0, 8, 4), (0, 0, 9, 2, 6), (0, 1, 4, 6, 2), (0, 4, 8, 5, 7, 4, 5), (0, 4, 8, 8, 5), (0, 4, 8, 8, 9), (0, 1, 5, 4), (0, 1, 6, 4, 3), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 8, 6, 1), (0, 5, 0, 2, 5), (0, 4, 6, 9), (0, 4, 8, 5, 8, 8), (0, 9, 7, 6), (0, 1, 1, 3), (0, 4, 0, 1, 1), (0, 2, 3, 6, 4), (0, 4, 8, 5, 9, 4), (0, 1, 1, 8), (0, 6, 0, 1, 3), (0, 6, 7, 3, 0), (0, 4, 8, 5, 8, 2), (0, 4, 8, 5, 8, 5, 9), (0, 1, 8, 3), (0, 3, 9), (0, 4, 8, 5, 7, 8, 3), (0, 1, 0, 8), (0, 6, 1, 3), (0, 1, 2, 4, 3), (0, 4, 0, 3, 3), (0, 4, 8, 5, 2, 8), (0, 4, 8, 5, 6, 0), (0, 4, 3, 9), (0, 4, 8, 4, 4, 2), (0, 4, 6, 6, 0), (0, 4, 6, 5, 0), (0, 4, 8, 5, 9, 4), (0, 2, 6, 5), (0, 1, 5, 9), (0, 4, 6, 5, 7), (0, 4, 6, 5, 0), (0, 4, 2, 2, 0), (0, 1, 0, 3), (0, 4, 6, 5, 9), (0, 4, 0, 6, 2), (0, 4, 8, 4, 8), (0, 4, 8, 5, 8, 3, 4), (0, 4, 8, 2, 4, 9), (0, 4, 6, 5, 0), (0, 4, 8, 9, 5), (0, 9, 0, 1, 7), (0, 2, 9, 9), (0, 2, 8, 3), (0, 4, 8, 5, 7, 8, 7), (0, 1, 5, 9), (0, 7, 1, 3), (0, 4, 6, 3), (0, 5, 9), (0, 4, 8, 5, 7, 4, 1), (0, 4, 8, 5, 7, 2, 1), (0, 4, 8, 5, 2, 5), (0, 6, 8, 3), (0, 1, 3, 3), (0, 1, 6, 4, 2), (0, 1, 2, 7, 4), (0, 4, 0, 0, 6), (0, 4, 6, 6, 6), (0, 6, 9, 7, 5), (0, 0, 0, 4, 0), (0, 7, 7, 2), (0, 6, 9, 2), (0, 1, 2, 5, 4), (0, 0, 0, 2), (0, 4, 8, 5, 9, 9), (0, 1, 4, 6, 7), (0, 4, 6, 0, 7), (0, 1, 6, 7, 5, 6), (0, 1, 6, 8), (0, 4, 5, 4, 9), (0, 1, 2, 5, 2), (0, 1, 0, 3), (0, 4, 0, 2, 3), (0, 4, 0, 5, 5), (0, 4, 0, 1, 5, 7), (0, 9, 4, 1), (0, 1, 2, 7, 1), (0, 4, 9, 5, 9), (0, 4, 5, 4, 2), (0, 7, 1, 1), (0, 4, 8, 3, 4, 2), (0, 4, 0, 6, 3, 5), (0, 4, 8, 9, 3), (0, 4, 8, 3, 4, 2), (0, 4, 8, 5, 7, 9, 1), (0, 0, 3, 9), (0, 4, 6, 1, 7), (0, 6, 8, 1), (0, 4, 8, 9, 0), (0, 6, 1, 1, 1), (0, 4, 8, 5, 0, 4), (0, 6, 6, 5), (0, 2, 3, 7, 3), (0, 6, 7, 3, 4), (0, 0, 4, 7), (0, 0, 3, 2), (0, 0, 0, 1), (0, 4, 8, 5, 8, 4, 7), (0, 6, 7, 0, 1), (0, 6, 7, 8), (0, 4, 8, 2, 4, 2), (0, 4, 8, 1, 6), (0, 4, 0, 2, 0), (0, 5, 3, 5), (0, 6, 1, 4), (0, 4, 8, 6, 4, 7), (0, 2, 0, 6), (0, 4, 6, 9), (0, 4, 8, 1, 5), (0, 5, 3, 4), (0, 7, 1, 5, 5), (0, 1, 8, 3), (0, 1, 2, 2, 3), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 8, 1), (0, 4, 8, 5, 7, 1, 1), (0, 4, 8, 0, 5, 3), (0, 4, 8, 5, 7, 1, 1), (0, 6, 9, 8, 6), (0, 0, 2, 9), (0, 1, 6, 7, 0, 6), (0, 1, 3, 8), (0, 4, 5, 4, 4), (0, 4, 5, 4, 1, 6), (0, 4, 8, 3, 1), (0, 1, 5, 4), (0, 4, 8, 5, 7, 4, 1), (0, 9, 7, 1), (0, 4, 8, 5, 7, 2, 0), (0, 9, 3, 2, 3), (0, 1, 0, 2, 4), (0, 4, 8, 3, 4, 1), (0, 6, 2, 9), (0, 1, 6, 7, 9), (0, 4, 8, 5, 7, 9, 9), (0, 1, 2, 6), (0, 1, 4, 7), (0, 9, 2, 8), (0, 9, 2, 7), (0, 4, 8, 5, 8, 5, 5), (0, 0, 1, 9, 1), (0, 4, 0, 5, 6), (0, 5, 8, 9), (0, 1, 0, 2, 1), (0, 4, 8, 5, 2, 6), (0, 4, 8, 5, 2, 6), (0, 5, 3, 5), (0, 1, 6, 7, 5, 3), (0, 9, 3, 4), (0, 6, 6, 7), (0, 4, 5, 7, 6), (0, 4, 6, 0, 5), (0, 4, 8, 5, 1, 0, 1), (0, 1, 5, 7), (0, 4, 8, 5, 2, 1), (0, 4, 8, 5, 7, 9, 1), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 6, 4, 2), (0, 4, 5, 9), (0, 1, 4, 7), (0, 4, 7, 3), (0, 6, 0, 9, 0), (0, 6, 0, 9, 0), (0, 6, 0, 9, 2), (0, 6, 0, 9, 9), (0, 4, 9, 2, 3), (0, 6, 0, 9, 2), (0, 6, 0, 9, 2), (0, 4, 8, 1, 4), (0, 1, 6, 6, 1), (0, 4, 8, 5, 8, 2), (0, 4, 8, 5, 1, 2), (0, 4, 8, 5, 8, 5, 5), (0, 4, 6, 5, 0), (0, 6, 0, 9, 2), (0, 4, 8, 5, 8, 7), (0, 4, 8, 5, 5, 5), (0, 4, 5, 3), (0, 1, 6, 9, 5), (0, 9, 0, 4), (0, 1, 5, 6, 7), (0, 4, 2, 0), (0, 9, 3, 2, 0), (0, 9, 2, 7), (0, 9, 6, 3), (0, 9, 6, 3), (0, 9, 9, 8), (0, 1, 2, 5, 3), (0, 9, 6, 3), (0, 1, 9, 2), (0, 4, 3, 1, 4), (0, 1, 7, 7, 7), (0, 4, 8, 5, 7, 3, 1), (0, 4, 8, 5, 7, 3, 3), (0, 6, 0, 3, 9), (0, 1, 1, 3), (0, 4, 8, 5, 7, 9, 1), (0, 1, 6, 1, 9), (0, 4, 3, 1, 8), (0, 1, 6, 6, 9), (0, 4, 8, 5, 3, 2), (0, 4, 8, 5, 1, 6), (0, 6, 6, 3), (0, 6, 7, 5, 9), (0, 1, 6, 3, 1, 9), (0, 4, 8, 5, 1, 0, 0), (0, 4, 8, 5, 7, 8, 7), (0, 7, 9, 2), (0, 4, 8, 3, 9), (0, 1, 0, 4), (0, 4, 7, 8, 2), (0, 4, 8, 5, 8, 5, 5), (0, 1, 5, 3), (0, 2, 3, 8, 3), (0, 4, 8, 5, 7, 9, 9), (0, 4, 8, 6, 4, 1), (0, 4, 8, 6, 4, 1), (0, 4, 8, 6, 4, 1), (0, 4, 8, 6, 4, 1), (0, 4, 8, 5, 7, 0, 9, 1), (0, 4, 8, 5, 7, 2, 7), (0, 4, 8, 4, 3), (0, 4, 6, 1, 0), (0, 0, 4, 4), (0, 0, 8, 8), (0, 4, 8, 5, 7, 6, 9), (0, 4, 8, 8, 0, 1), (0, 4, 8, 2, 0), (0, 4, 8, 2, 0), (0, 0, 0, 4, 9), (0, 1, 8, 1, 3), (0, 4, 6, 0, 2), (0, 4, 8, 5, 9, 5), (0, 6, 8, 9), (0, 6, 0, 9, 2), (0, 6, 6, 4, 7), (0, 4, 2, 0), (0, 4, 0, 5, 3), (0, 4, 8, 5, 4, 8, 1), (0, 4, 3, 3), (0, 4, 8, 8, 1), (0, 4, 8, 3, 4, 6), (0, 0, 9, 3), (0, 1, 4, 6, 1), (0, 4, 0, 1, 7), (0, 4, 8, 2, 0), (0, 4, 0, 0, 1), (0, 4, 8, 5, 7, 0, 1), (0, 4, 0, 1, 2), (0, 1, 3, 8), (0, 4, 8, 5, 3, 6), (0, 4, 7, 1), (0, 4, 2, 3), (0, 4, 2, 7, 9), (0, 4, 8, 5, 7, 1, 6), (0, 1, 5, 8, 4), (0, 1, 2, 5, 7, 1), (0, 4, 8, 5, 0, 6), (0, 4, 8, 5, 1, 2), (0, 4, 4, 7), (0, 1, 0, 2, 7), (0, 1, 6, 7, 1), (0, 1, 0, 2, 4), (0, 4, 8, 5, 8, 4, 3), (0, 4, 0, 0, 1), (0, 1, 7, 7, 4), (0, 9, 3, 2, 3), (0, 1, 6, 4, 0), (0, 6, 0, 3, 0), (0, 1, 7, 7, 0), (0, 1, 6, 1, 1), (0, 4, 8, 5, 0, 9), (0, 1, 0, 5), (0, 6, 0, 1, 6), (0, 6, 6, 6), (0, 4, 8, 5, 1, 8), (0, 4, 8, 5, 5, 3), (0, 9, 3, 5), (0, 5, 8, 1), (0, 1, 5, 3), (0, 0, 0, 1), (0, 2, 7, 2), (0, 4, 5, 2, 4), (0, 4, 8, 1, 9), (0, 4, 0, 8, 5), (0, 4, 7, 8, 5), (0, 2, 7, 2), (0, 4, 6, 1, 8), (0, 0, 7, 3), (0, 1, 6, 7, 2), (0, 9, 8, 3), (0, 4, 2, 7, 1, 1), (0, 4, 8, 8, 8, 0), (0, 4, 8, 1, 8), (0, 4, 4, 4, 8), (0, 4, 0, 5, 2), (0, 4, 0, 0, 4), (0, 4, 8, 5, 7, 9, 3), (0, 1, 2, 5, 1), (0, 4, 8, 5, 7, 0, 3, 5), (0, 1, 3, 5), (0, 4, 8, 0, 1), (0, 4, 8, 5, 3, 1), (0, 1, 0, 2, 2), (0, 4, 8, 0, 4), (0, 4, 8, 5, 7, 9, 4), (0, 0, 9, 9), (0, 4, 8, 4, 4, 0), (0, 4, 9, 1), (0, 4, 8, 5, 7, 8, 7), (0, 6, 7, 4, 1), (0, 1, 1, 0), (0, 4, 8, 1, 1), (0, 2, 1, 6), (0, 0, 0, 0), (0, 5, 8, 9), (0, 4, 8, 5, 7, 7, 4), (0, 6, 7, 8), (0, 4, 6, 0, 8), (0, 0, 9, 6), (0, 4, 8, 5, 7, 0, 9, 0), (0, 1, 2, 2, 8), (0, 6, 0, 6), (0, 4, 8, 5, 7, 0, 3, 9), (0, 6, 7, 4, 1), (0, 4, 8, 3, 2), (0, 0, 0, 3), (0, 1, 4, 6, 1), (0, 1, 4, 6, 3), (0, 4, 8, 8, 8, 1), (0, 7, 9, 8), (0, 4, 6, 3), (0, 7, 9, 8), (0, 4, 8, 5, 7, 0, 3, 9), (0, 1, 5, 5, 3), (0, 4, 8, 5, 7, 8, 7), (0, 4, 5, 2, 3), (0, 4, 8, 5, 7, 7, 3), (0, 2, 8, 9), (0, 6, 0, 5, 2), (0, 4, 6, 1, 7), (0, 4, 8, 5, 5, 1), (0, 4, 8, 0, 6), (0, 4, 8, 1, 6), (0, 1, 6, 8), (0, 4, 8, 5, 7, 1, 1), (0, 7, 1, 6, 4), (0, 0, 1, 3), (0, 4, 8, 3, 9), (0, 4, 5, 4, 1, 6), (0, 4, 8, 5, 7, 0, 9, 2), (0, 4, 8, 9, 6), (0, 5, 0, 2, 1, 1), (0, 4, 2, 2, 7), (0, 1, 6, 7, 0, 6), (0, 2, 9, 9), (0, 4, 7, 2), (0, 4, 0, 4, 8), (0, 6, 0, 1, 6), (0, 4, 8, 4, 4, 4), (0, 4, 6, 5, 7), (0, 4, 8, 5, 8, 5, 5), (0, 1, 2, 1), (0, 5, 7, 0), (0, 4, 8, 5, 7, 3, 4), (0, 4, 0, 2, 2), (0, 5, 0, 2, 9), (0, 4, 8, 2, 4, 2), (0, 9, 7, 3), (0, 4, 6, 1, 5), (0, 4, 0, 1, 0), (0, 6, 6, 7), (0, 4, 6, 7), (0, 0, 9, 6), (0, 4, 8, 5, 7, 8, 0), (0, 1, 2, 4, 1), (0, 4, 8, 3, 2), (0, 6, 7, 6), (0, 4, 8, 3, 2), (0, 4, 8, 4, 6), (0, 1, 2, 7, 2), (0, 4, 8, 7, 2), (0, 4, 8, 2, 1, 1), (0, 4, 8, 5, 4, 3), (0, 4, 2, 9), (0, 1, 6, 8), (0, 1, 0, 2, 1), (0, 6, 0, 9, 0), (0, 4, 9, 2, 3), (0, 4, 8, 2, 1, 5), (0, 4, 8, 5, 1, 7), (0, 4, 2, 7, 1, 1), (0, 4, 8, 5, 5, 8), (0, 4, 8, 5, 7, 6, 7), (0, 4, 8, 5, 6, 0), (0, 4, 8, 5, 5, 8), (0, 9, 0, 7), (0, 2, 3, 8, 4), (0, 2, 1, 6), (0, 2, 3, 7, 3), (0, 4, 8, 5, 1, 0, 3), (0, 1, 6, 7, 5, 4), (0, 5, 0, 1), (0, 1, 7, 8), (0, 4, 5, 9), (0, 4, 8, 7, 5), (0, 4, 2, 2, 0), (0, 6, 0, 7, 2), (0, 7, 3, 0), (0, 4, 8, 5, 7, 0, 3, 2), (0, 4, 8, 5, 9, 7), (0, 4, 2, 4, 3, 0), (0, 1, 4, 6, 4), (0, 1, 5, 4), (0, 4, 2, 4, 4), (0, 1, 7, 8), (0, 4, 6, 6, 1), (0, 4, 8, 5, 7, 8, 5), (0, 4, 8, 5, 5, 8), (0, 4, 8, 5, 5, 8), (0, 4, 8, 5, 3, 0), (0, 6, 7, 6), (0, 4, 8, 5, 5, 2), (0, 1, 2, 2, 8), (0, 4, 8, 5, 1, 0, 6), (0, 4, 8, 5, 5, 2), (0, 4, 8, 5, 5, 2), (0, 1, 7, 1), (0, 4, 8, 5, 1, 0, 1), (0, 5, 0, 2, 1, 1), (0, 4, 8, 5, 7, 9, 9), (0, 4, 7, 8, 5), (0, 7, 5, 9), (0, 1, 6, 3, 1, 7), (0, 6, 0, 5, 7), (0, 4, 8, 4, 4, 4), (0, 4, 7, 8, 8), (0, 5, 0, 2, 1, 1), (0, 4, 8, 8, 4), (0, 6, 2, 2), (0, 4, 8, 5, 3, 0), (0, 4, 0, 1, 4), (0, 4, 8, 2, 9), (0, 4, 8, 5, 5, 7), (0, 6, 2, 2), (0, 4, 8, 5, 1, 0, 4), (0, 6, 2, 2), (0, 4, 6, 6, 1), (0, 4, 0, 5, 4), (0, 4, 8, 5, 3, 4, 1), (0, 6, 0, 3, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 4), (0, 6, 5, 8, 4), (0, 1, 6, 3, 1, 9), (0, 0, 0, 1), (0, 4, 8, 5, 7, 0, 2, 6), (0, 4, 8, 5, 1, 7), (0, 1, 6, 9, 8), (0, 4, 8, 5, 7, 7, 3), (0, 1, 8, 4), (0, 4, 5, 9), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 3, 3), (0, 6, 9, 8, 1), (0, 5, 1, 1), (0, 0, 0, 5, 2), (0, 1, 5, 4), (0, 4, 0, 6, 3, 6), (0, 4, 0, 8, 4), (0, 6, 3, 7), (0, 4, 8, 5, 8, 9), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 3, 0), (0, 1, 6, 7, 6), (0, 9, 6, 6), (0, 4, 7, 0), (0, 7, 4, 1, 3), (0, 0, 7, 3), (0, 4, 8, 5, 0, 9), (0, 4, 8, 5, 7, 0, 2, 1), (0, 5, 8, 1), (0, 4, 8, 5, 7, 0, 4), (0, 4, 8, 0, 5, 6), (0, 4, 5, 3), (0, 4, 8, 5, 7, 0, 9, 4), (0, 4, 8, 2, 6), (0, 4, 8, 2, 0), (0, 1, 0, 2, 5), (0, 4, 9, 5, 0, 3), (0, 4, 8, 2, 6), (0, 1, 6, 7, 0, 3), (0, 5, 7, 3), (0, 5, 7, 0), (0, 5, 7, 0), (0, 4, 8, 5, 7, 0, 6), (0, 7, 5, 6), (0, 1, 6, 7, 3), (0, 4, 3, 8), (0, 4, 8, 5, 7, 4, 5), (0, 7, 5, 2), (0, 7, 5, 6), (0, 7, 5, 1, 7), (0, 7, 4, 2), (0, 6, 0, 9, 9), (0, 0, 8, 3), (0, 2, 0, 0), (0, 6, 5, 8, 4), (0, 1, 2, 6), (0, 1, 3, 0), (0, 7, 3, 3), (0, 4, 8, 2, 1, 8), (0, 5, 4, 0), (0, 4, 8, 5, 8, 8), (0, 6, 0, 1, 3), (0, 4, 8, 5, 7, 0, 8), (0, 7, 1, 9), (0, 7, 4, 2), (0, 4, 7, 8, 4), (0, 4, 8, 5, 2, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 9, 0), (0, 7, 3, 3), (0, 4, 0, 3, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 6, 1, 1, 3), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 2), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 2, 7, 5), (0, 1, 2, 0), (0, 4, 0, 1, 4), (0, 4, 5, 4, 9), (0, 7, 5, 0), (0, 2, 3, 2, 7), (0, 4, 7, 6, 2), (0, 1, 1, 6), (0, 6, 6, 7), (0, 4, 7, 8, 5), (0, 4, 8, 5, 3, 5), (0, 4, 0, 0, 5), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 9), (0, 7, 4, 1, 1), (0, 7, 4, 9), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 9, 3), (0, 4, 7, 6, 1), (0, 1, 6, 7, 9), (0, 1, 6, 3, 6), (0, 6, 5, 8, 6), (0, 4, 8, 5, 7, 0, 1), (0, 4, 6, 9), (0, 4, 5, 4, 1, 4, 6), (0, 4, 5, 4, 1, 4, 1), (0, 4, 5, 8, 1), (0, 4, 8, 4, 4, 4), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 9, 2, 1), (0, 4, 8, 5, 1, 0, 6), (0, 4, 8, 9, 6), (0, 7, 4, 2), (0, 4, 8, 5, 7, 5, 2), (0, 4, 8, 5, 8, 5, 1), (0, 0, 0, 6), (0, 4, 5, 4, 1, 6), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 2), (0, 7, 3, 3), (0, 7, 3, 3), (0, 7, 3, 0), (0, 6, 7, 2), (0, 1, 2, 5, 7, 5), (0, 6, 1, 7), (0, 5, 9), (0, 6, 3, 1), (0, 1, 0, 4), (0, 7, 3, 6), (0, 2, 3, 8, 8), (0, 1, 6, 9, 3), (0, 4, 0, 1, 2), (0, 4, 0, 6, 1), (0, 7, 4, 8), (0, 4, 6, 1, 3), (0, 4, 0, 3, 2), (0, 4, 0, 2, 4), (0, 4, 8, 5, 1, 6), (0, 7, 4, 2), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 2), (0, 1, 6, 7, 4), (0, 0, 6, 1), (0, 1, 2, 2, 8), (0, 4, 8, 5, 7, 1, 4), (0, 6, 1, 7), (0, 4, 8, 3, 2), (0, 1, 0, 2, 4), (0, 4, 8, 5, 1, 0, 8), (0, 0, 9, 6), (0, 4, 4, 0, 9), (0, 4, 8, 5, 0, 6), (0, 1, 2, 0), (0, 7, 3, 3), (0, 5, 3, 4), (0, 4, 8, 3, 9), (0, 4, 8, 8, 9), (0, 0, 7, 3), (0, 1, 5, 3), (0, 4, 0, 8, 8), (0, 4, 0, 3, 3), (0, 4, 8, 5, 3, 1), (0, 4, 7, 2), (0, 4, 8, 2, 9), (0, 5, 7, 5), (0, 7, 4, 9), (0, 7, 4, 2), (0, 1, 6, 7, 2), (0, 4, 8, 5, 8, 5, 2), (0, 4, 7, 5), (0, 1, 6, 7, 3), (0, 4, 0, 3, 1), (0, 4, 8, 5, 3, 1), (0, 6, 1, 9), (0, 4, 8, 0, 5, 3), (0, 1, 6, 7, 6), (0, 4, 0, 3, 0), (0, 4, 0, 2, 8), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 3, 3), (0, 4, 3, 1, 1), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 9, 7), (0, 9, 3, 6), (0, 1, 5, 3), (0, 4, 4, 0, 2), (0, 0, 7, 0), (0, 1, 2, 4, 3), (0, 4, 8, 0, 8, 2), (0, 4, 8, 5, 6, 9), (0, 4, 8, 2, 5, 2), (0, 1, 2, 4, 1), (0, 1, 2, 5, 9), (0, 1, 2, 4, 1), (0, 7, 1, 0, 2), (0, 0, 9, 8), (0, 4, 8, 5, 7, 0, 2, 7), (0, 1, 9, 0, 0), (0, 4, 8, 5, 7, 0, 9, 7), (0, 4, 6, 9), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 8, 5, 7, 8, 5), (0, 4, 8, 5, 1, 0, 3), (0, 9, 0, 1, 3), (0, 7, 4, 2), (0, 7, 3, 3), (0, 7, 3, 3), (0, 4, 8, 5, 7, 8, 2), (0, 4, 0, 2, 2), (0, 9, 3, 2, 1), (0, 7, 4, 2), (0, 4, 7, 2), (0, 9, 3, 9), (0, 0, 9, 2, 1), (0, 1, 5, 6, 7), (0, 3, 4), (0, 1, 1, 2), (0, 4, 5, 1), (0, 3, 4), (0, 4, 0, 1, 2), (0, 9, 2, 3), (0, 3, 4), (0, 4, 5, 3), (0, 1, 9, 1), (0, 4, 8, 8, 1), (0, 4, 8, 4, 4, 3), (0, 6, 0, 5, 6), (0, 0, 4, 4), (0, 0, 9, 2, 0), (0, 0, 4, 7), (0, 4, 8, 2, 4, 2), (0, 4, 8, 0, 8, 2), (0, 4, 0, 3, 0), (0, 4, 5, 4, 9), (0, 1, 7, 5), (0, 4, 5, 4, 9), (0, 4, 8, 4, 5), (0, 6, 1, 7), (0, 7, 5, 1, 0), (0, 4, 4, 7), (0, 9, 3, 4), (0, 4, 8, 9, 9), (0, 4, 8, 5, 8, 3, 4), (0, 6, 0, 3, 7), (0, 6, 5, 3), (0, 4, 8, 5, 9, 3), (0, 4, 5, 1), (0, 7, 3, 3), (0, 7, 3, 3), (0, 7, 3, 3), (0, 7, 3, 3), (0, 7, 3, 3), (0, 7, 3, 3), (0, 6, 1, 3), (0, 5, 0, 2, 1, 0), (0, 6, 0, 7, 4), (0, 4, 6, 5, 1), (0, 4, 5, 1), (0, 4, 5, 1), (0, 9, 8, 3), (0, 4, 5, 1), (0, 4, 5, 1), (0, 7, 3, 5), (0, 7, 4, 5), (0, 2, 3, 8, 8), (0, 2, 2, 5), (0, 7, 4, 5), (0, 4, 8, 0, 0, 0), (0, 2, 3, 8, 8), (0, 7, 4, 2), (0, 7, 6), (0, 4, 8, 5, 7, 0, 3, 1), (0, 1, 2, 3), (0, 4, 8, 0, 2), (0, 1, 6, 4, 2), (0, 1, 6, 7, 9), (0, 6, 7, 0, 2), (0, 0, 0, 5, 5), (0, 7, 4, 5), (0, 0, 3, 1), (0, 4, 8, 4, 8), (0, 4, 8, 8, 1), (0, 1, 9, 4), (0, 4, 5, 1), (0, 6, 7, 3, 9), (0, 4, 8, 0, 7), (0, 2, 7, 7), (0, 2, 3, 8, 1), (0, 4, 8, 5, 0, 4), (0, 6, 7, 0, 5), (0, 0, 2, 7), (0, 4, 7, 4), (0, 4, 9, 1), (0, 4, 8, 5, 3, 3), (0, 4, 8, 5, 8, 2), (0, 4, 8, 5, 8, 5, 3), (0, 1, 9, 8), (0, 1, 4, 6, 5), (0, 4, 8, 5, 7, 0, 1), (0, 4, 8, 5, 8, 6, 8), (0, 0, 9, 6), (0, 4, 8, 5, 3, 4, 3), (0, 1, 0, 3), (0, 1, 4, 3, 6), (0, 4, 5, 8, 2), (0, 0, 2, 4), (0, 0, 2, 7), (0, 2, 3, 8, 1), (0, 4, 8, 8, 8, 6), (0, 6, 3, 2), (0, 4, 8, 5, 7, 8, 9), (0, 4, 6, 1, 1, 1), (0, 4, 0, 6, 2), (0, 7, 4, 2), (0, 7, 4, 0), (0, 7, 4, 7), (0, 7, 4, 7), (0, 1, 0, 2, 4), (0, 4, 8, 5, 9, 4), (0, 4, 0, 1, 5, 0), (0, 4, 7, 8, 8), (0, 1, 4, 6, 9), (0, 4, 0, 9), (0, 7, 3, 3), (0, 1, 6, 6, 8), (0, 0, 5), (0, 6, 0, 1, 1), (0, 4, 8, 5, 0, 2), (0, 1, 6, 1, 1), (0, 4, 8, 0, 8, 7), (0, 4, 7, 6, 3), (0, 5, 1, 3), (0, 0, 0, 8), (0, 4, 6, 1, 1, 1), (0, 6, 9, 9), (0, 7, 1, 6, 3), (0, 4, 8, 5, 3, 5), (0, 4, 8, 0, 8, 8), (0, 4, 8, 5, 7, 0, 6), (0, 7, 1, 0, 1), (0, 4, 8, 8, 0, 3), (0, 7, 4, 5), (0, 4, 8, 8, 8, 0), (0, 2, 3, 6, 7), (0, 4, 9, 5, 0, 0, 0), (0, 4, 9, 5, 0, 0, 1), (0, 4, 8, 8, 5), (0, 4, 8, 0, 2), (0, 4, 9, 5, 0, 0, 0), (0, 4, 8, 0, 0, 2), (0, 1, 6, 7, 0, 5), (0, 6, 8, 0), (0, 4, 7, 8, 5), (0, 4, 5, 8, 0), (0, 4, 5, 8, 2), (0, 4, 5, 8, 2), (0, 2, 3, 2, 1), (0, 4, 5, 8, 2), (0, 4, 5, 6, 3), (0, 4, 8, 5, 3, 4, 1), (0, 1, 4, 1), (0, 4, 0, 5, 8), (0, 7, 4, 2), (0, 4, 5, 6, 3), (0, 4, 5, 4, 1, 2), (0, 4, 8, 6, 9), (0, 4, 7, 8, 4), (0, 1, 6, 7, 7), (0, 6, 1, 8), (0, 1, 8, 3), (0, 9, 4, 9), (0, 6, 1, 3), (0, 7, 4, 2), (0, 2, 6, 6), (0, 0, 4, 8), (0, 4, 8, 5, 5, 8), (0, 0, 7, 2), (0, 0, 2, 0), (0, 7, 9, 2), (0, 1, 2, 4, 9), (0, 4, 8, 2, 4, 5), (0, 4, 8, 8, 5), (0, 4, 7, 2), (0, 4, 8, 5, 3, 7), (0, 4, 8, 8, 1), (0, 1, 4, 7), (0, 4, 8, 5, 7, 3, 2), (0, 4, 5, 4, 9), (0, 4, 4, 0, 2), (0, 5, 8, 1), (0, 2, 9, 0), (0, 1, 6, 5, 6), (0, 1, 6, 6, 0), (0, 4, 8, 8, 6), (0, 7, 7, 2), (0, 4, 4, 1, 2), (0, 4, 0, 0, 3), (0, 1, 3, 5), (0, 5, 8, 1), (0, 4, 8, 7, 0), (0, 5, 8, 1), (0, 3, 8), (0, 4, 4, 3), (0, 4, 0, 1, 4), (0, 2, 0, 2), (0, 9, 3, 9), (0, 4, 0, 5, 4), (0, 0, 4, 4), (0, 6, 7, 0, 3), (0, 3, 8), (0, 4, 7, 4), (0, 3, 8), (0, 0, 7, 9), (0, 5, 4, 3), (0, 3, 8), (0, 6, 7, 3, 6), (0, 3, 8), (0, 4, 8, 9, 4, 1), (0, 1, 2, 2, 8), (0, 0, 0, 0), (0, 1, 4, 7), (0, 4, 8, 9, 4, 0), (0, 4, 8, 0, 8, 5), (0, 4, 8, 0, 8, 4), (0, 5, 0, 1), (0, 4, 8, 5, 7, 1, 5), (0, 2, 3, 8, 8), (0, 5, 0, 3), (0, 4, 5, 3), (0, 4, 8, 5, 8, 7), (0, 4, 8, 5, 8, 7), (0, 2, 3, 8, 8), (0, 1, 0, 2, 6), (0, 4, 0, 0, 3), (0, 4, 8, 8, 0, 1), (0, 7, 9, 2), (0, 4, 5, 3), (0, 6, 3, 6), (0, 4, 0, 3, 5), (0, 6, 3, 2), (0, 6, 3, 2), (0, 1, 5, 6, 8), (0, 6, 3, 6), (0, 6, 3, 6), (0, 6, 3, 6), (0, 1, 3, 2), (0, 6, 2, 6), (0, 4, 8, 9, 4, 9), (0, 6, 5, 1), (0, 6, 0, 9, 0), (0, 4, 8, 5, 0, 9), (0, 7, 4, 4), (0, 7, 4, 4), (0, 7, 1, 0, 3), (0, 4, 8, 0, 6), (0, 4, 3, 1, 0), (0, 4, 8, 5, 8, 4, 1), (0, 4, 8, 5, 1, 9), (0, 4, 8, 4, 4, 4), (0, 7, 4, 4), (0, 4, 6, 1, 5), (0, 4, 6, 1, 5), (0, 2, 9, 1), (0, 4, 8, 5, 8, 4, 1), (0, 2, 3, 2, 0), (0, 4, 2, 1), (0, 6, 1, 0), (0, 7, 4, 4), (0, 2, 1, 6), (0, 9, 2, 8), (0, 4, 8, 5, 7, 3, 6), (0, 4, 8, 0, 1), (0, 0, 4, 3), (0, 4, 6, 9), (0, 4, 4, 0, 4), (0, 1, 9, 0, 8), (0, 4, 8, 5, 7, 0, 9, 0), (0, 4, 6, 3), (0, 7, 5, 9), (0, 4, 0, 4, 8), (0, 3, 9), (0, 3, 4), (0, 4, 8, 0, 3), (0, 2, 3, 2, 3), (0, 2, 8, 3), (0, 1, 6, 7, 7), (0, 4, 8, 8, 5), (0, 4, 8, 5, 2, 6), (0, 1, 2, 5, 2), (0, 1, 7, 7, 1), (0, 4, 8, 5, 8, 4, 9), (0, 6, 1, 0), (0, 4, 8, 2, 5, 6), (0, 9, 0, 0), (0, 1, 5, 3), (0, 4, 7, 8, 4), (0, 4, 8, 0, 8, 4), (0, 4, 5, 2, 1), (0, 4, 8, 0, 3), (0, 4, 5, 2, 1), (0, 4, 9, 1), (0, 4, 8, 5, 0, 4), (0, 4, 7, 6, 6), (0, 4, 8, 2, 1, 5), (0, 1, 5, 5, 2), (0, 4, 7, 4), (0, 4, 7, 4), (0, 4, 8, 3, 8), (0, 4, 8, 5, 7, 0, 0, 1), (0, 4, 0, 6, 0), (0, 4, 2, 2, 8), (0, 4, 5, 4, 3), (0, 4, 9, 5, 5), (0, 4, 5, 2, 1), (0, 4, 7, 1), (0, 4, 9, 4), (0, 7, 7, 9), (0, 1, 2, 5, 6), (0, 4, 6, 5, 7), (0, 4, 5, 2, 5), (0, 4, 5, 2, 1), (0, 4, 5, 2, 1), (0, 4, 5, 2, 1), (0, 6, 0, 0, 1), (0, 4, 5, 2, 1), (0, 4, 5, 2, 1), (0, 1, 3, 6), (0, 4, 7, 6, 6), (0, 4, 7, 8, 4), (0, 4, 8, 5, 7, 7, 2), (0, 4, 7, 4), (0, 4, 8, 5, 4, 8, 7), (0, 4, 0, 6, 4, 1), (0, 4, 5, 2, 1), (0, 7, 1, 0, 5), (0, 4, 8, 5, 7, 7, 4), (0, 4, 5, 2, 1), (0, 4, 7, 4), (0, 6, 9, 7, 1), (0, 1, 0, 6), (0, 1, 6, 5, 1), (0, 4, 0, 5, 6), (0, 4, 2, 1), (0, 1, 5, 6, 7), (0, 4, 8, 5, 7, 6, 0), (0, 4, 6, 6, 1), (0, 4, 8, 9, 3), (0, 2, 1, 6), (0, 4, 2, 7, 1, 0), (0, 4, 8, 5, 8, 2), (0, 6, 5, 1), (0, 4, 8, 4, 4, 1), (0, 4, 8, 4, 4, 1), (0, 4, 8, 5, 1, 2), (0, 4, 8, 5, 3, 9), (0, 6, 7, 2), (0, 5, 8, 0), (0, 4, 8, 5, 0, 6), (0, 6, 7, 4, 0), (0, 2, 6, 3), (0, 4, 8, 0, 0, 6), (0, 2, 6, 1), (0, 2, 0, 1), (0, 2, 1, 2), (0, 4, 0, 1, 5, 9), (0, 1, 9, 0, 5), (0, 5, 8, 1), (0, 5, 3, 9), (0, 4, 5, 1), (0, 4, 2, 4, 3, 7), (0, 9, 4, 5), (0, 2, 1, 1), (0, 4, 0, 1, 5, 8), (0, 6, 2, 7), (0, 4, 2, 0), (0, 4, 0, 6, 5), (0, 4, 8, 5, 7, 1, 1), (0, 1, 0, 7), (0, 1, 2, 5, 2), (0, 4, 7, 1), (0, 1, 6, 1, 2), (0, 1, 4, 0), (0, 9, 3, 6), (0, 7, 3, 0), (0, 0, 3, 7), (0, 2, 9, 1), (0, 5, 0, 7), (0, 4, 0, 4, 4), (0, 1, 6, 7, 0, 8), (0, 4, 0, 6, 3, 0), (0, 2, 9, 1), (0, 1, 5, 9), (0, 3, 8), (0, 3, 4), (0, 4, 8, 5, 7, 8, 7), (0, 7, 1, 3), (0, 2, 1, 6), (0, 2, 5, 1), (0, 2, 3, 8, 8), (0, 4, 9, 2, 2), (0, 2, 5, 1), (0, 4, 8, 0, 9, 3), (0, 4, 8, 3, 4, 6), (0, 4, 4, 1, 1, 0), (0, 0, 2, 0), (0, 4, 6, 9), (0, 4, 8, 1, 2), (0, 6, 0, 1, 6), (0, 4, 8, 5, 7, 4, 7), (0, 4, 0, 6, 3, 3), (0, 6, 1, 2, 9), (0, 1, 6, 7, 3), (0, 4, 8, 5, 2, 0), (0, 4, 5, 4, 3), (0, 1, 6, 7, 2), (0, 1, 2, 5, 2), (0, 4, 5, 4, 1, 2), (0, 1, 4, 5), (0, 4, 8, 5, 7, 1, 5), (0, 2, 3, 2, 1), (0, 9, 0, 7), (0, 2, 3, 2, 3), (0, 4, 8, 0, 8, 0), (0, 0, 1, 6), (0, 5, 6), (0, 4, 4, 4, 4), (0, 7, 7, 2), (0, 1, 2, 5, 8), (0, 4, 4, 4, 8), (0, 4, 4, 4, 4), (0, 4, 2, 7, 6), (0, 0, 1, 0), (0, 4, 4, 4, 4), (0, 4, 8, 5, 7, 5, 2), (0, 1, 4, 3, 5), (0, 4, 8, 5, 7, 4, 5), (0, 4, 4, 1, 1, 0), (0, 6, 0, 3, 7), (0, 4, 7, 0), (0, 2, 8, 4), (0, 2, 3, 2, 1), (0, 2, 1, 0), (0, 4, 8, 1, 6), (0, 4, 0, 6, 2), (0, 4, 8, 0, 0, 1), (0, 4, 2, 4, 6), (0, 5, 8, 0), (0, 4, 2, 2, 6), (0, 4, 9, 5, 0, 0, 5), (0, 2, 2, 9), (0, 4, 8, 5, 2, 6), (0, 2, 8, 0), (0, 1, 2, 7, 3), (0, 2, 1, 7), (0, 2, 1, 7), (0, 4, 6, 8, 1), (0, 4, 8, 0, 9, 5), (0, 4, 2, 7, 5), (0, 1, 6, 5, 5), (0, 4, 8, 5, 7, 0, 6), (0, 4, 0, 3, 0), (0, 2, 8, 2), (0, 4, 0, 0, 6), (0, 7, 4, 2), (0, 4, 8, 5, 2, 8), (0, 2, 8, 4), (0, 2, 1, 2), (0, 4, 5, 8, 1), (0, 4, 5, 8, 0), (0, 4, 5, 8, 0), (0, 0, 0, 3), (0, 4, 4, 2), (0, 1, 6, 7, 4), (0, 4, 4, 8), (0, 2, 3, 7, 0), (0, 4, 5, 8, 1), (0, 2, 0, 4), (0, 4, 8, 4, 1), (0, 2, 1, 5), (0, 5, 3, 3, 9), (0, 7, 3, 0), (0, 5, 0, 2, 1, 5), (0, 4, 8, 5, 3, 8), (0, 4, 8, 5, 3, 4, 9), (0, 6, 0, 4), (0, 4, 8, 5, 7, 0, 6), (0, 4, 4, 1, 1, 0), (0, 4, 8, 5, 7, 8, 3), (0, 4, 8, 5, 7, 0, 6), (0, 4, 8, 5, 4, 8, 1), (0, 4, 4, 4, 8), (0, 6, 7, 8), (0, 4, 5, 7, 1), (0, 4, 6, 5, 2), (0, 2, 9, 3), (0, 4, 8, 5, 4, 8, 1), (0, 4, 0, 0, 6), (0, 9, 3, 2, 7, 0), (0, 4, 8, 0, 4), (0, 9, 2, 8), (0, 9, 5, 1), (0, 1, 4, 3, 4), (0, 0, 3, 3), (0, 4, 8, 5, 7, 7, 5), (0, 4, 8, 4, 1), (0, 4, 5, 3), (0, 1, 2, 8, 2), (0, 4, 5, 3), (0, 4, 2, 6), (0, 4, 8, 2, 5, 1), (0, 4, 8, 2, 7), (0, 2, 1, 1), (0, 1, 6, 7, 3), (0, 6, 2, 5), (0, 4, 8, 0, 8, 0), (0, 6, 6, 7), (0, 2, 7, 2), (0, 5, 3, 3, 9), (0, 1, 7, 7, 6), (0, 1, 6, 7, 4), (0, 9, 8, 7), (0, 7, 0, 2), (0, 7, 0, 8), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 9, 2), (0, 7, 0, 0, 0, 0, 0, 0), (0, 4, 5, 4, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 7, 0, 4), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 4, 8, 5, 1, 9), (0, 7, 0, 1), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 5), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 9), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 7), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 7, 0, 1), (0, 7, 0, 1), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 3), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 4), (0, 7, 0, 4), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 1, 5, 3), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 6), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 4, 8, 8, 5), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 3), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 7, 0, 1), (0, 7, 0, 1), (0, 7, 0, 0, 0, 0, 0, 0), (0, 7, 0, 1), (0, 4, 5, 4, 1, 1), (0, 4, 4, 4, 3), (0, 4, 5, 6, 0), (0, 4, 6, 5, 0), (0, 4, 5, 4, 1, 4, 2), (0, 4, 5, 6, 6), (0, 4, 5, 6, 1), (0, 4, 5, 1), (0, 4, 5, 6, 6), (0, 4, 5, 3), (0, 9, 0, 2), (0, 4, 5, 6, 1), (0, 5, 8, 1), (0, 4, 8, 3, 1), (0, 4, 6, 3), (0, 4, 5, 6, 4), (0, 2, 8, 2), (0, 1, 3, 9, 5), (0, 4, 8, 6, 4, 1), (0, 4, 8, 5, 1, 2), (0, 9, 2, 9), (0, 4, 4, 4, 4), (0, 2, 3, 0), (0, 4, 8, 6, 9), (0, 2, 3, 2, 1), (0, 8, 3, 9), (0, 4, 5, 6, 2), (0, 4, 5, 6, 2), (0, 4, 4, 1, 9), (0, 2, 5, 2), (0, 4, 0, 6, 4, 3), (0, 4, 6, 9), (0, 4, 0, 2, 1), (0, 4, 4, 1, 1, 5), (0, 1, 2, 5, 7, 3), (0, 4, 6, 2, 5), (0, 2, 9, 7), (0, 1, 6, 7, 8), (0, 1, 4, 6, 1), (0, 4, 0, 8, 0), (0, 4, 8, 5, 3, 9), (0, 4, 8, 5, 7, 0, 7), (0, 1, 2, 3), (0, 4, 9, 5, 0, 7), (0, 1, 6, 7, 2), (0, 4, 9, 5, 0, 0, 2), (0, 4, 3, 4), (0, 4, 9, 2, 3), (0, 2, 3, 8, 9), (0, 4, 8, 0, 6), (0, 7, 1, 5, 2), (0, 3, 2, 0), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 7, 9, 6), (0, 3, 8), (0, 4, 8, 5, 7, 4, 6), (0, 3, 2, 0), (0, 3, 8), (0, 9, 3, 8), (0, 8, 5, 5), (0, 6, 7, 4, 1), (0, 4, 8, 5, 1, 7), (0, 3, 2, 0), (0, 3, 2, 0), (0, 0, 1, 4), (0, 4, 7, 1), (0, 9, 8, 4), (0, 0, 7, 8), (0, 1, 2, 5, 6), (0, 4, 8, 3, 9), (0, 3, 2, 0), (0, 4, 8, 5, 8, 3, 4), (0, 4, 8, 5, 7, 9, 0), (0, 9, 7, 0), (0, 4, 8, 5, 2, 3), (0, 4, 8, 5, 2, 8), (0, 3, 2, 0), (0, 3, 2, 0), (0, 4, 8, 5, 8, 4, 6), (0, 4, 5, 6, 2), (0, 4, 5, 8, 1), (0, 4, 5, 6, 2), (0, 4, 5, 7, 4), (0, 3, 2, 3), (0, 3, 2, 0), (0, 4, 5, 6, 5), (0, 4, 5, 6, 7), (0, 4, 5, 7, 0), (0, 3, 2, 2), (0, 3, 2, 4), (0, 7, 1, 6, 0), (0, 4, 5, 6, 9), (0, 4, 8, 3, 8), (0, 1, 0, 5), (0, 1, 5, 6, 7), (0, 3, 2, 0), (0, 3, 2, 0), (0, 3, 2, 0), (0, 3, 3, 9), (0, 3, 2, 0), (0, 3, 2, 1), (0, 3, 3, 1), (0, 3, 2, 1), (0, 3, 2, 1), (0, 1, 6, 1, 5), (0, 9, 3, 1), (0, 4, 8, 2, 5, 0), (0, 3, 2, 1), (0, 3, 2, 0), (0, 3, 2, 0), (0, 1, 5, 2, 9), (0, 4, 8, 9, 3), (0, 6, 0, 1, 7), (0, 7, 1, 5, 6), (0, 4, 8, 1, 2), (0, 4, 8, 1, 2), (0, 0, 8, 7), (0, 4, 5, 7, 3), (0, 9, 0, 1, 3), (0, 4, 9, 2, 0), (0, 4, 8, 5, 1, 4), (0, 4, 8, 5, 1, 1, 6), (0, 2, 3, 0), (0, 4, 5, 4, 1, 4, 2), (0, 4, 9, 5, 0, 0, 3), (0, 4, 8, 4, 4, 1), (0, 4, 8, 5, 8, 0), (0, 4, 7, 5), (0, 2, 3, 0), (0, 2, 3, 2, 1), (0, 1, 6, 7, 5, 2), (0, 4, 8, 0, 8, 4), (0, 6, 5, 1), (0, 9, 2, 2), (0, 4, 2, 2, 8), (0, 1, 6, 7, 5, 2), (0, 0, 0, 1), (0, 9, 6, 8), (0, 9, 0, 4), (0, 0, 4, 3), (0, 1, 4, 6, 4), (0, 4, 8, 5, 7, 6, 1), (0, 4, 8, 5, 7, 0, 2, 0), (0, 6, 9, 8, 1), (0, 4, 8, 5, 2, 6), (0, 4, 5, 5), (0, 4, 8, 3, 2), (0, 4, 8, 5, 1, 6), (0, 2, 8, 9), (0, 6, 0, 3, 5), (0, 4, 8, 5, 7, 2, 1), (0, 4, 2, 4, 0), (0, 1, 3, 6), (0, 4, 8, 8, 7), (0, 4, 8, 0, 9, 5), (0, 0, 4, 4), (0, 4, 0, 1, 8), (0, 2, 0, 8), (0, 6, 5, 6), (0, 4, 0, 7, 5), (0, 4, 0, 7, 5), (0, 4, 8, 0, 1), (0, 0, 5), (0, 4, 5, 1), (0, 1, 4, 1), (0, 4, 8, 4, 4, 7), (0, 4, 8, 5, 1, 1, 6), (0, 7, 1, 5, 0), (0, 4, 8, 5, 7, 2, 1), (0, 6, 2, 9), (0, 6, 5, 4), (0, 4, 6, 1, 1, 1), (0, 4, 8, 9, 7), (0, 4, 8, 5, 2, 0), (0, 4, 8, 5, 3, 2), (0, 0, 0, 0), (0, 2, 1, 5), (0, 1, 0, 8), (0, 2, 3, 2, 1), (0, 1, 0, 2, 4), (0, 4, 5, 6, 1), (0, 1, 6, 9, 2), (0, 4, 0, 0, 6), (0, 4, 8, 5, 1, 2), (0, 9, 2, 8), (0, 0, 3, 0), (0, 6, 7, 8), (0, 6, 7, 0, 2), (0, 1, 6, 6, 2), (0, 7, 4, 3), (0, 4, 2, 0), (0, 6, 7, 7), (0, 2, 5, 1), (0, 4, 8, 5, 7, 8, 3), (0, 1, 6, 7, 2), (0, 4, 7, 8, 1), (0, 4, 8, 0, 8, 0), (0, 0, 6, 1), (0, 9, 4, 6), (0, 4, 8, 5, 8, 6, 4), (0, 4, 8, 5, 7, 0, 8), (0, 4, 8, 5, 7, 0, 1), (0, 4, 8, 5, 7, 7, 8), (0, 4, 8, 3, 4, 0), (0, 0, 1, 9, 1), (0, 3, 5), (0, 1, 0, 6), (0, 3, 5), (0, 4, 9, 7), (0, 4, 0, 1, 1), (0, 9, 2, 4), (0, 6, 5, 1), (0, 7, 1, 1), (0, 1, 0, 6), (0, 1, 5, 3), (0, 3, 4), (0, 7, 1, 1), (0, 7, 1, 1), (0, 6, 1, 0), (0, 7, 1, 1), (0, 7, 1, 1), (0, 7, 1, 1), (0, 7, 1, 1), (0, 8, 3, 3), (0, 1, 5, 8, 1), (0, 4, 5, 4, 1, 7), (0, 6, 0, 5, 8), (0, 4, 8, 7, 7), (0, 2, 3, 8, 8), (0, 5, 5), (0, 0, 0, 4, 6), (0, 4, 8, 8, 5), (0, 0, 0, 0), (0, 1, 6, 7, 1), (0, 3, 5), (0, 4, 4, 4, 7), (0, 4, 8, 4, 1), (0, 3, 4), (0, 5, 1, 0), (0, 0, 6, 1), (0, 4, 2, 9), (0, 4, 7, 8, 3), (0, 2, 0, 7), (0, 4, 2, 2, 4), (0, 4, 8, 0, 0, 4), (0, 2, 2, 9), (0, 8, 2, 7), (0, 4, 5, 7, 5), (0, 2, 3, 6, 8), (0, 4, 0, 1, 4), (0, 0, 0, 6), (0, 2, 8, 3), (0, 1, 5, 2, 1), (0, 8, 6, 7), (0, 4, 8, 5, 7, 0, 3, 9), (0, 4, 8, 5, 5, 1), (0, 0, 8, 6), (0, 4, 7, 4), (0, 8, 5, 3), (0, 8, 1, 8), (0, 4, 8, 5, 0, 4), (0, 4, 7, 8, 5), (0, 5, 0, 2, 1, 1), (0, 6, 0, 3, 7), (0, 8, 5, 4), (0, 4, 8, 5, 8, 4, 4), (0, 0, 7, 2), (0, 8, 2, 3), (0, 4, 8, 5, 5, 4), (0, 1, 5, 5, 2), (0, 1, 2, 4, 8), (0, 1, 2, 9), (0, 4, 5, 4, 1, 4, 2), (0, 4, 8, 8, 0, 6), (0, 0, 3, 1), (0, 4, 8, 5, 1, 4), (0, 4, 8, 0, 9, 1, 1), (0, 4, 6, 0, 3), (0, 4, 0, 5, 8), (0, 4, 7, 4), (0, 6, 1, 3), (0, 6, 0, 1, 1), (0, 6, 9, 6), (0, 6, 6, 4, 9), (0, 0, 2, 1), (0, 1, 2, 5, 7, 3), (0, 4, 6, 3), (0, 8, 3, 3), (0, 6, 1, 7), (0, 1, 0, 2, 5), (0, 4, 6, 3), (0, 0, 9, 2, 1), (0, 4, 8, 0, 9, 1, 1), (0, 4, 3, 8), (0, 4, 5, 4, 1, 4, 9), (0, 4, 9, 5, 0, 0, 0), (0, 8, 3, 8), (0, 4, 9, 5, 0, 2), (0, 5, 0, 2, 1, 3), (0, 8, 1, 2), (0, 2, 8, 1), (0, 4, 8, 5, 3, 5), (0, 4, 8, 5, 1, 1, 5), (0, 4, 8, 5, 8, 6, 4), (0, 4, 8, 4, 1), (0, 4, 8, 5, 8, 7), (0, 4, 8, 4, 8), (0, 1, 6, 9, 7), (0, 2, 7, 1), (0, 6, 0, 3, 1), (0, 4, 7, 6, 3), (0, 6, 8, 4), (0, 4, 8, 5, 7, 9, 1), (0, 9, 3, 2, 0), (0, 0, 0, 5, 4), (0, 4, 2, 2, 4), (0, 4, 0, 5, 4), (0, 4, 8, 5, 7, 0, 5), (0, 4, 0, 1, 5, 1), (0, 0, 6, 4), (0, 4, 7, 2), (0, 6, 1, 6), (0, 4, 8, 5, 7, 8, 6), (0, 1, 5, 2, 5), (0, 2, 9, 3), (0, 4, 8, 5, 1, 1, 2), (0, 4, 8, 0, 6), (0, 0, 0, 4, 4), (0, 4, 8, 5, 8, 3, 9), (0, 0, 9, 3), (0, 6, 5, 1), (0, 7, 5, 4), (0, 4, 5, 4, 0), (0, 2, 1, 3), (0, 4, 8, 5, 7, 9, 3), (0, 1, 2, 8, 2), (0, 1, 4, 3, 4), (0, 8, 5, 3), (0, 4, 8, 8, 7), (0, 4, 8, 8, 4), (0, 2, 0, 2), (0, 4, 8, 5, 7, 3, 9), (0, 2, 1, 3), (0, 1, 2, 2, 7), (0, 1, 3, 9, 8), (0, 4, 0, 1, 5, 3), (0, 4, 8, 8, 8, 0), (0, 4, 0, 0, 1), (0, 4, 0, 6, 1), (0, 4, 0, 2, 5), (0, 1, 6, 3, 1, 2), (0, 2, 3, 1), (0, 4, 0, 6, 3, 1), (0, 6, 7, 9), (0, 1, 8, 1, 1), (0, 8, 5, 9), (0, 0, 0, 3), (0, 0, 2, 7), (0, 4, 8, 5, 1, 1, 1), (0, 5, 3, 5), (0, 4, 5, 4, 1, 4, 5), (0, 4, 8, 5, 1, 3), (0, 1, 5, 9), (0, 2, 0, 9), (0, 4, 8, 5, 3, 0), (0, 4, 8, 5, 7, 4, 3), (0, 8, 3, 3), (0, 3, 6, 3), (0, 3, 6, 1), (0, 3, 6, 6), (0, 3, 6, 1), (0, 3, 6, 8), (0, 3, 6, 1), (0, 6, 3, 9), (0, 4, 8, 5, 1, 0, 3), (0, 1, 4, 3, 8), (0, 4, 6, 0, 1), (0, 1, 4, 9), (0, 6, 9, 7, 8), (0, 1, 6, 2), (0, 1, 5, 0, 0), (0, 3, 6, 3), (0, 3, 6, 2), (0, 3, 6, 2), (0, 3, 6, 5), (0, 3, 6, 1), (0, 3, 6, 1), (0, 4, 8, 3, 4, 0), (0, 4, 0, 6, 4, 1), (0, 9, 1, 9), (0, 1, 6, 4, 7), (0, 4, 8, 0, 8, 3), (0, 6, 6, 5), (0, 1, 7, 7, 8), (0, 4, 0, 2, 4), (0, 4, 8, 5, 2, 4), (0, 2, 3, 6, 1), (0, 1, 6, 5, 2), (0, 2, 4, 5), (0, 8, 3, 0), (0, 4, 2, 4, 1), (0, 4, 8, 0, 3), (0, 4, 8, 0, 3), (0, 4, 0, 1, 5, 5), (0, 4, 8, 0, 4), (0, 6, 7, 5, 4), (0, 4, 8, 0, 0, 2), (0, 6, 8, 4), (0, 6, 0, 2), (0, 6, 9, 8, 2), (0, 6, 9, 1), (0, 4, 8, 5, 7, 9, 3), (0, 1, 6, 7, 1), (0, 3, 6, 1), (0, 3, 6, 1), (0, 3, 6, 1), (0, 3, 6, 1), (0, 4, 8, 5, 2, 6), (0, 3, 6, 1), (0, 3, 4), (0, 1, 6, 1, 6), (0, 3, 4), (0, 3, 4), (0, 5, 4, 1), (0, 5, 4, 1), (0, 5, 4, 1), (0, 3, 4), (0, 3, 4), (0, 1, 5, 2, 3), (0, 6, 9, 5), (0, 8, 6, 2), (0, 6, 9, 8, 3), (0, 4, 0, 1, 8), (0, 4, 8, 0, 4), (0, 4, 8, 5, 8, 5, 5), (0, 4, 8, 5, 8, 5, 8), (0, 4, 8, 8, 8, 6), (0, 4, 8, 5, 4, 8, 6), (0, 1, 2, 1), (0, 1, 4, 6, 3), (0, 4, 8, 5, 7, 7, 3), (0, 1, 6, 9, 2), (0, 1, 2, 2, 6), (0, 1, 2, 5, 7, 6), (0, 4, 8, 9, 4, 1), (0, 9, 0, 1, 4), (0, 3, 8), (0, 4, 8, 0, 8, 8), (0, 4, 8, 5, 7, 9, 2), (0, 1, 2, 5, 3), (0, 0, 8, 6), (0, 0, 1, 6), (0, 8, 0), (0, 9, 0, 1, 2), (0, 9, 3, 6), (0, 9, 3, 2, 3), (0, 9, 0, 4), (0, 0, 4, 9), (0, 6, 5, 1), (0, 4, 5, 5), (0, 4, 2, 4, 0), (0, 9, 0, 5), (0, 9, 3, 6), (0, 4, 8, 5, 4, 8, 1), (0, 6, 6, 5), (0, 4, 8, 5, 9, 9), (0, 4, 8, 5, 8, 4, 9), (0, 4, 8, 5, 7, 9, 2), (0, 4, 8, 0, 4), (0, 1, 9, 8), (0, 2, 8, 1), (0, 4, 8, 5, 0, 1), (0, 5, 0, 3), (0, 4, 5, 7, 6), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 1, 6), (0, 4, 8, 5, 1, 0, 0), (0, 4, 6, 5, 1), (0, 4, 8, 2, 3), (0, 4, 8, 2, 3), (0, 4, 8, 5, 2, 7), (0, 4, 5, 4, 1, 2), (0, 6, 7, 7), (0, 4, 8, 5, 7, 9, 9), (0, 4, 5, 4, 1, 2), (0, 4, 0, 6, 4, 1), (0, 1, 6, 7, 4), (0, 4, 8, 0, 7), (0, 4, 8, 9, 4, 1), (0, 9, 8, 7), (0, 4, 8, 5, 2, 3), (0, 1, 0, 1), (0, 6, 0, 0, 9), (0, 1, 8, 1, 4), (0, 4, 8, 0, 7), (0, 6, 5, 8, 7), (0, 6, 9, 0), (0, 7, 1, 5, 4), (0, 4, 8, 5, 3, 5), (0, 4, 5, 4, 1, 1), (0, 3, 6, 1), (0, 3, 6, 1), (0, 3, 6, 1), (0, 4, 8, 5, 7, 2, 0), (0, 4, 6, 9), (0, 4, 8, 1, 6), (0, 4, 8, 5, 7, 9, 3), (0, 1, 5, 3), (0, 1, 9, 6), (0, 4, 6, 0, 9), (0, 4, 2, 3), (0, 4, 8, 5, 4, 6), (0, 4, 6, 2, 2), (0, 4, 0, 1, 4), (0, 4, 8, 5, 7, 1, 6), (0, 4, 8, 5, 7, 5, 4), (0, 4, 8, 2, 5, 1), (0, 4, 8, 5, 8, 6, 4), (0, 5, 0, 2, 8), (0, 6, 7, 0, 8), (0, 1, 6, 4, 5), (0, 4, 6, 1, 1, 1), (0, 4, 5, 6, 8), (0, 4, 5, 6, 1), (0, 4, 5, 6, 1), (0, 4, 5, 6, 1), (0, 4, 5, 6, 1), (0, 4, 5, 6, 6), (0, 4, 0, 5, 5), (0, 1, 5, 8, 3), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 7, 4, 1), (0, 1, 6, 7, 0, 7), (0, 4, 0, 9), (0, 7, 5, 2), (0, 7, 1, 5, 2), (0, 4, 2, 4, 7), (0, 4, 8, 5, 1, 1, 6), (0, 6, 6, 8), (0, 0, 6, 2), (0, 5, 7, 0), (0, 7, 1, 9), (0, 7, 5, 2), (0, 7, 1, 9), (0, 7, 5, 2), (0, 2, 0, 6), (0, 4, 2, 5), (0, 4, 8, 3, 2), (0, 4, 0, 1, 3), (0, 4, 8, 5, 7, 3, 8), (0, 4, 8, 1, 0), (0, 2, 1, 2), (0, 4, 9, 6), (0, 4, 5, 4, 1, 8), (0, 9, 3, 1), (0, 4, 7, 6, 7), (0, 7, 1, 3), (0, 4, 9, 1), (0, 4, 8, 5, 7, 9, 9), (0, 0, 0, 1), (0, 1, 1, 2), (0, 4, 8, 0, 9, 1, 1), (0, 1, 2, 4, 5), (0, 4, 5, 4, 3), (0, 4, 8, 5, 8, 1), (0, 1, 7, 5), (0, 4, 8, 5, 8, 4, 3), (0, 4, 0, 1, 9), (0, 4, 8, 5, 5, 4), (0, 2, 6, 7), (0, 4, 8, 5, 7, 5, 1), (0, 4, 8, 5, 7, 5, 1), (0, 1, 6, 7, 1), (0, 1, 9, 1), (0, 4, 8, 0, 9, 2), (0, 0, 0, 3), (0, 0, 0, 1), (0, 4, 8, 5, 7, 5, 9), (0, 1, 6, 6, 8), (0, 0, 6, 3), (0, 0, 6, 3), (0, 2, 3, 8, 9), (0, 4, 8, 5, 7, 7, 1), (0, 1, 2, 5, 3), (0, 4, 0, 9), (0, 4, 8, 6, 4, 1), (0, 1, 2, 8, 2), (0, 2, 1, 1), (0, 4, 1, 0), (0, 4, 0, 0, 8), (0, 1, 2, 3), (0, 4, 0, 4, 6), (0, 1, 6, 3, 2), (0, 4, 6, 1, 2), (0, 1, 6, 4, 5), (0, 4, 8, 5, 7, 0, 2, 0), (0, 7, 5, 2), (0, 7, 5, 2), (0, 4, 0, 2, 6), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 3, 9, 0), (0, 4, 0, 4, 6), (0, 4, 8, 5, 5, 1), (0, 4, 0, 1, 5, 1), (0, 4, 8, 0, 9, 2), (0, 4, 0, 4, 1), (0, 4, 8, 5, 5, 4), (0, 4, 0, 4, 1), (0, 4, 0, 4, 6), (0, 1, 2, 8, 6), (0, 7, 7, 1), (0, 7, 1, 5, 8), (0, 4, 6, 0, 6), (0, 7, 5, 2), (0, 7, 5, 0), (0, 4, 8, 5, 8, 1), (0, 4, 8, 7, 0), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 3, 5), (0, 7, 5, 0), (0, 7, 1, 5, 7), (0, 7, 5, 0), (0, 7, 1, 2), (0, 7, 1, 4), (0, 4, 6, 1, 1, 1), (0, 4, 6, 8, 6), (0, 4, 6, 8, 6), (0, 4, 6, 8, 4), (0, 3, 6, 0), (0, 4, 5, 0), (0, 4, 6, 8, 2), (0, 3, 6, 4), (0, 3, 6, 0), (0, 4, 6, 1, 1, 1), (0, 3, 6, 9), (0, 4, 0, 6, 3, 7), (0, 4, 6, 8, 4), (0, 3, 8), (0, 3, 5), (0, 3, 7), (0, 3, 8), (0, 7, 5, 1, 2), (0, 3, 3, 6), (0, 7, 1, 0, 1), (0, 3, 8), (0, 3, 3, 6), (0, 3, 3, 6), (0, 3, 8), (0, 7, 1, 5, 2), (0, 3, 3, 6), (0, 7, 5, 1, 2), (0, 3, 6, 0), (0, 3, 6, 7), (0, 3, 6, 0), (0, 3, 6, 0), (0, 3, 6, 2), (0, 3, 6, 0), (0, 1, 6, 7, 0, 8), (0, 2, 9, 0), (0, 2, 9, 0), (0, 4, 8, 5, 1, 8), (0, 4, 4, 1, 1, 0), (0, 1, 3, 0), (0, 4, 8, 0, 7), (0, 1, 5, 3), (0, 3, 9), (0, 3, 9), (0, 3, 9), (0, 3, 7), (0, 3, 9), (0, 3, 9), (0, 4, 8, 5, 3, 2), (0, 1, 0, 5), (0, 4, 8, 8, 8, 1), (0, 6, 7, 3, 7), (0, 4, 6, 4), (0, 7, 5, 1, 6), (0, 7, 1, 5, 2), (0, 1, 6, 2), (0, 7, 1, 5, 2), (0, 5, 3, 5), (0, 7, 1, 5, 2), (0, 4, 6, 6, 7), (0, 7, 5, 1, 1), (0, 1, 5, 8, 3), (0, 4, 4, 0, 2), (0, 4, 8, 6, 5), (0, 6, 0, 0, 4), (0, 6, 2, 9), (0, 1, 6, 7, 2), (0, 4, 8, 0, 8, 2), (0, 3, 1, 1), (0, 3, 1, 1), (0, 3, 1, 4), (0, 3, 1, 7), (0, 3, 1, 0), (0, 3, 1, 9), (0, 3, 1, 0), (0, 3, 1, 6), (0, 3, 1, 0), (0, 3, 1, 0), (0, 3, 1, 0), (0, 3, 1, 0), (0, 4, 8, 5, 7, 9, 6), (0, 3, 1, 0), (0, 3, 1, 0), (0, 3, 1, 0), (0, 7, 5, 1, 1), (0, 3, 1, 6), (0, 3, 1, 3), (0, 3, 1, 0), (0, 3, 3, 7), (0, 3, 4), (0, 3, 1, 0), (0, 3, 1, 0), (0, 5, 0, 2, 1, 9), (0, 4, 8, 8, 8, 5), (0, 4, 8, 4, 1), (0, 3, 1, 5), (0, 3, 1, 2), (0, 3, 1, 8), (0, 3, 1, 2), (0, 3, 1, 0), (0, 7, 1, 5, 9), (0, 7, 5, 3), (0, 7, 5, 1, 6), (0, 7, 5, 1, 3), (0, 7, 1, 5, 2), (0, 7, 5, 0), (0, 7, 5, 1, 1), (0, 4, 6, 8, 4), (0, 4, 6, 0, 2), (0, 7, 1, 5, 2), (0, 7, 5, 3), (0, 7, 5, 2), (0, 7, 5, 1, 8), (0, 4, 6, 8, 4), (0, 7, 5, 3), (0, 4, 0, 6, 5), (0, 4, 6, 8, 4), (0, 6, 7, 0, 5), (0, 4, 0, 6, 4, 1), (0, 7, 5, 1, 1), (0, 4, 6, 1, 7), (0, 7, 5, 1, 1), (0, 4, 0, 6, 4, 1), (0, 7, 5, 6), (0, 7, 5, 1, 1), (0, 4, 6, 2, 3), (0, 4, 6, 8, 4), (0, 7, 5, 1, 1), (0, 4, 6, 8, 2), (0, 7, 5, 1, 1), (0, 4, 6, 8, 4), (0, 4, 6, 8, 1), (0, 4, 6, 8, 9), (0, 4, 6, 8, 4), (0, 4, 8, 6, 9), (0, 4, 8, 5, 4, 6), (0, 4, 8, 5, 2, 1), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 8), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 8, 6, 2), (0, 8, 5, 4), (0, 7, 1, 1), (0, 8, 2, 7), (0, 8, 2, 2), (0, 2, 4, 5), (0, 8, 4, 2), (0, 4, 6, 6, 7), (0, 4, 6, 8, 2), (0, 4, 6, 8, 4), (0, 7, 5, 1, 9), (0, 4, 0, 5, 0), (0, 8, 2, 5), (0, 8, 5, 1), (0, 3, 9), (0, 3, 9), (0, 3, 9), (0, 3, 9), (0, 3, 9), (0, 7, 7, 1), (0, 4, 8, 5, 7, 7, 1), (0, 8, 9, 5), (0, 8, 5, 1), (0, 8, 4, 0), (0, 4, 8, 5, 7, 0, 1), (0, 2, 4, 2), (0, 8, 8), (0, 4, 8, 5, 7, 5, 4), (0, 4, 8, 5, 7, 5, 4), (0, 4, 0, 4, 2), (0, 4, 0, 4, 6), (0, 4, 0, 4, 2), (0, 1, 5, 0, 1), (0, 4, 0, 1, 6), (0, 4, 0, 5, 5), (0, 1, 9, 0, 3), (0, 4, 0, 4, 6), (0, 4, 8, 6, 1), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 1), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 5), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 4), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 5), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 5), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 1), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 3, 3, 0, 2, 0, 0, 0), (0, 1, 6, 7, 9), (0, 4, 0, 4, 1), (0, 4, 8, 6, 8), (0, 4, 8, 5, 7, 5, 9), (0, 8, 1, 0), (0, 8, 6, 7), (0, 8, 3, 4), (0, 8, 6, 7), (0, 6, 9, 3), (0, 4, 2, 2, 3), (0, 4, 8, 8, 8, 1), (0, 4, 8, 5, 7, 5, 9), (0, 4, 8, 6, 8), (0, 1, 2, 8, 8), (0, 1, 5, 6, 2), (0, 8, 5, 1), (0, 4, 8, 6, 0), (0, 4, 8, 6, 0), (0, 4, 8, 6, 0), (0, 1, 2, 3), (0, 4, 0, 4, 9), (0, 4, 0, 3, 9), (0, 1, 5, 2, 0), (0, 8, 4, 4), (0, 1, 5, 7), (0, 4, 8, 5, 7, 7, 0), (0, 4, 2, 2, 3), (0, 4, 6, 5, 1), (0, 4, 5, 0), (0, 4, 8, 8, 7), (0, 4, 8, 9, 8), (0, 8, 9, 5), (0, 8, 5, 1), (0, 1, 8, 8), (0, 1, 4, 6, 5), (0, 4, 2, 2, 0), (0, 5, 9), (0, 4, 4, 3), (0, 1, 2, 8, 2), (0, 8, 8), (0, 8, 9, 5), (0, 1, 6, 3, 4), (0, 6, 1, 9), (0, 8, 2, 6), (0, 0, 9, 2, 1), (0, 9, 0, 5), (0, 8, 4, 3), (0, 8, 5, 3), (0, 4, 8, 5, 7, 0, 9, 5), (0, 4, 8, 5, 1, 0, 0), (0, 8, 3, 3), (0, 8, 2, 7), (0, 8, 3, 2), (0, 4, 6, 4), (0, 1, 5, 3), (0, 1, 6, 5, 3), (0, 8, 3, 3), (0, 8, 7, 2), (0, 4, 0, 0, 0), (0, 8, 9, 8), (0, 8, 3, 0), (0, 8, 3, 0), (0, 4, 8, 5, 7, 3, 4), (0, 4, 0, 1, 4), (0, 6, 0, 9, 4), (0, 4, 4, 1, 1, 0), (0, 3, 3, 2), (0, 3, 3, 2), (0, 2, 1, 6), (0, 2, 3, 6, 0), (0, 6, 0, 6), (0, 8, 2, 5), (0, 5, 0, 4), (0, 1, 2, 8, 2), (0, 2, 7, 2), (0, 8, 9, 0), (0, 7, 9, 1), (0, 4, 7, 3), (0, 4, 7, 2), (0, 5, 0, 2, 1, 4), (0, 8, 8), (0, 5, 9), (0, 8, 3, 8), (0, 8, 2, 5), (0, 8, 3, 1), (0, 6, 1, 9), (0, 4, 8, 8, 1), (0, 1, 6, 7, 5, 2), (0, 8, 9, 5), (0, 8, 5, 4), (0, 0, 0, 0), (0, 2, 4, 3), (0, 6, 3, 2), (0, 7, 5, 6), (0, 4, 8, 5, 5, 5), (0, 4, 4, 3), (0, 4, 8, 5, 4, 8, 9), (0, 1, 6, 5, 5), (0, 4, 8, 8, 7), (0, 4, 9, 5, 0, 9), (0, 4, 8, 8, 8, 1), (0, 4, 8, 8, 4), (0, 4, 0, 9), (0, 4, 0, 9), (0, 4, 8, 5, 7, 9, 0), (0, 4, 0, 4, 0), (0, 4, 8, 5, 2, 2), (0, 5, 3, 1), (0, 6, 3, 3, 1), (0, 6, 3, 7), (0, 6, 0, 2), (0, 4, 0, 6, 2), (0, 4, 0, 1, 5, 1), (0, 2, 9, 3), (0, 4, 6, 7), (0, 4, 8, 3, 4, 3), (0, 4, 0, 3, 2), (0, 6, 7, 3, 6), (0, 1, 2, 7, 0), (0, 1, 4, 0), (0, 1, 7, 3), (0, 1, 6, 7, 8), (0, 1, 2, 5, 7, 3), (0, 6, 5, 2), (0, 1, 7, 7, 1), (0, 6, 8, 1), (0, 4, 4, 6), (0, 3, 8), (0, 1, 6, 8), (0, 1, 2, 8, 2), (0, 1, 3, 6), (0, 4, 0, 2, 4), (0, 1, 5, 9), (0, 4, 8, 5, 8, 0), (0, 4, 8, 5, 7, 7, 8), (0, 1, 2, 7, 5), (0, 1, 2, 5, 5), (0, 4, 4, 1, 1, 6), (0, 1, 5, 0, 9), (0, 4, 8, 1, 4), (0, 4, 8, 5, 1, 1, 7), (0, 6, 0, 2), (0, 2, 8, 3), (0, 1, 6, 6, 8), (0, 1, 6, 7, 3), (0, 6, 1, 4), (0, 0, 8, 0), (0, 4, 3, 3), (0, 1, 0, 2, 0), (0, 2, 5, 7), (0, 6, 9, 4), (0, 1, 6, 3, 1, 0), (0, 4, 1, 5), (0, 4, 8, 5, 7, 2, 3), (0, 4, 8, 1, 4), (0, 4, 8, 5, 3, 8), (0, 4, 8, 5, 7, 7, 2), (0, 4, 5, 8, 1), (0, 1, 9, 9), (0, 5, 0, 2, 3), (0, 1, 6, 4, 1), (0, 6, 0, 7, 9), (0, 2, 7, 5), (0, 2, 7, 0, 0), (0, 2, 7, 0, 0), (0, 2, 7, 0, 0), (0, 2, 7, 0, 0), (0, 2, 7, 0, 0), (0, 9, 0, 9), (0, 4, 8, 6, 7), (0, 4, 8, 6, 4, 3), (0, 4, 8, 2, 1, 2), (0, 4, 8, 2, 4, 2), (0, 4, 8, 5, 8, 6, 6), (0, 4, 8, 5, 7, 8, 1), (0, 1, 6, 7, 5, 7), (0, 4, 8, 5, 7, 6, 1), (0, 1, 5, 2, 1), (0, 1, 5, 2, 8), (0, 1, 0, 5), (0, 4, 8, 5, 4, 8, 1), (0, 4, 8, 5, 8, 5, 0), (0, 4, 8, 5, 8, 1), (0, 6, 7, 2), (0, 9, 0, 9), (0, 5, 0, 8), (0, 4, 5, 3), (0, 4, 6, 1, 1, 3), (0, 4, 5, 4, 1, 4, 1), (0, 1, 9, 5), (0, 5, 5), (0, 5, 5), (0, 2, 7, 0, 0), (0, 5, 5), (0, 5, 8, 1), (0, 5, 5), (0, 2, 7, 0, 0), (0, 2, 7, 0, 8), (0, 2, 7, 0, 8), (0, 1, 6, 7, 6), (0, 4, 5, 3), (0, 4, 5, 1), (0, 4, 8, 3, 8), (0, 4, 8, 2, 5, 1), (0, 4, 5, 3), (0, 4, 8, 2, 5, 1), (0, 9, 3, 6), (0, 0, 5), (0, 4, 2, 9), (0, 1, 4, 3, 9), (0, 1, 2, 5, 7, 3), (0, 1, 2, 4, 8), (0, 4, 8, 5, 2, 4), (0, 1, 8, 9), (0, 4, 8, 0, 2), (0, 1, 6, 8), (0, 1, 3, 8), (0, 1, 6, 3, 4), (0, 4, 8, 5, 7, 2, 5), (0, 1, 5, 0, 2), (0, 4, 5, 8, 1), (0, 4, 8, 5, 7, 0, 2, 0), (0, 7, 9, 0), (0, 7, 3, 0), (0, 4, 0, 9), (0, 6, 6, 4, 0), (0, 4, 5, 4, 1, 2), (0, 3, 3, 7), (0, 6, 9, 5), (0, 4, 8, 5, 3, 4, 1), (0, 4, 9, 2, 4), (0, 8, 6, 4), (0, 4, 8, 2, 4, 0), (0, 4, 2, 4, 3, 0), (0, 8, 1, 5), (0, 2, 4, 5), (0, 4, 8, 3, 4, 9), (0, 4, 8, 5, 7, 5, 3), (0, 1, 6, 9, 7), (0, 7, 1, 6, 2), (0, 1, 0, 2, 4), (0, 8, 2, 2), (0, 8, 6, 2), (0, 7, 9, 5), (0, 4, 8, 5, 7, 7, 1), (0, 6, 0, 3, 2), (0, 8, 6, 0), (0, 8, 6, 2), (0, 2, 2, 1), (0, 6, 0, 1, 9), (0, 4, 2, 4, 3, 0), (0, 8, 6, 2), (0, 8, 6, 2), (0, 8, 2, 2), (0, 8, 2, 2), (0, 4, 8, 5, 7, 4, 6), (0, 4, 8, 5, 7, 9, 7), (0, 8, 6, 2), (0, 4, 9, 4), (0, 4, 9, 4), (0, 5, 0, 2, 3), (0, 1, 6, 7, 3), (0, 4, 0, 1, 2), (0, 4, 0, 1, 8), (0, 6, 8, 2), (0, 4, 8, 5, 7, 0, 0, 3), (0, 6, 5, 3), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 5, 7, 0, 3, 6), (0, 4, 8, 6, 4, 1), (0, 4, 8, 5, 3, 4, 1), (0, 2, 8, 0), (0, 4, 6, 2, 8), (0, 4, 8, 5, 3, 4, 1), (0, 4, 8, 8, 0, 1), (0, 4, 0, 8, 0), (0, 4, 8, 2, 8), (0, 4, 8, 5, 5, 8), (0, 4, 8, 0, 0, 6), (0, 4, 5, 8, 4), (0, 4, 9, 5, 3), (0, 9, 0, 1, 8), (0, 8, 7, 4), (0, 8, 2, 0, 2), (0, 8, 6, 0), (0, 0, 8, 3), (0, 0, 0, 5, 5), (0, 8, 2, 3), (0, 8, 6, 6), (0, 8, 6, 0), (0, 8, 6, 0), (0, 8, 6, 0), (0, 8, 2, 3), (0, 4, 5, 2, 4), (0, 1, 4, 2), (0, 4, 8, 8, 6), (0, 7, 9, 2), (0, 8, 6, 2), (0, 8, 6, 2), (0, 8, 6, 2), (0, 8, 2, 3), (0, 4, 8, 5, 0, 9), (0, 4, 8, 5, 9, 8), (0, 1, 6, 9, 0), (0, 6, 9, 8, 4), (0, 7, 9, 2), (0, 2, 7, 3), (0, 2, 7, 3), (0, 2, 7, 3), (0, 1, 5, 2, 0), (0, 2, 7, 3), (0, 2, 7, 3), (0, 2, 7, 3), (0, 2, 7, 3), (0, 1, 6, 3, 0), (0, 1, 5, 6, 7), (0, 1, 8, 2), (0, 8, 2, 0, 1), (0, 8, 2, 0, 1), (0, 4, 0, 3, 0), (0, 1, 6, 7, 6), (0, 4, 8, 4, 7), (0, 8, 2, 1), (0, 8, 2, 7), (0, 8, 6, 1), (0, 8, 6, 0), (0, 4, 2, 4, 3, 8), (0, 4, 0, 1, 1), (0, 4, 0, 8, 7), (0, 6, 7, 2), (0, 6, 0, 3, 0), (0, 6, 0, 7, 7), (0, 6, 0, 8, 1), (0, 4, 8, 5, 9, 8), (0, 8, 6, 1), (0, 8, 6, 1), (0, 1, 2, 8, 2), (0, 4, 0, 5, 4), (0, 8, 2, 0, 1), (0, 5, 3, 5), (0, 1, 6, 5, 3), (0, 6, 1, 1, 9), (0, 8, 6, 1), (0, 8, 6, 0), (0, 4, 8, 0, 5, 0), (0, 8, 6, 0), (0, 8, 6, 1), (0, 8, 6, 1), (0, 8, 6, 1), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 7, 0, 9, 0), (0, 4, 8, 6, 6), (0, 8, 7, 7), (0, 8, 7, 7), (0, 8, 7, 7), (0, 8, 6, 5), (0, 8, 7, 1), (0, 8, 7, 7), (0, 8, 7, 1), (0, 8, 7, 6), (0, 7, 1, 6, 0), (0, 8, 7, 3), (0, 8, 7, 1), (0, 7, 1, 1), (0, 8, 7, 6), (0, 7, 1, 0, 7), (0, 8, 7, 1), (0, 8, 6, 6), (0, 8, 7, 1), (0, 8, 7, 6), (0, 8, 7, 8), (0, 0, 0, 4, 1), (0, 1, 8, 1, 1), (0, 2, 0, 0), (0, 8, 7, 6), (0, 8, 6, 6), (0, 8, 7, 6), (0, 8, 7, 7), (0, 9, 8, 8), (0, 4, 0, 5, 6), (0, 4, 8, 5, 7, 0, 0, 0), (0, 8, 6, 0), (0, 4, 8, 0, 8, 9), (0, 4, 8, 5, 7, 1, 1), (0, 1, 8, 3), (0, 4, 8, 8, 0, 3), (0, 4, 8, 2, 5, 5), (0, 8, 2, 2), (0, 4, 8, 5, 1, 0, 3), (0, 4, 8, 5, 7, 0, 9, 6), (0, 4, 7, 4), (0, 4, 8, 5, 7, 0, 6), (0, 1, 6, 0), (0, 1, 6, 4, 2), (0, 5, 8, 1), (0, 7, 1, 9), (0, 4, 6, 2, 7), (0, 4, 8, 0, 0, 3), (0, 1, 5, 0, 0), (0, 4, 4, 4, 6), (0, 1, 3, 2), (0, 3, 7), (0, 4, 8, 0, 0, 7), (0, 4, 8, 5, 7, 0, 1), (0, 1, 6, 0), (0, 2, 2, 6), (0, 9, 0, 1, 7), (0, 4, 8, 5, 7, 9, 0), (0, 4, 8, 5, 8, 4, 5), (0, 4, 8, 1, 6), (0, 4, 8, 5, 5, 1), (0, 4, 8, 0, 0, 2), (0, 2, 7, 1), (0, 5, 8, 7), (0, 4, 8, 6, 1), (0, 4, 8, 5, 8, 4, 0), (0, 7, 1, 4), (0, 4, 4, 0, 1), (0, 6, 0, 9, 1), (0, 4, 8, 6, 1), (0, 4, 8, 5, 5, 1), (0, 4, 8, 5, 9, 8), (0, 7, 1, 8), (0, 4, 8, 3, 4, 9), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 3, 9, 4), (0, 1, 6, 9, 3), (0, 9, 7, 0), (0, 4, 8, 5, 9, 3), (0, 4, 7, 5), (0, 4, 8, 5, 0, 4), (0, 9, 3, 2, 1), (0, 4, 8, 5, 8, 9), (0, 4, 5, 5), (0, 4, 8, 8, 9), (0, 4, 8, 2, 0), (0, 4, 8, 5, 7, 6, 4), (0, 9, 7, 9), (0, 1, 6, 5, 2), (0, 4, 8, 8, 8, 0), (0, 4, 7, 3), (0, 2, 3, 7, 6), (0, 2, 3, 8, 0), (0, 4, 8, 6, 6), (0, 0, 0, 1), (0, 4, 8, 6, 6), (0, 1, 2, 1), (0, 4, 8, 8, 8, 9), (0, 6, 8, 8), (0, 3, 3, 2), (0, 4, 8, 5, 0, 0), (0, 4, 8, 5, 7, 1, 1), (0, 4, 8, 0, 8, 4), (0, 5, 9), (0, 6, 0, 4), (0, 4, 8, 0, 0, 8), (0, 6, 0, 8, 2), (0, 6, 8, 3), (0, 4, 8, 5, 2, 8), (0, 1, 4, 8), (0, 0, 0, 0), (0, 6, 0, 0, 1), (0, 1, 8, 6), (0, 4, 6, 5, 1), (0, 5, 9), (0, 0, 0, 4, 8), (0, 4, 3, 1, 5), (0, 7, 1, 4), (0, 1, 2, 5, 6), (0, 0, 3, 6), (0, 8, 8), (0, 4, 8, 2, 1, 2), (0, 4, 4, 1, 1, 1), (0, 2, 4, 2), (0, 8, 2, 0, 5), (0, 4, 8, 6, 6), (0, 4, 5, 1), (0, 4, 7, 4), (0, 8, 3, 1), (0, 8, 1, 2), (0, 1, 6, 5, 3), (0, 1, 3, 9, 0), (0, 1, 5, 8, 2), (0, 1, 9, 0, 2), (0, 4, 8, 6, 4, 9), (0, 4, 7, 8, 5), (0, 4, 7, 8, 5), (0, 4, 8, 5, 6, 7), (0, 4, 8, 6, 6), (0, 1, 6, 3, 6), (0, 8, 9, 0), (0, 1, 9, 9), (0, 4, 5, 7, 6), (0, 4, 8, 4, 4, 4), (0, 1, 6, 3, 7), (0, 1, 2, 6), (0, 8, 3, 0), (0, 8, 5, 6), (0, 8, 0), (0, 4, 8, 5, 8, 0), (0, 9, 3, 3), (0, 2, 0, 2), (0, 8, 5, 6), (0, 8, 7, 6), (0, 1, 6, 3, 1, 9), (0, 8, 1, 3), (0, 4, 9, 5, 4), (0, 5, 9), (0, 5, 9), (0, 5, 9), (0, 5, 9), (0, 5, 1, 5), (0, 4, 8, 5, 9, 3), (0, 4, 8, 5, 0, 6), (0, 8, 5, 3), (0, 4, 8, 5, 8, 3, 5), (0, 8, 8), (0, 8, 4, 0), (0, 5, 9), (0, 1, 4, 7), (0, 1, 5, 9), (0, 2, 2, 7), (0, 2, 2, 2), (0, 2, 2, 7), (0, 2, 2, 2), (0, 2, 2, 2), (0, 2, 2, 7), (0, 1, 6, 7, 2), (0, 4, 8, 5, 1, 7), (0, 4, 8, 5, 3, 6), (0, 4, 8, 5, 1, 1, 0), (0, 6, 7, 7), (0, 6, 3, 1), (0, 8, 9, 8), (0, 8, 3, 3), (0, 8, 2, 0, 2), (0, 8, 3, 0), (0, 8, 2, 0, 0), (0, 4, 2, 9), (0, 2, 2, 7), (0, 2, 2, 7), (0, 2, 2, 7), (0, 6, 3, 9), (0, 4, 2, 7, 5), (0, 4, 8, 8, 0, 4), (0, 4, 0, 3, 4), (0, 1, 5, 9), (0, 4, 8, 5, 7, 0, 0, 5), (0, 2, 3, 7, 0), (0, 6, 7, 9), (0, 8, 4, 2), (0, 2, 2, 6), (0, 4, 5, 7, 4), (0, 7, 3, 7), (0, 4, 8, 5, 1, 0, 0), (0, 8, 4, 1), (0, 8, 4, 4), (0, 2, 2, 6), (0, 2, 4, 2), (0, 8, 4, 2), (0, 8, 4, 3), (0, 4, 6, 7), (0, 4, 8, 8, 8, 4), (0, 2, 3, 8, 1), (0, 2, 9, 1), (0, 4, 8, 5, 3, 4, 1), (0, 2, 2, 6), (0, 2, 2, 6), (0, 4, 8, 0, 1), (0, 4, 4, 3), (0, 1, 0, 4), (0, 4, 8, 5, 9, 4), (0, 4, 9, 5, 8), (0, 4, 0, 6, 5), (0, 8, 5, 7), (0, 8, 9, 5), (0, 2, 9, 1), (0, 8, 4, 4), (0, 1, 4, 9), (0, 1, 8, 1, 7), (0, 8, 9, 5), (0, 4, 8, 5, 7, 9, 8), (0, 2, 2, 2), (0, 2, 2, 2), (0, 2, 2, 2), (0, 8, 6, 9), (0, 8, 2, 6), (0, 8, 9, 5), (0, 8, 3, 0), (0, 4, 0, 4, 2), (0, 4, 8, 5, 2, 6), (0, 4, 8, 8, 4), (0, 2, 8, 5), (0, 4, 9, 5, 0, 0, 7), (0, 1, 3, 9, 0), (0, 1, 4, 9), (0, 4, 2, 7, 5), (0, 4, 8, 5, 7, 9, 1), (0, 2, 0, 6), (0, 8, 3, 0), (0, 4, 2, 7, 1, 1), (0, 0, 8, 1), (0, 0, 8, 1), (0, 4, 8, 5, 3, 6), (0, 8, 2, 8), (0, 8, 2, 0, 8), (0, 6, 7, 3, 2), (0, 1, 2, 4, 2), (0, 6, 0, 7, 2), (0, 6, 9, 6), (0, 1, 0, 2, 4), (0, 8, 6, 3), (0, 4, 8, 8, 7), (0, 1, 2, 2, 4), (0, 8, 4, 1), (0, 4, 8, 5, 3, 4, 4), (0, 8, 6, 3), (0, 2, 2, 6), (0, 2, 2, 6), (0, 2, 2, 6), (0, 6, 7, 3, 4), (0, 8, 3, 0), (0, 4, 8, 5, 8, 2), (0, 4, 4, 9), (0, 6, 0, 9, 1), (0, 4, 8, 5, 5, 1), (0, 4, 8, 5, 5, 1), (0, 2, 2, 2), (0, 4, 8, 0, 8, 0), (0, 2, 2, 6), (0, 8, 9, 7), (0, 8, 1, 3), (0, 8, 9, 8), (0, 4, 8, 5, 2, 8), (0, 4, 8, 9, 4, 6), (0, 2, 2, 6), (0, 2, 2, 6), (0, 8, 1, 7), (0, 4, 8, 1, 4), (0, 2, 4, 2), (0, 8, 9, 8), (0, 4, 6, 1, 1, 8), (0, 4, 4, 1, 4), (0, 4, 8, 8, 0, 3), (0, 8, 2, 5), (0, 8, 1, 3), (0, 8, 1, 0), (0, 1, 5, 6, 6), (0, 0, 6, 1), (0, 1, 2, 5, 1), (0, 4, 8, 1, 4), (0, 8, 2, 0, 2), (0, 0, 3, 4), (0, 4, 8, 9, 8), (0, 9, 0, 8), (0, 1, 1, 1), (0, 0, 9, 2, 4), (0, 4, 0, 1, 5, 0), (0, 1, 9, 0, 1), (0, 1, 8, 1, 9), (0, 8, 5, 7), (0, 4, 8, 5, 1, 0, 3), (0, 4, 8, 5, 8, 3, 0), (0, 4, 8, 5, 8, 5, 1), (0, 6, 6, 3), (0, 4, 8, 5, 1, 3), (0, 7, 9, 0), (0, 4, 5, 1), (0, 4, 8, 5, 2, 0), (0, 4, 8, 4, 9), (0, 8, 9, 5), (0, 4, 8, 5, 3, 5), (0, 2, 4, 2), (0, 2, 2, 6), (0, 8, 9, 9), (0, 2, 2, 6), (0, 8, 4, 1), (0, 8, 2, 3), (0, 2, 3, 2, 1), (0, 4, 8, 5, 1, 1, 6), (0, 0, 2, 0), (0, 8, 2, 0, 5), (0, 2, 4, 4), (0, 8, 6, 7), (0, 8, 9, 8), (0, 4, 8, 5, 4, 2), (0, 1, 2, 5, 6), (0, 4, 3, 1, 0), (0, 4, 8, 5, 2, 4), (0, 4, 0, 1, 4), (0, 4, 8, 5, 7, 0, 6), (0, 4, 8, 5, 1, 6), (0, 2, 2, 3), (0, 2, 2, 5), (0, 8, 4, 3), (0, 4, 6, 8, 5), (0, 8, 3, 1), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 5, 7, 7, 9), (0, 2, 7, 0, 0), (0, 2, 2, 6), (0, 2, 2, 6), (0, 8, 3, 1), (0, 8, 4, 1), (0, 1, 6, 1, 2), (0, 4, 5, 5), (0, 4, 1, 4), (0, 6, 7, 5, 7), (0, 8, 5, 1), (0, 8, 5, 1), (0, 8, 2, 7), (0, 8, 9, 5), (0, 8, 4, 2), (0, 4, 5, 4, 5), (0, 4, 7, 9), (0, 4, 7, 7), (0, 7, 9, 8), (0, 4, 7, 8, 5), (0, 4, 2, 7, 1, 4), (0, 4, 8, 5, 1, 2), (0, 6, 1, 4), (0, 1, 6, 7, 8), (0, 1, 5, 1), (0, 4, 7, 8, 4), (0, 4, 0, 1, 3), (0, 8, 9, 5), (0, 0, 3, 8), (0, 8, 9, 8), (0, 2, 2, 8), (0, 2, 2, 8), (0, 8, 2, 5), (0, 4, 8, 5, 2, 1), (0, 4, 8, 5, 7, 4, 1), (0, 4, 5, 8, 3), (0, 4, 8, 5, 7, 4, 4), (0, 4, 8, 5, 8, 4, 7), (0, 4, 8, 5, 1, 0, 6), (0, 2, 4, 3), (0, 8, 3, 4), (0, 2, 2, 6), (0, 8, 9, 5), (0, 4, 0, 1, 5, 1), (0, 4, 8, 2, 1, 7), (0, 1, 2, 4, 1), (0, 8, 4, 3), (0, 9, 4, 4), (0, 0, 4, 3), (0, 5, 4, 3), (0, 6, 7, 4, 7), (0, 8, 4, 0), (0, 8, 3, 0), (0, 8, 5, 3), (0, 6, 7, 4, 1), (0, 8, 1, 8), (0, 6, 0, 7, 0), (0, 6, 3, 0), (0, 4, 8, 0, 1), (0, 2, 0, 8), (0, 5, 5), (0, 8, 2, 0, 9), (0, 2, 4, 5), (0, 8, 3, 6), (0, 8, 7, 9), (0, 2, 8, 5), (0, 7, 1, 6, 0), (0, 0, 4, 1), (0, 2, 2, 3), (0, 8, 5, 4), (0, 2, 9, 8), (0, 2, 2, 3), (0, 4, 0, 1, 8), (0, 8, 2, 0, 6), (0, 7, 7, 7), (0, 1, 6, 9, 2), (0, 4, 8, 5, 3, 2), (0, 8, 2, 5), (0, 6, 0, 0, 1), (0, 4, 8, 5, 3, 2), (0, 4, 8, 5, 3, 1), (0, 2, 9, 3), (0, 4, 8, 5, 6, 8), (0, 4, 8, 5, 3, 7), (0, 7, 5, 1, 1), (0, 1, 3, 9, 0), (0, 8, 2, 3), (0, 9, 4, 8), (0, 9, 0, 0), (0, 2, 0, 0), (0, 4, 2, 2, 7), (0, 8, 4, 7), (0, 4, 8, 8, 5), (0, 9, 2, 2), (0, 8, 3, 0), (0, 2, 8, 2), (0, 1, 4, 3, 7), (0, 1, 2, 7, 2), (0, 4, 0, 5, 7), (0, 7, 5, 3), (0, 2, 9, 8), (0, 2, 2, 0), (0, 6, 0, 4), (0, 4, 8, 5, 8, 6, 9), (0, 4, 8, 5, 4, 1), (0, 4, 4, 0, 2), (0, 2, 9, 0), (0, 0, 4, 0), (0, 4, 8, 5, 7, 4, 3), (0, 4, 8, 4, 1), (0, 8, 1, 6), (0, 4, 5, 4, 1, 4, 2), (0, 4, 2, 6), (0, 2, 4, 1), (0, 4, 0, 6, 3, 8), (0, 4, 0, 6, 3, 3), (0, 2, 2, 4), (0, 4, 0, 6, 1), (0, 4, 6, 5, 9), (0, 8, 6, 8), (0, 6, 7, 4, 1), (0, 8, 6, 0), (0, 4, 8, 5, 8, 8), (0, 8, 1, 1), (0, 4, 8, 9, 2), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 7, 9, 2), (0, 2, 4, 1), (0, 2, 2, 6), (0, 5, 1, 3), (0, 4, 8, 8, 8, 1), (0, 5, 4, 7), (0, 4, 8, 5, 8, 3, 5), (0, 4, 2, 7, 5), (0, 4, 6, 4), (0, 4, 8, 5, 5, 3), (0, 2, 9, 9), (0, 4, 4, 9), (0, 1, 4, 6, 0), (0, 4, 2, 4, 3, 0), (0, 4, 8, 2, 5, 8), (0, 1, 2, 7, 6), (0, 4, 8, 5, 3, 1), (0, 4, 8, 5, 7, 3, 0), (0, 4, 2, 4, 3, 9), (0, 8, 0), (0, 5, 0, 2, 1, 3), (0, 5, 7, 2), (0, 1, 5, 0, 4), (0, 0, 6, 1), (0, 2, 6, 1), (0, 4, 2, 4, 3, 1), (0, 4, 7, 8, 9), (0, 0, 5), (0, 5, 5), (0, 5, 8, 0), (0, 2, 7, 7), (0, 1, 4, 6, 3), (0, 4, 0, 6, 3, 5), (0, 9, 3, 1), (0, 6, 9, 3), (0, 6, 0, 8, 0), (0, 1, 5, 6, 4), (0, 0, 1, 5), (0, 4, 4, 3), (0, 1, 4, 1), (0, 4, 8, 3, 8), (0, 1, 6, 4, 5), (0, 4, 5, 7, 2), (0, 4, 8, 0, 7), (0, 4, 8, 3, 2), (0, 1, 5, 9), (0, 0, 0, 2), (0, 6, 8, 5), (0, 4, 8, 6, 2), (0, 4, 8, 5, 1, 0, 5), (0, 4, 8, 5, 7, 9, 1), (0, 4, 8, 5, 7, 7, 6), (0, 7, 7, 2), (0, 1, 4, 3, 1), (0, 4, 8, 5, 6, 9), (0, 4, 8, 1, 5), (0, 4, 5, 3), (0, 1, 3, 7), (0, 4, 8, 3, 2), (0, 4, 5, 4, 1, 3), (0, 4, 5, 4, 1, 3), (0, 4, 8, 5, 8, 1), (0, 2, 6, 4, 1), (0, 4, 8, 5, 2, 0), (0, 0, 2, 2), (0, 4, 8, 5, 7, 8, 2), (0, 4, 8, 3, 2), (0, 9, 0, 2), (0, 7, 1, 9), (0, 4, 8, 5, 7, 9, 1), (0, 6, 9, 4), (0, 4, 4, 1, 1, 8), (0, 8, 4, 6), (0, 1, 6, 1, 8), (0, 0, 4, 7), (0, 6, 0, 1, 9), (0, 6, 7, 6), (0, 1, 2, 2, 4), (0, 6, 6, 1), (0, 8, 9, 8), (0, 8, 9, 1), (0, 8, 9, 9), (0, 7, 9, 3), (0, 4, 8, 3, 2), (0, 8, 6, 3), (0, 4, 9, 3), (0, 4, 8, 0, 8, 0), (0, 1, 2, 5, 8), (0, 4, 8, 3, 2), (0, 8, 3, 4), (0, 8, 2, 0, 7), (0, 4, 8, 5, 8, 3, 5), (0, 4, 8, 6, 1), (0, 2, 1, 6), (0, 8, 6, 0), (0, 8, 3, 7), (0, 8, 2, 9), (0, 9, 3, 2, 0), (0, 4, 8, 5, 1, 7), (0, 8, 3, 0), (0, 1, 0, 3), (0, 4, 0, 6, 9), (0, 4, 6, 6, 9), (0, 4, 0, 6, 9), (0, 4, 6, 8, 4), (0, 4, 0, 1, 7), (0, 4, 0, 6, 6), (0, 6, 1, 1, 2), (0, 8, 2, 3), (0, 8, 3, 0), (0, 6, 4, 8), (0, 1, 3, 9, 0), (0, 4, 8, 5, 2, 2), (0, 4, 8, 0, 5, 7), (0, 1, 7, 1), (0, 6, 1, 6), (0, 8, 2, 2), (0, 8, 2, 0, 2), (0, 4, 2, 4, 2), (0, 4, 8, 0, 5, 8), (0, 6, 3, 1), (0, 4, 8, 5, 1, 0, 7), (0, 8, 2, 7), (0, 4, 5, 4, 3), (0, 6, 0, 1, 7), (0, 4, 6, 5, 7), (0, 8, 2, 3), (0, 8, 3, 3), (0, 4, 9, 1), (0, 1, 6, 1, 7), (0, 8, 4, 1), (0, 8, 0), (0, 4, 8, 5, 4, 8, 3), (0, 0, 0, 0), (0, 5, 3, 5), (0, 4, 8, 5, 7, 0, 3, 6), (0, 4, 8, 0, 7), (0, 4, 2, 0), (0, 4, 2, 0), (0, 4, 8, 3, 4, 9), (0, 4, 8, 5, 3, 1), (0, 6, 2, 3), (0, 4, 5, 7, 9), (0, 1, 5, 0, 1), (0, 4, 8, 2, 4, 5), (0, 1, 6, 9, 4), (0, 4, 8, 9, 2), (0, 4, 8, 5, 7, 6, 1), (0, 1, 6, 3, 2), (0, 1, 6, 4, 1), (0, 4, 8, 5, 3, 2), (0, 6, 0, 9, 4), (0, 4, 8, 2, 1, 4), (0, 4, 8, 8, 7), (0, 4, 5, 4, 9), (0, 1, 3, 7), (0, 4, 4, 1, 1, 0), (0, 4, 2, 4, 3, 0), (0, 4, 8, 5, 8, 6, 7), (0, 4, 4, 7), (0, 6, 4, 9), (0, 1, 5, 5, 3), (0, 4, 5, 4, 3), (0, 5, 0, 8), (0, 4, 8, 5, 1, 8), (0, 4, 8, 3, 8), (0, 1, 5, 6, 9), (0, 4, 2, 7, 5), (0, 1, 4, 5), (0, 4, 2, 5), (0, 4, 2, 7, 3), (0, 2, 2, 5), (0, 6, 7, 8), (0, 4, 2, 0), (0, 6, 0, 3, 1), (0, 6, 0, 2), (0, 4, 8, 5, 1, 0, 7), (0, 1, 1, 9), (0, 4, 8, 5, 4, 9), (0, 9, 2, 2), (0, 4, 8, 3, 2), (0, 4, 8, 1, 6), (0, 0, 8, 9), (0, 4, 8, 0, 9, 1, 6), (0, 4, 8, 5, 3, 8), (0, 2, 0, 8), (0, 0, 0, 1), (0, 4, 5, 4, 4), (0, 2, 3, 2, 1), (0, 0, 3, 5), (0, 0, 0, 1), (0, 1, 4, 4), (0, 4, 2, 2, 5), (0, 2, 9, 5), (0, 4, 6, 1, 2), (0, 1, 6, 4, 8), (0, 1, 1, 6), (0, 1, 7, 7, 1), (0, 4, 2, 2, 7), (0, 1, 3, 2), (0, 1, 0, 5), (0, 0, 5), (0, 2, 6, 4, 1), (0, 2, 7, 9), (0, 4, 8, 0, 1), (0, 0, 0, 1), (0, 0, 1, 9, 8), (0, 4, 4, 0, 1), (0, 4, 9, 8), (0, 4, 8, 5, 7, 8, 0), (0, 4, 2, 6), (0, 2, 3, 8, 3), (0, 1, 5, 3), (0, 4, 8, 5, 9, 4), (0, 1, 5, 6, 1), (0, 2, 3, 2, 9), (0, 1, 4, 3, 1), (0, 2, 3, 7, 4), (0, 4, 7, 2), (0, 4, 0, 6, 4, 6), (0, 4, 8, 5, 8, 7), (0, 4, 8, 5, 7, 8, 7), (0, 1, 9, 1), (0, 6, 6, 5), (0, 0, 6, 2), (0, 1, 5, 7), (0, 1, 5, 8, 6), (0, 4, 8, 5, 7, 7, 0), (0, 4, 4, 0, 0), (0, 4, 8, 5, 8, 4, 3), (0, 1, 0, 2, 2), (0, 4, 8, 5, 8, 1), (0, 4, 8, 5, 1, 1, 9), (0, 4, 9, 5, 0, 1), (0, 4, 5, 2, 1), (0, 4, 5, 2, 5), (0, 1, 4, 0), (0, 8, 3, 3), (0, 4, 4, 5), (0, 1, 5, 0, 3), (0, 2, 7, 4), (0, 1, 1, 5), (0, 1, 1, 5), (0, 1, 1, 5), (0, 1, 1, 5), (0, 4, 8, 5, 7, 9, 8), (0, 5, 1, 4), (0, 4, 7, 2), (0, 2, 3, 6, 5), (0, 4, 8, 5, 8, 7), (0, 4, 7, 6, 1), (0, 0, 5), (0, 4, 0, 6, 4, 1), (0, 4, 8, 8, 0, 3), (0, 2, 7, 3), (0, 1, 0, 2, 4), (0, 4, 8, 4, 4, 4), (0, 4, 8, 5, 2, 4), (0, 4, 3, 1, 6), (0, 2, 4, 0), (0, 8, 1, 3), (0, 4, 6, 5, 0), (0, 4, 8, 8, 6), (0, 4, 8, 5, 7, 0, 8), (0, 4, 8, 5, 1, 0, 0), (0, 4, 8, 5, 7, 4, 1), (0, 4, 4, 4, 2), (0, 4, 3, 2), (0, 2, 2, 5), (0, 6, 1, 2, 1), (0, 0, 4, 1), (0, 6, 3, 6), (0, 4, 0, 9), (0, 4, 3, 8), (0, 4, 8, 0, 9, 1, 2), (0, 8, 4, 7), (0, 2, 4, 1), (0, 2, 4, 2), (0, 8, 2, 2), (0, 4, 3, 8), (0, 4, 2, 9), (0, 4, 3, 8), (0, 4, 8, 0, 1), (0, 4, 3, 8), (0, 4, 0, 1, 9), (0, 8, 5, 8), (0, 2, 3, 7, 4), (0, 6, 1, 7), (0, 2, 8, 2), (0, 8, 7, 0), (0, 9, 4, 8), (0, 6, 6, 5), (0, 0, 1, 8), (0, 8, 2, 5), (0, 8, 6, 7), (0, 8, 3, 0), (0, 4, 6, 7), (0, 4, 6, 3), (0, 4, 3, 8), (0, 4, 8, 5, 7, 5, 2), (0, 8, 2, 8), (0, 8, 2, 7), (0, 8, 3, 4), (0, 8, 4, 3), (0, 4, 9, 5, 9), (0, 2, 4, 3), (0, 4, 8, 5, 2, 5), (0, 8, 6, 0), (0, 4, 8, 2, 4, 5), (0, 6, 0, 3, 0), (0, 0, 7, 4), (0, 8, 5, 7), (0, 8, 1, 3), (0, 6, 1, 0), (0, 0, 3, 1), (0, 4, 8, 5, 7, 7, 1), (0, 7, 1, 6, 0), (0, 8, 3, 4), (0, 8, 9, 8), (0, 4, 7, 6, 4), (0, 4, 8, 0, 8, 6), (0, 4, 8, 2, 4, 2), (0, 9, 5, 2), (0, 8, 9, 0), (0, 0, 9, 6), (0, 8, 1, 2), (0, 2, 4, 4), (0, 0, 0, 3), (0, 2, 5, 0), (0, 0, 0, 5, 7), (0, 8, 7, 5), (0, 8, 7, 0), (0, 8, 9, 5), (0, 2, 0, 0), (0, 2, 4, 4), (0, 8, 9, 7), (0, 2, 1, 6), (0, 8, 4, 0), (0, 8, 3, 2), (0, 8, 4, 3), (0, 1, 2, 7, 6), (0, 8, 1, 3), (0, 2, 4, 6), (0, 8, 9, 0), (0, 8, 5, 6), (0, 8, 7, 0), (0, 8, 4, 3), (0, 8, 3, 4), (0, 4, 8, 0, 2), (0, 8, 4, 6), (0, 2, 6, 6), (0, 1, 6, 3, 2), (0, 8, 9, 8), (0, 4, 2, 4, 3, 4), (0, 4, 8, 5, 7, 9, 3), (0, 5, 1, 1), (0, 8, 1, 3), (0, 5, 0, 2, 6), (0, 4, 4, 9), (0, 8, 5, 7), (0, 8, 2, 6), (0, 8, 3, 1), (0, 8, 4, 5), (0, 4, 8, 5, 8, 4, 5), (0, 2, 7, 3), (0, 8, 4, 6), (0, 9, 5, 0), (0, 1, 6, 7, 2), (0, 8, 2, 3), (0, 8, 3, 4), (0, 1, 3, 6), (0, 4, 8, 5, 7, 0, 2, 0), (0, 8, 1, 1), (0, 8, 9, 5), (0, 8, 3, 4), (0, 8, 4, 1), (0, 4, 8, 5, 9, 0), (0, 2, 4, 5), (0, 4, 2, 7, 9), (0, 4, 8, 5, 9, 0), (0, 4, 0, 5, 4), (0, 2, 4, 9), (0, 4, 5, 8, 1), (0, 4, 6, 6, 2), (0, 2, 1, 9), (0, 8, 2, 1), (0, 8, 2, 2), (0, 8, 2, 5), (0, 2, 4, 4), (0, 2, 4, 6), (0, 8, 1, 0), (0, 1, 5, 5, 3), (0, 8, 3, 3), (0, 8, 2, 4), (0, 4, 8, 8, 5), (0, 8, 3, 4), (0, 8, 6, 2), (0, 5, 8, 6), (0, 8, 3, 4), (0, 8, 9, 0), (0, 8, 7, 2), (0, 8, 5, 0), (0, 4, 8, 5, 1, 4), (0, 8, 9, 7), (0, 6, 3, 5), (0, 4, 6, 8, 8), (0, 4, 2, 4, 3, 7), (0, 8, 3, 0), (0, 8, 2, 0, 3), (0, 2, 6, 4, 1), (0, 6, 8, 7), (0, 8, 2, 0, 2), (0, 8, 3, 2), (0, 4, 8, 5, 7, 0, 0, 2), (0, 1, 0, 4), (0, 8, 2, 0, 4), (0, 8, 5, 3), (0, 4, 6, 2, 8), (0, 4, 8, 5, 2, 7), (0, 1, 6, 2), (0, 2, 0, 4), (0, 5, 0, 3), (0, 8, 2, 8), (0, 2, 5, 2), (0, 8, 3, 0), (0, 8, 4, 6), (0, 2, 2, 6), (0, 4, 8, 5, 7, 2, 1), (0, 8, 3, 4), (0, 8, 4, 6), (0, 2, 4, 2), (0, 4, 5, 7, 6), (0, 2, 4, 2), (0, 4, 8, 5, 6, 2), (0, 8, 4, 4), (0, 2, 2, 5), (0, 5, 8, 8), (0, 8, 2, 2), (0, 2, 7, 3), (0, 6, 5, 5), (0, 4, 2, 2, 3), (0, 5, 0, 0), (0, 6, 0, 1, 3), (0, 4, 8, 3, 4, 7), (0, 4, 8, 5, 3, 3), (0, 4, 8, 0, 5, 6), (0, 8, 9, 3), (0, 8, 4, 4), (0, 8, 4, 5), (0, 8, 4, 5), (0, 4, 2, 4, 3, 6), (0, 8, 9, 6), (0, 5, 0, 1), (0, 2, 5, 1), (0, 9, 3, 2, 0), (0, 1, 6, 3, 1, 9), (0, 8, 9, 9), (0, 8, 4, 5), (0, 8, 3, 4), (0, 1, 2, 5, 0), (0, 1, 5, 8, 3), (0, 2, 4, 6), (0, 8, 4, 3), (0, 4, 0, 6, 1), (0, 8, 4, 1), (0, 1, 6, 7, 7), (0, 8, 9, 9), (0, 8, 9, 3), (0, 8, 2, 0, 9), (0, 8, 2, 0, 4), (0, 8, 2, 1), (0, 9, 6, 1), (0, 2, 8, 2), (0, 1, 4, 4), (0, 4, 8, 5, 7, 0, 8), (0, 8, 5, 4), (0, 8, 4, 6), (0, 2, 4, 4), (0, 8, 6, 0), (0, 8, 2, 2), (0, 2, 9, 8), (0, 4, 6, 5, 7), (0, 8, 8), (0, 8, 9, 1), (0, 2, 4, 4), (0, 8, 3, 0), (0, 8, 2, 4), (0, 4, 8, 5, 7, 0, 1), (0, 1, 5, 2, 5), (0, 8, 5, 8), (0, 1, 3, 9, 3), (0, 2, 3, 6, 6), (0, 4, 2, 3), (0, 4, 8, 5, 7, 5, 2), (0, 8, 4, 1), (0, 2, 4, 4), (0, 8, 3, 4), (0, 6, 3, 8), (0, 8, 2, 6), (0, 8, 5, 2), (0, 8, 4, 3), (0, 8, 5, 2), (0, 1, 2, 5, 8), (0, 1, 7, 6), (0, 4, 8, 5, 7, 0, 6), (0, 8, 3, 4), (0, 8, 3, 0), (0, 8, 6, 3), (0, 8, 9, 3), (0, 8, 2, 6), (0, 2, 4, 2), (0, 2, 4, 2), (0, 8, 1, 3), (0, 8, 9, 0), (0, 2, 0, 8), (0, 2, 9, 2), (0, 6, 3, 3, 1), (0, 8, 8), (0, 2, 4, 2), (0, 8, 8), (0, 8, 3, 0), (0, 6, 1, 4), (0, 8, 9, 5), (0, 4, 2, 7, 1, 6), (0, 2, 2, 5), (0, 8, 4, 3), (0, 8, 4, 6), (0, 8, 8), (0, 8, 8), (0, 8, 8), (0, 8, 8), (0, 8, 4, 3), (0, 4, 8, 5, 7, 0, 5), (0, 1, 2, 4, 8), (0, 1, 5, 1), (0, 4, 8, 5, 7, 7, 6), (0, 4, 2, 4, 3, 0), (0, 8, 8), (0, 8, 8), (0, 8, 4, 2), (0, 8, 9, 8), (0, 6, 5, 8, 8), (0, 8, 9, 8), (0, 8, 9, 2), (0, 8, 1, 0), (0, 8, 5, 3), (0, 4, 8, 5, 0, 8), (0, 5, 8, 8), (0, 6, 7, 7), (0, 8, 1, 3), (0, 8, 3, 7), (0, 1, 3, 3), (0, 1, 2, 5, 2), (0, 4, 8, 5, 7, 4, 5), (0, 2, 4, 6), (0, 8, 3, 2), (0, 4, 7, 2), (0, 4, 3, 1, 6), (0, 2, 4, 1), (0, 8, 2, 2), (0, 4, 8, 5, 1, 6), (0, 4, 2, 2, 1), (0, 2, 4, 6), (0, 2, 2, 0), (0, 2, 3, 0), (0, 8, 6, 0), (0, 8, 4, 4), (0, 8, 2, 0, 9), (0, 8, 2, 3), (0, 4, 8, 2, 8), (0, 4, 8, 5, 7, 7, 5), (0, 8, 9, 2), (0, 4, 8, 2, 8), (0, 4, 3, 1, 6), (0, 4, 6, 0, 4), (0, 4, 6, 1, 1, 6), (0, 2, 9, 2), (0, 6, 0, 3, 1), (0, 4, 8, 5, 5, 8), (0, 4, 0, 5, 6), (0, 4, 8, 6, 5), (0, 6, 9, 3), (0, 4, 0, 3, 3), (0, 4, 2, 2, 3), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 1, 3), (0, 4, 0, 3, 0), (0, 3, 4), (0, 0, 7, 0), (0, 4, 0, 6, 3, 5), (0, 2, 3, 5), (0, 4, 8, 5, 7, 9, 5), (0, 9, 6, 1), (0, 4, 8, 3, 4, 8), (0, 2, 8, 1), (0, 4, 8, 5, 8, 6, 0), (0, 4, 8, 0, 1), (0, 9, 0, 6), (0, 6, 5, 8, 3), (0, 6, 6, 8), (0, 6, 0, 3, 1), (0, 4, 8, 6, 3), (0, 4, 6, 5, 1), (0, 2, 7, 1), (0, 4, 0, 6, 7), (0, 4, 2, 4, 3, 2), (0, 1, 9, 0, 0), (0, 6, 0, 7, 5), (0, 4, 8, 5, 7, 9, 7), (0, 1, 6, 3, 1, 4), (0, 0, 0, 2), (0, 1, 3, 6), (0, 6, 8, 2), (0, 4, 6, 1, 5), (0, 4, 8, 0, 8, 4), (0, 4, 2, 2, 4), (0, 1, 2, 7, 5), (0, 4, 6, 5, 1), (0, 9, 8, 9), (0, 9, 2, 3), (0, 4, 8, 2, 5, 4), (0, 2, 1, 9), (0, 0, 4, 7), (0, 4, 4, 1, 1, 0), (0, 1, 6, 5, 3), (0, 4, 0, 2, 3), (0, 5, 0, 1), (0, 0, 1, 9, 1), (0, 4, 6, 8, 3), (0, 0, 1, 1), (0, 0, 1, 9, 0), (0, 6, 1, 9), (0, 2, 0, 0), (0, 6, 7, 4, 2), (0, 4, 8, 8, 7), (0, 2, 0, 4), (0, 7, 1, 0, 3), (0, 7, 1, 9), (0, 1, 5, 7), (0, 4, 8, 0, 1), (0, 9, 0, 7), (0, 4, 8, 6, 4, 4), (0, 4, 0, 6, 8), (0, 4, 0, 6, 8), (0, 1, 6, 0), (0, 1, 6, 0), (0, 4, 8, 0, 1), (0, 1, 7, 5), (0, 0, 4, 4), (0, 1, 7, 9), (0, 4, 2, 4, 2), (0, 1, 6, 3, 1, 9), (0, 4, 7, 6, 2), (0, 5, 0, 2, 1, 3), (0, 2, 3, 6, 9), (0, 4, 2, 4, 3, 0), (0, 1, 5, 5, 8), (0, 4, 8, 5, 5, 6), (0, 1, 9, 2), (0, 1, 3, 9, 0), (0, 6, 6, 4, 1), (0, 4, 8, 0, 7), (0, 4, 8, 5, 7, 5, 5), (0, 0, 8, 4), (0, 4, 8, 0, 1), (0, 0, 0, 3), (0, 9, 9, 4), (0, 1, 2, 5, 2), (0, 4, 8, 5, 7, 8, 9), (0, 4, 8, 5, 3, 5), (0, 4, 8, 5, 7, 9, 9), (0, 4, 8, 5, 2, 8), (0, 6, 7, 4, 1), (0, 6, 7, 7), (0, 0, 6, 3), (0, 4, 0, 1, 2), (0, 5, 8, 1), (0, 4, 8, 0, 4), (0, 2, 8, 4), (0, 4, 8, 0, 4), (0, 4, 0, 1, 2), (0, 1, 7, 5), (0, 1, 2, 2, 8), (0, 6, 7, 4, 3), (0, 4, 0, 6, 3, 1), (0, 7, 1, 6, 0), (0, 4, 3, 4), (0, 7, 1, 6, 1), (0, 4, 0, 1, 2), (0, 4, 8, 2, 2), (0, 1, 6, 7, 0, 1), (0, 1, 2, 4, 2), (0, 4, 8, 6, 4, 1), (0, 4, 8, 0, 4), (0, 4, 8, 6, 4, 0), (0, 4, 8, 0, 2), (0, 4, 8, 0, 9, 1, 1), (0, 4, 8, 0, 4), (0, 1, 2, 2, 3), (0, 4, 8, 1, 6), (0, 4, 0, 9), (0, 4, 8, 0, 0, 4), (0, 4, 8, 8, 0, 9), (0, 2, 8, 4), (0, 1, 2, 8, 9), (0, 4, 8, 5, 7, 0, 8), (0, 4, 4, 0, 2), (0, 6, 6, 5), (0, 1, 2, 8, 2), (0, 4, 8, 5, 7, 0, 3, 1), (0, 4, 8, 5, 7, 2, 1), (0, 4, 0, 1, 6), (0, 2, 2, 5), (0, 4, 7, 8, 4), (0, 2, 0, 8), (0, 4, 8, 3, 2), (0, 7, 1, 1), (0, 4, 8, 8, 6), (0, 2, 2, 5), (0, 6, 6, 7), (0, 4, 8, 5, 3, 4, 6), (0, 4, 6, 1, 1, 1), (0, 2, 8, 4), (0, 4, 8, 5, 7, 9, 0), (0, 4, 8, 5, 3, 1), (0, 1, 6, 4, 6), (0, 1, 7, 5), (0, 4, 0, 1, 5, 0), (0, 4, 8, 5, 7, 7, 7), (0, 2, 1, 2), (0, 6, 2, 9), (0, 1, 2, 8, 1), (0, 4, 8, 5, 7, 8, 2), (0, 1, 6, 0), (0, 0, 0, 4, 5), (0, 6, 5, 6), (0, 1, 2, 2, 1), (0, 1, 6, 4, 6), (0, 1, 2, 2, 1), (0, 1, 5, 2, 6), (0, 1, 6, 7, 2), (0, 1, 6, 5, 6), (0, 1, 6, 7, 5, 3), (0, 4, 2, 1), (0, 4, 8, 4, 1), (0, 0, 9, 5), (0, 4, 8, 5, 4, 1), (0, 4, 8, 5, 7, 6, 9), (0, 1, 2, 2, 3), (0, 1, 2, 8, 2), (0, 4, 0, 1, 7), (0, 4, 6, 8, 0), (0, 4, 8, 5, 7, 0, 6), (0, 6, 9, 2), (0, 4, 8, 5, 9, 3), (0, 2, 3, 8, 3), (0, 4, 6, 5, 5), (0, 4, 6, 5, 5), (0, 1, 7, 5), (0, 4, 8, 5, 7, 9, 0), (0, 1, 6, 0), (0, 4, 8, 8, 8, 6), (0, 1, 2, 8, 1), (0, 1, 2, 2, 3), (0, 1, 9, 2), (0, 4, 8, 5, 7, 4, 1), (0, 4, 8, 5, 7, 4, 2), (0, 2, 7, 2), (0, 4, 8, 5, 7, 5, 3), (0, 0, 6, 5), (0, 2, 0, 2), (0, 1, 4, 1), (0, 4, 5, 4, 6), (0, 1, 7, 5), (0, 4, 8, 5, 3, 4, 9), (0, 2, 3, 5), (0, 4, 8, 5, 2, 8), (0, 4, 2, 1), (0, 9, 9, 4), (0, 6, 0, 2), (0, 4, 2, 1), (0, 4, 0, 2, 5), (0, 1, 2, 6), (0, 4, 8, 5, 7, 7, 8), (0, 2, 0, 4), (0, 6, 1, 7), (0, 0, 2, 0), (0, 0, 0, 3), (0, 4, 8, 8, 8, 1), (0, 4, 7, 4), (0, 1, 4, 9), (0, 1, 4, 9), (0, 6, 7, 3, 8), (0, 9, 3, 2, 7, 0), (0, 1, 0, 8), (0, 4, 8, 5, 1, 0, 1), (0, 4, 8, 5, 7, 8, 7), (0, 1, 6, 4, 7), (0, 1, 7, 5), (0, 4, 8, 5, 7, 8, 3), (0, 4, 8, 5, 7, 2, 1), (0, 6, 7, 3, 9), (0, 4, 8, 2, 5, 5), (0, 4, 2, 2, 6), (0, 2, 3, 8, 5), (0, 4, 8, 5, 7, 7, 5), (0, 5, 4, 3), (0, 4, 8, 5, 7, 7, 5), (0, 4, 8, 5, 7, 0, 8), (0, 4, 2, 2, 1), (0, 4, 4, 0, 2), (0, 4, 0, 7, 4), (0, 1, 5, 1), (0, 4, 8, 5, 3, 4, 3), (0, 4, 2, 2, 6), (0, 1, 6, 0), (0, 1, 2, 2, 8), (0, 4, 5, 9), (0, 6, 3, 2), (0, 0, 0, 3), (0, 4, 8, 5, 7, 9, 8), (0, 4, 8, 2, 5, 5), (0, 4, 9, 2, 6), (0, 6, 3, 2), (0, 4, 3, 1, 6), (0, 6, 5, 1), (0, 4, 8, 5, 2, 8), (0, 4, 8, 5, 2, 0), (0, 6, 0, 5, 4), (0, 1, 7, 5), (0, 1, 2, 8, 1), (0, 0, 0, 4, 6), (0, 6, 8, 1), (0, 6, 3, 2), (0, 4, 8, 0, 9, 1, 1), (0, 6, 0, 6), (0, 1, 6, 3, 1, 5), (0, 1, 6, 8), (0, 6, 0, 5, 4), (0, 1, 3, 5), (0, 4, 7, 2), (0, 0, 2, 6), (0, 1, 1, 4), (0, 6, 1, 3), (0, 4, 8, 5, 1, 2), (0, 4, 8, 8, 8, 8), (0, 6, 6, 5), (0, 4, 8, 0, 2), (0, 4, 0, 9), (0, 0, 1, 0), (0, 4, 8, 9, 0), (0, 4, 8, 5, 7, 0, 9, 1), (0, 4, 6, 2, 6), (0, 4, 0, 1, 2), (0, 1, 2, 2, 1), (0, 4, 7, 8, 1), (0, 4, 8, 0, 5, 1), (0, 1, 5, 6, 4), (0, 4, 4, 6), (0, 1, 8, 1, 1), (0, 4, 0, 1, 2), (0, 4, 8, 5, 7, 4, 1), (0, 1, 5, 4), (0, 4, 6, 6, 1), (0, 1, 6, 7, 5, 2), (0, 1, 5, 6, 6), (0, 8, 9, 4), (0, 6, 8, 2), (0, 4, 5, 4, 1, 2), (0, 2, 0, 9), (0, 1, 6, 4, 6), (0, 6, 1, 8), (0, 6, 0, 7, 4), (0, 6, 2, 2), (0, 1, 6, 0), (0, 1, 2, 7, 3), (0, 6, 7, 4, 2), (0, 1, 9, 7), (0, 0, 9, 3), (0, 4, 0, 6, 8), (0, 4, 2, 4, 2), (0, 4, 8, 8, 2), (0, 4, 8, 5, 7, 0, 0, 4), (0, 4, 0, 5, 0), (0, 1, 6, 7, 5, 0), (0, 1, 4, 0), (0, 5, 0, 7), (0, 1, 7, 5), (0, 0, 0, 3), (0, 0, 2, 6), (0, 6, 0, 7, 3), (0, 7, 9, 2), (0, 1, 2, 2, 9), (0, 1, 2, 2, 8), (0, 1, 2, 2, 0), (0, 1, 6, 7, 3), (0, 0, 8, 0), (0, 6, 8, 8), (0, 6, 3, 8), (0, 4, 8, 5, 7, 7, 1), (0, 4, 8, 2, 4, 2), (0, 2, 7, 2), (0, 4, 8, 5, 2, 1), (0, 4, 6, 6, 2), (0, 4, 5, 4, 1, 4, 7), (0, 6, 7, 4, 9), (0, 1, 8, 8), (0, 4, 8, 5, 7, 6, 9), (0, 6, 3, 7), (0, 4, 8, 5, 7, 0, 1), (0, 4, 0, 1, 5, 6), (0, 0, 6, 4), (0, 0, 2, 3), (0, 9, 3, 9), (0, 1, 6, 6, 8), (0, 0, 1, 6), (0, 0, 2, 3), (0, 6, 0, 4), (0, 9, 2, 0), (0, 4, 2, 1), (0, 1, 4, 4), (0, 1, 5, 6, 8), (0, 4, 8, 8, 8, 1), (0, 6, 7, 0, 1), (0, 4, 8, 5, 7, 7, 6), (0, 4, 2, 1), (0, 1, 8, 7), (0, 1, 6, 2), (0, 4, 8, 5, 7, 3, 4), (0, 4, 8, 5, 1, 4), (0, 2, 3, 7, 5), (0, 4, 6, 1, 9), (0, 4, 8, 5, 7, 0, 1), (0, 4, 8, 5, 2, 4), (0, 9, 2, 9), (0, 4, 5, 2, 8), (0, 4, 5, 2, 5), (0, 4, 8, 0, 5, 3), (0, 8, 0), (0, 4, 8, 0, 2), (0, 6, 7, 0, 1), (0, 4, 8, 5, 6, 0), (0, 4, 0, 6, 3, 5), (0, 4, 2, 4, 0), (0, 5, 0, 2, 1, 1), (0, 2, 6, 4, 9), (0, 1, 2, 3), (0, 9, 3, 0), (0, 4, 8, 2, 1, 2), (0, 6, 5, 4), (0, 5, 1, 9), (0, 4, 2, 9), (0, 1, 4, 6, 4), (0, 4, 8, 2, 4, 8), (0, 4, 8, 2, 4, 2), (0, 0, 9, 2, 0), (0, 1, 8, 1, 8), (0, 2, 3, 6, 8), (0, 1, 6, 4, 6), (0, 4, 8, 5, 2, 4), (0, 4, 9, 1), (0, 4, 8, 5, 5, 1), (0, 5, 8, 5), (0, 4, 8, 2, 4, 1), (0, 4, 8, 5, 5, 1), (0, 4, 8, 0, 0, 4), (0, 0, 2, 1), (0, 4, 8, 5, 6, 0), (0, 1, 6, 7, 0, 4), (0, 4, 5, 4, 1, 4, 2), (0, 0, 9, 7), (0, 4, 0, 1, 4), (0, 2, 9, 9), (0, 0, 8, 1), (0, 1, 0, 2, 2), (0, 4, 6, 2, 3), (0, 4, 8, 5, 7, 0, 9, 1), (0, 6, 3, 2), (0, 0, 1, 7), (0, 0, 2, 3), (0, 9, 3, 2, 8), (0, 7, 1, 1), (0, 2, 9, 4), (0, 4, 8, 5, 4, 1), (0, 7, 1, 1), (0, 6, 8, 5), (0, 0, 0, 5, 5), (0, 7, 1, 1), (0, 7, 1, 1), (0, 2, 0, 8), (0, 4, 8, 5, 7, 2, 5), (0, 0, 0, 1), (0, 4, 8, 0, 9, 7), (0, 4, 0, 2, 8), (0, 4, 0, 1, 1), (0, 4, 7, 6, 4), (0, 8, 0), (0, 4, 8, 5, 7, 9, 4), (0, 4, 2, 2, 9), (0, 1, 5, 0, 2), (0, 4, 8, 5, 7, 6, 3), (0, 4, 5, 4, 0), (0, 9, 8, 1), (0, 2, 8, 4), (0, 4, 0, 3, 2), (0, 4, 7, 6, 1), (0, 0, 0, 5, 1), (0, 2, 3, 8, 8), (0, 4, 8, 0, 0, 2), (0, 4, 8, 5, 1, 0, 1), (0, 0, 7, 0), (0, 4, 0, 1, 5, 1), (0, 9, 9, 9), (0, 2, 0, 3), (0, 0, 2, 4), (0, 1, 5, 0, 6), (0, 6, 7, 0, 6), (0, 1, 6, 7, 6), (0, 1, 6, 7, 7), (0, 4, 7, 2), (0, 8, 0), (0, 0, 5), (0, 0, 4, 1), (0, 4, 8, 5, 1, 6), (0, 4, 8, 4, 4, 4), (0, 1, 2, 3), (0, 1, 6, 0), (0, 4, 8, 5, 7, 7, 1), (0, 4, 2, 2, 1), (0, 4, 2, 1), (0, 7, 1, 3), (0, 1, 6, 5, 1), (0, 1, 5, 8, 4), (0, 0, 0, 5, 1), (0, 4, 0, 2, 6), (0, 9, 8, 1), (0, 4, 8, 5, 8, 1), (0, 7, 1, 6, 0), (0, 9, 9, 5), (0, 0, 1, 1), (0, 4, 5, 4, 9), (0, 4, 1, 7), (0, 6, 1, 1, 7), (0, 6, 0, 1, 0), (0, 9, 0, 8), (0, 6, 7, 4, 1), (0, 4, 0, 7, 1), (0, 9, 3, 9), (0, 4, 8, 4, 6), (0, 4, 8, 5, 7, 4, 5), (0, 1, 4, 4), (0, 4, 8, 8, 7), (0, 2, 9, 2), (0, 4, 8, 5, 9, 4), (0, 1, 6, 0), (0, 4, 8, 5, 1, 0, 1), (0, 7, 1, 5, 4), (0, 7, 1, 6, 0), (0, 4, 8, 5, 1, 1, 4), (0, 4, 8, 0, 5, 3), (0, 4, 8, 5, 1, 8), (0, 5, 1, 0), (0, 4, 3, 3), (0, 4, 0, 0, 2), (0, 4, 2, 4, 1), (0, 2, 5, 4), (0, 4, 8, 5, 0, 0), (0, 6, 3, 3, 2), (0, 4, 0, 3, 0), (0, 9, 9, 9), (0, 4, 0, 3, 0), (0, 1, 5, 3), (0, 0, 2, 9), (0, 6, 9, 8, 7), (0, 6, 8, 6), (0, 4, 8, 5, 4, 1), (0, 2, 0, 3), (0, 4, 6, 1, 8), (0, 2, 0, 1), (0, 2, 5, 2), (0, 4, 0, 1, 7), (0, 4, 8, 5, 1, 9), (0, 1, 9, 1), (0, 9, 0, 1, 9), (0, 9, 2, 7), (0, 9, 5, 1), (0, 4, 8, 6, 3), (0, 4, 5, 8, 9), (0, 2, 5, 2), (0, 0, 9, 5), (0, 2, 5, 2), (0, 9, 5, 5), (0, 9, 0, 1, 2), (0, 0, 3, 0), (0, 4, 8, 1, 6), (0, 1, 4, 3, 3), (0, 7, 1, 5, 4), (0, 4, 4, 5), (0, 4, 4, 5), (0, 4, 4, 8), (0, 4, 8, 5, 7, 0, 3, 2), (0, 7, 1, 0, 1), (0, 4, 8, 5, 7, 1, 2), (0, 4, 8, 5, 7, 0, 3, 1), (0, 4, 8, 5, 7, 9, 4), (0, 4, 8, 5, 7, 9, 1), (0, 4, 8, 5, 1, 1, 6), (0, 4, 8, 5, 1, 6), (0, 6, 1, 2, 5), (0, 4, 4, 5), (0, 4, 4, 5), (0, 9, 2, 1), (0, 9, 2, 2), (0, 4, 6, 1, 1, 7), (0, 2, 0, 8), (0, 4, 8, 5, 1, 5), (0, 5, 3, 3, 0), (0, 6, 6, 9), (0, 4, 8, 5, 7, 8, 2), (0, 6, 7, 5, 0), (0, 9, 8, 7), (0, 1, 8, 0), (0, 9, 0, 1, 3), (0, 1, 1, 0), (0, 1, 6, 5, 1), (0, 1, 6, 7, 3), (0, 4, 4, 4, 8), (0, 4, 6, 0, 4), (0, 4, 8, 4, 1), (0, 2, 7, 7), (0, 4, 8, 0, 5, 2), (0, 6, 7, 1), (0, 2, 8, 4), (0, 5, 8, 1), (0, 4, 8, 5, 0, 6), (0, 4, 6, 7), (0, 9, 5, 3), (0, 6, 7, 7), (0, 4, 8, 6, 1), (0, 6, 0, 1, 4), (0, 1, 9, 8), (0, 4, 2, 2, 4), (0, 4, 8, 9, 4, 1), (0, 4, 8, 5, 1, 2), (0, 4, 8, 5, 3, 0), (0, 4, 8, 5, 0, 2), (0, 1, 6, 3, 6), (0, 0, 9, 7), (0, 6, 0, 8, 5), (0, 1, 7, 7, 1), (0, 2, 2, 5), (0, 0, 4, 7), (0, 4, 2, 0), (0, 0, 9, 0), (0, 1, 0, 8), (0, 4, 8, 5, 7, 5, 4), (0, 1, 9, 0, 0), (0, 4, 8, 5, 4, 2), (0, 1, 6, 9, 3), (0, 0, 4, 1), (0, 0, 1, 9, 1), (0, 1, 2, 5, 3), (0, 2, 7, 2), (0, 4, 0, 3, 5), (0, 1, 6, 7, 0, 1), (0, 5, 0, 6), (0, 4, 8, 0, 0, 0), (0, 4, 8, 5, 7, 9, 5), (0, 1, 5, 8, 9), (0, 6, 3, 9), (0, 7, 1, 0, 0), (0, 4, 0, 6, 4, 1), (0, 4, 8, 5, 7, 9, 8), (0, 4, 8, 0, 3), (0, 4, 8, 5, 7, 6, 1), (0, 4, 1, 4), (0, 4, 1, 6), (0, 0, 0, 0), (0, 0, 4, 9), (0, 0, 4, 9), (0, 4, 8, 5, 8, 2), (0, 2, 6, 4, 1), (0, 4, 8, 5, 7, 5, 4), (0, 0, 0, 9), (0, 4, 8, 5, 3, 5), (0, 4, 8, 5, 3, 5), (0, 9, 3, 2, 9), (0, 4, 6, 1, 1, 1), (0, 1, 0, 2, 4), (0, 4, 4, 6), (0, 4, 8, 5, 6, 9), (0, 4, 2, 4, 1), (0, 4, 4, 1, 1, 0), (0, 5, 0, 2, 1, 6), (0, 4, 8, 5, 2, 1), (0, 4, 8, 5, 6, 1), (0, 4, 9, 2, 4), (0, 4, 8, 1, 6), (0, 4, 6, 2, 5), (0, 0, 9, 3), (0, 0, 3, 1), (0, 4, 8, 5, 7, 0, 2, 5), (0, 1, 6, 7, 3), (0, 0, 8, 7), (0, 4, 1, 6), (0, 4, 1, 6), (0, 4, 1, 6), (0, 4, 1, 4), (0, 4, 1, 6), (0, 4, 5, 4, 1, 4, 2), (0, 4, 1, 6), (0, 4, 4, 4, 4), (0, 9, 0, 1, 1), (0, 4, 7, 6, 3), (0, 5, 0, 2, 0), (0, 2, 0, 9), (0, 4, 0, 6, 3, 1), (0, 4, 8, 6, 1), (0, 6, 6, 2), (0, 4, 8, 5, 6, 0), (0, 9, 4, 3), (0, 2, 6, 4, 4), (0, 4, 2, 2, 3), (0, 4, 1, 6), (0, 1, 1, 8), (0, 4, 8, 0, 9, 1, 0), (0, 4, 1, 1, 3), (0, 0, 8, 3), (0, 0, 2, 5), (0, 4, 8, 5, 3, 3), (0, 6, 0, 9, 0), (0, 4, 1, 5), (0, 4, 5, 7, 0), (0, 4, 8, 4, 5), (0, 4, 0, 2, 1), (0, 1, 6, 8), (0, 1, 3, 3), (0, 4, 0, 2, 1), (0, 2, 6, 4, 6), (0, 1, 6, 7, 5, 5), (0, 2, 6, 3), (0, 4, 8, 8, 7), (0, 9, 0, 4), (0, 0, 2, 7), (0, 0, 3, 3), (0, 4, 1, 3), (0, 4, 1, 3), (0, 2, 9, 5), (0, 4, 6, 1, 1, 9), (0, 4, 8, 8, 9), (0, 4, 8, 0, 4), (0, 6, 5, 0), (0, 2, 6, 4, 1), (0, 5, 1, 6), (0, 4, 0, 3, 2), (0, 4, 1, 3), (0, 4, 8, 3, 4, 6), (0, 4, 1, 3), (0, 6, 0, 1, 3), (0, 1, 3, 1), (0, 3, 3, 4), (0, 4, 8, 7, 0), (0, 4, 8, 6, 0), (0, 2, 6, 3), (0, 2, 6, 4, 1), (0, 6, 0, 7, 1), (0, 2, 0, 1), (0, 2, 0, 1), (0, 4, 8, 2, 1, 2), (0, 5, 1, 0), (0, 4, 1, 3), (0, 4, 1, 3), (0, 4, 8, 7, 0), (0, 4, 5, 4, 1, 4, 8), (0, 4, 1, 3), (0, 4, 1, 3), (0, 4, 1, 3), (0, 4, 1, 3), (0, 3, 4), (0, 7, 1, 9), (0, 4, 8, 7, 0), (0, 4, 0, 0, 1), (0, 3, 3, 3), (0, 4, 8, 1, 6), (0, 3, 4), (0, 4, 2, 7, 1, 1), (0, 4, 8, 7, 0), (0, 2, 7, 7), (0, 4, 8, 7, 5), (0, 4, 8, 5, 2, 4), (0, 4, 8, 5, 7, 0, 0, 2), (0, 1, 9, 8), (0, 5, 1, 1), (0, 2, 7, 0, 5), (0, 4, 1, 3), (0, 4, 1, 3), (0, 4, 8, 5, 7, 6, 7), (0, 8, 3, 5), (0, 0, 4, 1), (0, 4, 8, 5, 3, 4, 1), (0, 2, 5, 1), (0, 6, 0, 1, 0), (0, 0, 4, 1), (0, 6, 1, 1, 3), (0, 4, 8, 5, 2, 3), (0, 0, 4, 1), (0, 0, 0, 5, 5), (0, 4, 5, 4, 8), (0, 4, 5, 3), (0, 0, 6, 5), (0, 1, 6, 7, 2), (0, 1, 6, 4, 7), (0, 4, 2, 4, 9), (0, 2, 3, 1), (0, 4, 8, 7, 1), (0, 4, 8, 5, 7, 5, 4), (0, 6, 8, 2), (0, 2, 8, 3), (0, 4, 8, 5, 8, 1), (0, 2, 5, 1), (0, 4, 1, 3), (0, 4, 1, 0), (0, 4, 1, 3), (0, 4, 8, 9, 2), (0, 0, 9, 2, 5), (0, 2, 7, 9), (0, 7, 7, 2), (0, 4, 8, 5, 3, 1), (0, 4, 4, 9), (0, 4, 8, 5, 7, 2, 5), (0, 2, 3, 9), (0, 9, 3, 1), (0, 2, 3, 8, 8), (0, 2, 3, 9), (0, 4, 8, 0, 4), (0, 9, 8, 1), (0, 4, 8, 1, 6), (0, 4, 6, 9), (0, 4, 9, 2, 9), (0, 4, 8, 5, 1, 1, 3), (0, 4, 2, 0), (0, 4, 8, 5, 4, 2), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 2, 1), (0, 4, 8, 5, 7, 6, 1), (0, 4, 2, 8), (0, 2, 7, 7), (0, 4, 9, 2, 1), (0, 1, 4, 4), (0, 1, 9, 1), (0, 4, 1, 3), (0, 4, 1, 3), (0, 9, 5, 1), (0, 9, 3, 6), (0, 4, 8, 3, 6), (0, 1, 0, 2, 5), (0, 1, 2, 5, 6), (0, 4, 8, 5, 7, 9, 9), (0, 4, 5, 7, 5), (0, 6, 7, 4, 1), (0, 1, 5, 5, 6), (0, 4, 8, 5, 7, 0, 0, 1), (0, 4, 8, 5, 7, 0, 9, 8), (0, 6, 0, 8, 6), (0, 4, 1, 7), (0, 4, 3, 3), (0, 1, 2, 5, 3), (0, 0, 7, 1), (0, 4, 8, 5, 7, 8, 7), (0, 6, 8, 3), (0, 6, 3, 8), (0, 4, 6, 2, 7), (0, 2, 7, 7), (0, 1, 4, 6, 6), (0, 4, 8, 6, 4, 8), (0, 2, 3, 9), (0, 4, 1, 6), (0, 4, 8, 5, 5, 8), (0, 4, 8, 5, 0, 4), (0, 6, 5, 2), (0, 8, 0), (0, 4, 8, 8, 8, 1), (0, 4, 8, 8, 5), (0, 1, 6, 3, 1, 3), (0, 9, 3, 2, 7, 1), (0, 5, 0, 3), (0, 5, 8, 9), (0, 2, 3, 2, 1), (0, 4, 8, 1, 2), (0, 5, 1, 7), (0, 4, 8, 5, 7, 6, 6), (0, 4, 8, 5, 7, 7, 1), (0, 2, 3, 8, 8), (0, 4, 8, 0, 5, 4), (0, 6, 7, 0, 6), (0, 4, 6, 2, 7), (0, 4, 8, 5, 4, 1), (0, 4, 8, 5, 7, 0, 1), (0, 2, 8, 3), (0, 4, 8, 5, 1, 1, 3), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 7, 0, 1), (0, 9, 0, 1, 3), (0, 1, 0, 0), (0, 1, 5, 0, 8), (0, 2, 3, 4), (0, 2, 3, 9), (0, 2, 3, 4), (0, 2, 3, 4), (0, 4, 8, 5, 7, 2, 5), (0, 2, 7, 7), (0, 4, 8, 5, 7, 0, 4), (0, 1, 0, 5), (0, 1, 2, 9), (0, 1, 2, 5, 7, 9), (0, 9, 0, 7), (0, 4, 0, 4, 3), (0, 4, 2, 0), (0, 1, 5, 9), (0, 4, 8, 5, 3, 2), (0, 6, 9, 8, 4), (0, 1, 4, 6, 0), (0, 4, 2, 0), (0, 1, 6, 4, 8), (0, 1, 2, 5, 4), (0, 4, 8, 5, 5, 1), (0, 1, 1, 3), (0, 4, 2, 3), (0, 2, 1, 4), (0, 0, 2, 0), (0, 0, 2, 5), (0, 6, 0, 2), (0, 4, 8, 0, 7), (0, 1, 0, 9), (0, 1, 6, 7, 5, 1), (0, 6, 1, 1, 6), (0, 4, 2, 0), (0, 4, 8, 5, 3, 6), (0, 4, 8, 5, 7, 1, 0), (0, 4, 2, 2, 8), (0, 6, 6, 8), (0, 5, 0, 2, 1, 2), (0, 2, 5, 9), (0, 0, 5), (0, 4, 8, 5, 2, 1), (0, 4, 4, 1, 2), (0, 2, 7, 0, 4), (0, 4, 4, 1, 2), (0, 4, 0, 6, 6), (0, 7, 5, 3), (0, 1, 7, 7, 1), (0, 1, 9, 3), (0, 4, 7, 7), (0, 2, 3, 9), (0, 2, 3, 8, 8), (0, 0, 2, 2), (0, 6, 7, 3, 4), (0, 4, 8, 5, 8, 8), (0, 4, 8, 0, 9, 1, 7), (0, 4, 0, 5, 9), (0, 7, 5, 7), (0, 1, 7, 7, 5), (0, 2, 9, 9), (0, 4, 8, 5, 9, 4), (0, 2, 9, 9), (0, 4, 8, 0, 6), (0, 4, 8, 5, 3, 2), (0, 8, 4, 6), (0, 4, 8, 5, 1, 7), (0, 4, 8, 5, 9, 8), (0, 6, 7, 3, 5), (0, 6, 0, 8, 0), (0, 4, 2, 2, 2), (0, 4, 8, 5, 1, 1, 6), (0, 0, 6, 1), (0, 1, 5, 8, 5), (0, 1, 6, 7, 0, 4), (0, 6, 7, 9), (0, 1, 6, 7, 4), (0, 1, 6, 4, 2), (0, 4, 9, 3), (0, 2, 3, 8, 0), (0, 5, 8, 6), (0, 4, 8, 0, 7), (0, 4, 8, 3, 4, 1), (0, 2, 9, 6), (0, 5, 0, 6), (0, 1, 6, 7, 6), (0, 2, 3, 8, 1), (0, 4, 4, 0, 1), (0, 1, 2, 1), (0, 1, 5, 6, 5), (0, 5, 0, 6), (0, 5, 0, 9), (0, 1, 6, 0), (0, 4, 8, 5, 7, 0, 9, 3), (0, 4, 8, 5, 4, 8, 4), (0, 2, 9, 5), (0, 0, 0, 4, 3), (0, 0, 0, 0), (0, 0, 0, 5, 5), (0, 4, 4, 5), (0, 2, 1, 1), (0, 4, 8, 5, 1, 1, 1), (0, 4, 8, 5, 3, 0), (0, 4, 0, 2, 2), (0, 1, 0, 0), (0, 0, 4, 4), (0, 2, 6, 6), (0, 2, 0, 1), (0, 0, 9, 2, 1), (0, 1, 2, 5, 2), (0, 2, 9, 9), (0, 4, 4, 3), (0, 9, 5, 6), (0, 4, 0, 1, 6), (0, 4, 8, 5, 2, 9), (0, 7, 5, 6), (0, 4, 8, 0, 3), (0, 4, 8, 5, 2, 6), (0, 5, 8, 8), (0, 4, 8, 5, 7, 2, 1), (0, 0, 4, 1), (0, 0, 1, 1), (0, 0, 0, 5, 5), (0, 1, 5, 8, 1), (0, 4, 8, 3, 4, 6), (0, 4, 8, 0, 6), (0, 2, 6, 8), (0, 6, 0, 4), (0, 4, 8, 5, 7, 6, 6), (0, 6, 3, 4), (0, 5, 0, 2, 1, 7), (0, 5, 0, 2, 1, 1), (0, 1, 5, 8, 1), (0, 4, 8, 5, 3, 3), (0, 1, 8, 3), (0, 4, 8, 5, 7, 3, 2), (0, 6, 1, 8), (0, 4, 0, 8, 9), (0, 4, 4, 1, 1, 7), (0, 4, 8, 5, 7, 9, 4), (0, 4, 2, 3), (0, 0, 9, 2, 3), (0, 1, 2, 5, 7, 7), (0, 1, 5, 8, 9), (0, 0, 2, 1), (0, 5, 0, 2, 1, 1), (0, 4, 8, 5, 1, 7), (0, 4, 3, 8), (0, 4, 8, 5, 4, 8, 1), (0, 1, 6, 9, 9), (0, 9, 8, 4), (0, 6, 7, 0, 3), (0, 6, 7, 0, 3), (0, 4, 0, 1, 5, 5), (0, 7, 9, 3), (0, 4, 3, 3), (0, 9, 9, 5), (0, 1, 6, 0), (0, 0, 7, 3), (0, 2, 2, 0), (0, 0, 1, 1), (0, 2, 1, 1), (0, 6, 0, 9, 1), (0, 4, 8, 9, 1), (0, 0, 1, 9, 7), (0, 6, 1, 2, 5), (0, 4, 0, 8, 2), (0, 4, 8, 0, 9, 4), (0, 6, 9, 8, 0), (0, 5, 8, 0), (0, 4, 0, 4, 5), (0, 0, 1, 9, 1), (0, 4, 8, 5, 7, 7, 3), (0, 4, 8, 0, 2), (0, 0, 4, 9), (0, 4, 8, 4, 1), (0, 4, 8, 8, 7), (0, 4, 4, 1, 1, 4), (0, 4, 8, 2, 2), (0, 4, 8, 8, 0, 2), (0, 9, 7, 6), (0, 4, 8, 9, 2), (0, 4, 9, 4), (0, 4, 8, 5, 9, 8), (0, 6, 0, 9, 2), (0, 1, 3, 2), (0, 9, 8, 1), (0, 4, 8, 5, 7, 2, 1), (0, 5, 3, 3, 3), (0, 1, 2, 5, 4), (0, 4, 8, 5, 7, 8, 3), (0, 0, 7, 5), (0, 9, 7, 6), (0, 9, 4, 3), (0, 4, 8, 5, 7, 4, 5), (0, 4, 4, 1, 0), (0, 1, 0, 4), (0, 4, 8, 2, 5, 5), (0, 0, 4, 3), (0, 4, 8, 5, 2, 3), (0, 6, 0, 5, 3), (0, 0, 9, 4), (0, 9, 9, 2), (0, 4, 3, 1, 0), (0, 6, 6, 4, 0), (0, 1, 6, 9, 7), (0, 6, 0, 3, 6), (0, 4, 7, 0), (0, 2, 6, 4, 8), (0, 4, 9, 2, 3), (0, 4, 8, 5, 1, 9), (0, 4, 0, 2, 3), (0, 4, 3, 5), (0, 9, 0, 4), (0, 1, 6, 0), (0, 4, 8, 5, 8, 8), (0, 9, 8, 0), (0, 4, 5, 9), (0, 1, 4, 6, 2), (0, 4, 8, 5, 3, 5), (0, 4, 6, 1, 1, 1), (0, 4, 0, 1, 5, 4), (0, 4, 3, 5), (0, 4, 5, 4, 1, 2), (0, 0, 7, 4), (0, 9, 9, 2), (0, 4, 8, 5, 9, 6), (0, 2, 9, 2), (0, 4, 0, 1, 1), (0, 6, 0, 0, 4), (0, 6, 6, 5), (0, 4, 8, 5, 2, 7), (0, 7, 4, 6), (0, 1, 7, 7, 1), (0, 2, 1, 6), (0, 4, 3, 1, 7), (0, 4, 7, 8, 6), (0, 4, 2, 2, 9), (0, 0, 9, 4), (0, 6, 0, 7, 0), (0, 4, 6, 5, 3), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 9, 0), (0, 0, 8, 7), (0, 4, 3, 2), (0, 4, 3, 2), (0, 4, 3, 0), (0, 6, 2, 3), (0, 4, 8, 5, 1, 1, 6), (0, 4, 3, 0), (0, 4, 8, 5, 7, 9, 1), (0, 1, 8, 1, 1), (0, 4, 8, 2, 4, 2), (0, 4, 2, 2, 1), (0, 4, 3, 0), (0, 9, 0, 1, 2), (0, 4, 3, 3), (0, 4, 3, 2), (0, 0, 8, 7), (0, 4, 3, 2), (0, 1, 1, 3), (0, 4, 3, 0), (0, 4, 2, 4, 8), (0, 1, 3, 9, 7), (0, 2, 3, 6, 5), (0, 4, 8, 5, 7, 0, 5), (0, 1, 3, 3), (0, 5, 1, 1), (0, 4, 2, 4, 3, 0), (0, 4, 8, 5, 1, 7), (0, 4, 8, 5, 8, 2), (0, 1, 2, 7, 2), (0, 1, 2, 4, 7), (0, 4, 6, 2, 2), (0, 4, 8, 2, 8), (0, 4, 8, 0, 3), (0, 1, 0, 4), (0, 4, 3, 2), (0, 9, 6, 6), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 7, 0), (0, 4, 5, 4, 1, 2), (0, 4, 0, 1, 5, 4), (0, 4, 2, 4, 2), (0, 1, 5, 4), (0, 4, 0, 1, 2), (0, 4, 4, 9), (0, 4, 0, 1, 5, 4), (0, 4, 3, 0), (0, 4, 3, 0), (0, 1, 8, 1, 1), (0, 4, 9, 5, 6), (0, 4, 3, 8), (0, 5, 8, 4), (0, 1, 6, 3, 1, 1), (0, 5, 8, 4), (0, 5, 8, 1), (0, 1, 6, 7, 3), (0, 4, 8, 6, 4, 9), (0, 5, 3, 5), (0, 5, 8, 4), (0, 1, 5, 2, 5), (0, 4, 8, 5, 7, 0, 9, 6), (0, 2, 5, 3), (0, 4, 6, 2, 8), (0, 1, 5, 5, 1), (0, 4, 8, 5, 7, 8, 7), (0, 4, 2, 7, 0), (0, 1, 8, 1, 1), (0, 4, 8, 2, 4, 2), (0, 1, 6, 8), (0, 4, 6, 5, 1), (0, 4, 3, 9), (0, 4, 8, 5, 7, 7, 5), (0, 1, 3, 2), (0, 4, 3, 0), (0, 4, 4, 4, 0), (0, 9, 7, 3), (0, 5, 3, 5), (0, 1, 2, 7, 2), (0, 4, 5, 9), (0, 9, 8, 7), (0, 5, 2), (0, 6, 9, 0), (0, 2, 9, 9), (0, 4, 8, 5, 5, 1), (0, 4, 3, 6), (0, 4, 3, 6), (0, 1, 0, 7), (0, 4, 3, 6), (0, 4, 4, 9), (0, 4, 8, 5, 1, 4), (0, 4, 3, 6), (0, 5, 1, 2), (0, 2, 3, 6, 2), (0, 2, 7, 7), (0, 4, 3, 0), (0, 1, 9, 2), (0, 4, 8, 5, 7, 9, 3), (0, 4, 4, 4, 9), (0, 0, 9, 3), (0, 4, 3, 6), (0, 6, 1, 1, 6), (0, 5, 3, 6), (0, 6, 7, 8), (0, 4, 2, 4, 2), (0, 4, 3, 6), (0, 0, 9, 6), (0, 4, 3, 6), (0, 4, 3, 6), (0, 9, 8, 3), (0, 6, 0, 5, 9), (0, 6, 9, 4), (0, 9, 3, 1), (0, 4, 8, 5, 3, 2), (0, 4, 8, 0, 8, 1), (0, 2, 3, 3), (0, 2, 8, 8), (0, 2, 3, 0), (0, 4, 5, 4, 5), (0, 4, 8, 5, 5, 7), (0, 0, 2, 2), (0, 1, 6, 2), (0, 1, 6, 7, 8), (0, 4, 3, 1, 0), (0, 4, 3, 5), (0, 4, 8, 6, 4, 1), (0, 4, 4, 3), (0, 4, 0, 2, 7), (0, 1, 6, 2), (0, 4, 7, 8, 5), (0, 4, 8, 2, 5, 4), (0, 7, 3, 8), (0, 1, 5, 5, 9), (0, 6, 0, 4), (0, 0, 0, 1), (0, 4, 3, 3), (0, 5, 0, 1), (0, 9, 9, 9), (0, 4, 8, 5, 6, 3), (0, 0, 3, 2), (0, 2, 1, 4), (0, 1, 6, 3, 1, 1), (0, 9, 0, 7), (0, 4, 2, 2, 6), (0, 6, 1, 2, 2), (0, 4, 8, 0, 9, 1, 1), (0, 4, 8, 8, 8, 0), (0, 9, 5, 3), (0, 4, 3, 1, 9), (0, 6, 3, 6), (0, 4, 4, 0, 2), (0, 0, 0, 3), (0, 1, 1, 3), (0, 6, 8, 5), (0, 4, 3, 1, 6), (0, 2, 8, 0), (0, 2, 8, 6), (0, 2, 8, 0), (0, 0, 0, 5, 0), (0, 2, 3, 5), (0, 2, 9, 1), (0, 4, 8, 4, 1), (0, 0, 6, 1), (0, 4, 3, 6), (0, 9, 5, 4), (0, 4, 8, 5, 8, 4, 2), (0, 9, 0, 0), (0, 4, 8, 0, 9, 1, 8), (0, 4, 8, 5, 1, 1, 7), (0, 9, 3, 1), (0, 4, 8, 5, 3, 5), (0, 4, 8, 5, 8, 2), (0, 9, 0, 4), (0, 4, 8, 7, 5), (0, 4, 3, 0), (0, 4, 8, 5, 8, 9), (0, 9, 3, 2, 1), (0, 9, 0, 0), (0, 4, 5, 2, 5), (0, 4, 6, 1, 1, 7), (0, 6, 6, 9), (0, 6, 6, 5), (0, 4, 9, 2, 4), (0, 4, 3, 1, 6), (0, 4, 8, 8, 8, 0), (0, 4, 8, 8, 2), (0, 4, 8, 5, 1, 9), (0, 4, 0, 6, 1), (0, 7, 1, 6, 0), (0, 7, 1, 6, 0), (0, 7, 1, 6, 0), (0, 1, 2, 5, 7, 5), (0, 4, 8, 7, 2), (0, 4, 6, 0, 1), (0, 4, 8, 8, 8, 0), (0, 2, 0, 1), (0, 9, 2, 9), (0, 4, 3, 3), (0, 4, 8, 2, 5, 3), (0, 1, 9, 8), (0, 4, 8, 5, 7, 0, 1), (0, 4, 7, 9), (0, 9, 0, 5), (0, 4, 8, 5, 1, 1, 4), (0, 4, 8, 5, 3, 0), (0, 4, 8, 5, 3, 5), (0, 6, 6, 5), (0, 0, 4, 1), (0, 4, 7, 8, 3), (0, 6, 0, 6), (0, 4, 3, 0), (0, 2, 0, 2), (0, 4, 8, 5, 7, 0, 1), (0, 5, 2), (0, 1, 6, 7, 6), (0, 4, 6, 5, 4), (0, 4, 8, 5, 0, 1), (0, 2, 4, 2), (0, 1, 2, 5, 3), (0, 4, 4, 4, 5), (0, 1, 5, 6, 0), (0, 4, 8, 5, 7, 0, 9, 0), (0, 6, 0, 9, 2), (0, 6, 7, 4, 2), (0, 4, 0, 6, 4, 4), (0, 9, 3, 2, 7, 0), (0, 9, 3, 2, 7, 0), (0, 1, 0, 2, 4), (0, 4, 3, 9), (0, 1, 1, 3), (0, 8, 0), (0, 6, 0, 1, 5), (0, 4, 7, 6, 2), (0, 4, 8, 5, 8, 9), (0, 4, 6, 5, 7), (0, 1, 6, 0), (0, 4, 8, 5, 7, 7, 1), (0, 8, 1, 9), (0, 8, 1, 4), (0, 6, 1, 7), (0, 2, 4, 7), (0, 2, 4, 7), (0, 2, 4, 8), (0, 2, 4, 8), (0, 2, 4, 8), (0, 2, 4, 7), (0, 8, 4, 1), (0, 8, 4, 4), (0, 2, 4, 7), (0, 2, 4, 8), (0, 8, 4, 4), (0, 8, 4, 4), (0, 2, 4, 7), (0, 8, 4, 1), (0, 8, 4, 9), (0, 8, 4, 9), (0, 8, 4, 9), (0, 4, 7, 8, 4), (0, 4, 8, 6, 5), (0, 6, 6, 7), (0, 4, 7, 3), (0, 9, 9, 7), (0, 1, 0, 2, 2), (0, 2, 0, 6), (0, 4, 8, 5, 3, 9), (0, 1, 3, 9, 9), (0, 1, 4, 3, 3), (0, 1, 4, 6, 2), (0, 2, 0, 9), (0, 4, 8, 0, 6), (0, 4, 8, 5, 7, 3, 3), (0, 6, 7, 4, 8), (0, 4, 3, 7), (0, 2, 1, 6), (0, 4, 8, 0, 9, 1, 1), (0, 6, 3, 9), (0, 8, 0), (0, 4, 8, 5, 1, 4), (0, 2, 5, 4), (0, 9, 7, 0), (0, 4, 8, 5, 5, 2), (0, 9, 0, 1, 3), (0, 2, 9, 9), (0, 2, 9, 3), (0, 9, 5, 6), (0, 1, 5, 5, 2), (0, 4, 8, 5, 3, 4, 1), (0, 4, 0, 1, 4), (0, 6, 2, 4), (0, 4, 8, 0, 6), (0, 9, 3, 4), (0, 4, 8, 0, 6), (0, 9, 3, 9), (0, 1, 2, 5, 2), (0, 4, 7, 4), (0, 4, 3, 4), (0, 7, 1, 6, 7), (0, 4, 8, 1, 2), (0, 6, 1, 1, 3), (0, 2, 1, 1), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 2, 2), (0, 1, 9, 4), (0, 2, 4, 2), (0, 0, 4, 4), (0, 4, 8, 5, 2, 0), (0, 4, 8, 1, 7), (0, 7, 1, 6, 0), (0, 4, 4, 1, 1, 0), (0, 4, 5, 1), (0, 4, 1, 9), (0, 1, 5, 9), (0, 6, 6, 5), (0, 5, 1, 3), (0, 9, 6, 7), (0, 1, 6, 7, 4), (0, 4, 8, 0, 3), (0, 2, 5, 4), (0, 8, 3, 1), (0, 4, 2, 2, 7), (0, 9, 2, 8), (0, 6, 7, 4, 4), (0, 1, 2, 3), (0, 2, 8, 6), (0, 4, 8, 5, 5, 1), (0, 1, 6, 7, 5, 0), (0, 8, 9, 3), (0, 4, 6, 3), (0, 1, 2, 8, 4), (0, 4, 8, 0, 1), (0, 4, 3, 4), (0, 4, 8, 5, 9, 8), (0, 4, 8, 0, 6), (0, 6, 7, 5, 2), (0, 4, 6, 1, 1, 7), (0, 4, 0, 1, 5, 1), (0, 6, 8, 0), (0, 4, 3, 9), (0, 4, 3, 9), (0, 4, 3, 9), (0, 5, 7, 0), (0, 4, 9, 5, 0, 0, 0), (0, 6, 8, 5), (0, 1, 9, 0, 0), (0, 9, 9, 6), (0, 4, 9, 5, 0, 0, 0), (0, 4, 9, 5, 0, 0, 0), (0, 4, 8, 5, 6, 1), (0, 9, 5, 6), (0, 4, 9, 5, 0, 0, 0), (0, 2, 4, 0), (0, 1, 3, 9, 6), (0, 2, 4, 1), (0, 4, 9, 5, 0, 0, 0), (0, 4, 8, 5, 1, 7), (0, 4, 8, 5, 7, 4, 8), (0, 4, 3, 6), (0, 4, 0, 6, 3, 5), (0, 4, 9, 5, 0, 0, 0), (0, 1, 6, 3, 8), (0, 6, 3, 9), (0, 4, 8, 6, 8), (0, 4, 8, 5, 7, 0, 8), (0, 4, 2, 6), (0, 4, 9, 5, 0, 0, 0), (0, 4, 9, 7), (0, 4, 9, 5, 0, 0, 0), (0, 4, 8, 5, 1, 6), (0, 4, 9, 5, 0, 0, 0), (0, 4, 7, 8, 4), (0, 1, 1, 1), (0, 4, 9, 5, 0, 0, 0), (0, 9, 6, 4), (0, 6, 6, 2), (0, 4, 8, 1, 3), (0, 4, 8, 1, 2), (0, 4, 3, 1, 3), (0, 1, 6, 6, 8), (0, 6, 9, 7, 2), (0, 4, 0, 0, 1), (0, 4, 6, 5, 1), (0, 5, 6), (0, 5, 6), (0, 6, 2, 9), (0, 6, 7, 0, 4), (0, 4, 5, 8, 2), (0, 4, 8, 1, 6), (0, 6, 9, 7, 2), (0, 4, 8, 5, 2, 6), (0, 7, 6), (0, 4, 8, 2, 9), (0, 4, 8, 8, 7), (0, 9, 9, 8), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 4, 8, 5, 7, 8, 7), (0, 7, 3, 9), (0, 0, 9, 5), (0, 6, 3, 2), (0, 4, 0, 1, 0), (0, 4, 5, 4, 8), (0, 1, 6, 7, 5, 6), (0, 7, 6), (0, 4, 2, 7, 1, 0), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 7, 6), (0, 4, 8, 5, 2, 9), (0, 6, 2, 1), (0, 4, 0, 1, 2), (0, 4, 5, 4, 1, 9), (0, 4, 7, 1), (0, 2, 0, 0), (0, 4, 5, 2, 1), (0, 2, 9, 1), (0, 4, 6, 5, 1), (0, 4, 6, 1, 7), (0, 4, 3, 9), (0, 5, 2), (0, 4, 8, 7, 3), (0, 4, 5, 9), (0, 4, 2, 7, 1, 1), (0, 9, 2, 8), (0, 1, 6, 1, 1), (0, 5, 2), (0, 4, 3, 2), (0, 3, 8), (0, 2, 2, 6), (0, 4, 3, 2), (0, 5, 2), (0, 1, 4, 4), (0, 4, 8, 0, 2), (0, 4, 7, 4), (0, 4, 8, 5, 7, 4, 4), (0, 4, 4, 9), (0, 6, 0, 0, 7), (0, 2, 3, 0), (0, 4, 0, 2, 9), (0, 4, 5, 7, 6), (0, 4, 8, 5, 8, 5, 6), (0, 9, 0, 5), (0, 6, 2, 9), (0, 4, 3, 1, 6), (0, 4, 8, 2, 0), (0, 4, 8, 2, 0), (0, 4, 8, 1, 2), (0, 4, 8, 5, 9, 0), (0, 4, 3, 5), (0, 1, 5, 0, 7), (0, 4, 5, 9), (0, 4, 0, 1, 6), (0, 4, 0, 7, 1), (0, 4, 8, 5, 0, 9), (0, 4, 8, 3, 4, 4), (0, 4, 8, 4, 4, 4), (0, 4, 8, 5, 2, 0), (0, 4, 8, 4, 9), (0, 4, 8, 2, 6), (0, 2, 0, 1), (0, 4, 3, 2), (0, 9, 3, 9), (0, 9, 0, 4), (0, 9, 2, 5), (0, 6, 7, 5, 5), (0, 4, 0, 1, 3), (0, 6, 7, 6), (0, 4, 4, 7), (0, 4, 0, 0, 5), (0, 4, 5, 7, 7), (0, 4, 8, 5, 2, 1), (0, 2, 1, 4), (0, 9, 8, 7), (0, 4, 8, 5, 7, 2, 0), (0, 1, 5, 8, 5), (0, 1, 9, 8), (0, 6, 7, 8), (0, 5, 0, 2, 1, 1), (0, 4, 4, 0, 1), (0, 4, 8, 5, 1, 4), (0, 1, 6, 8), (0, 4, 4, 0, 8), (0, 4, 3, 2), (0, 4, 8, 0, 9, 8), (0, 4, 8, 7, 4), (0, 4, 8, 7, 4), (0, 6, 7, 6), (0, 4, 8, 4, 0), (0, 4, 8, 5, 3, 4, 0), (0, 3, 8), (0, 2, 3, 2, 1), (0, 4, 6, 6, 4), (0, 4, 8, 4, 3), (0, 4, 6, 0, 2), (0, 4, 6, 5, 6), (0, 4, 8, 8, 4), (0, 6, 6, 7), (0, 4, 0, 1, 0), (0, 4, 6, 1, 1, 6), (0, 4, 8, 5, 7, 9, 2), (0, 1, 6, 7, 3), (0, 1, 6, 7, 8), (0, 4, 8, 0, 2), (0, 4, 8, 3, 4, 2), (0, 4, 5, 5), (0, 4, 2, 7, 7), (0, 4, 5, 5), (0, 4, 5, 5), (0, 4, 8, 5, 7, 2, 1), (0, 4, 2, 7, 7), (0, 4, 8, 5, 2, 4), (0, 4, 0, 0, 2), (0, 4, 0, 0, 2), (0, 4, 8, 2, 4, 2), (0, 4, 8, 2, 5, 5), (0, 4, 4, 1, 6), (0, 4, 8, 3, 1), (0, 4, 8, 2, 9), (0, 4, 8, 2, 0), (0, 4, 8, 2, 9), (0, 4, 8, 2, 0), (0, 4, 8, 5, 3, 4, 5), (0, 4, 8, 5, 8, 2), (0, 4, 7, 8, 5), (0, 4, 6, 5, 3), (0, 4, 0, 0, 2), (0, 6, 3, 2), (0, 4, 8, 5, 7, 0, 8), (0, 1, 5, 2, 4), (0, 4, 3, 4), (0, 4, 6, 3), (0, 4, 4, 4, 8), (0, 9, 7, 6), (0, 4, 8, 5, 7, 0, 2, 4), (0, 4, 8, 5, 7, 0, 6), (0, 6, 1, 3), (0, 0, 0, 4, 0), (0, 4, 8, 5, 7, 1, 5), (0, 6, 5, 9), (0, 0, 5), (0, 4, 1, 9), (0, 4, 3, 2), (0, 4, 8, 4, 8), (0, 4, 8, 5, 5, 0), (0, 1, 6, 7, 4), (0, 4, 7, 1), (0, 4, 8, 7, 5), (0, 4, 8, 0, 7), (0, 4, 8, 2, 1, 3), (0, 4, 4, 1, 0), (0, 4, 8, 5, 8, 3, 5), (0, 4, 2, 4, 2), (0, 9, 0, 0), (0, 4, 2, 4, 4), (0, 6, 9, 2), (0, 4, 2, 9), (0, 5, 1, 1), (0, 4, 2, 4, 4), (0, 4, 3, 2), (0, 4, 8, 5, 6, 0), (0, 1, 3, 7), (0, 1, 5, 2, 5), (0, 0, 7, 5), (0, 0, 1, 9, 6), (0, 2, 7, 5), (0, 4, 3, 2), (0, 2, 0, 0), (0, 4, 3, 2), (0, 4, 2, 0), (0, 4, 3, 1, 6), (0, 5, 0, 4), (0, 4, 8, 6, 4, 9), (0, 4, 8, 6, 0), (0, 4, 6, 5, 6), (0, 6, 3, 9), (0, 4, 8, 6, 4, 1), (0, 4, 5, 2, 9), (0, 7, 9, 2), (0, 4, 8, 1, 6), (0, 6, 6, 7), (0, 1, 5, 0, 2), (0, 4, 8, 5, 2, 4), (0, 4, 8, 4, 7), (0, 4, 6, 6, 3), (0, 4, 6, 8, 2), (0, 4, 8, 8, 8, 1), (0, 4, 8, 2, 5, 5), (0, 9, 3, 2, 0), (0, 2, 5, 5), (0, 6, 1, 0), (0, 6, 8, 3), (0, 4, 8, 3, 4, 6), (0, 4, 8, 0, 9, 1, 3), (0, 4, 2, 7, 5), (0, 4, 2, 5), (0, 4, 8, 4, 9), (0, 4, 2, 5), (0, 4, 3, 4), (0, 4, 8, 5, 1, 3), (0, 4, 8, 0, 5, 9), (0, 4, 3, 2), (0, 5, 1, 6), (0, 1, 7, 7, 1), (0, 1, 6, 9, 0), (0, 4, 8, 4, 4, 9), (0, 1, 6, 7, 7), (0, 5, 8, 0), (0, 5, 0, 2, 1, 8), (0, 6, 2, 1), (0, 6, 6, 5), (0, 0, 1, 8), (0, 4, 8, 4, 4, 4), (0, 2, 4, 5), (0, 9, 0, 0), (0, 9, 6, 4), (0, 4, 8, 5, 8, 1), (0, 5, 3, 4), (0, 4, 8, 5, 7, 1, 3), (0, 1, 9, 7), (0, 4, 8, 5, 7, 9, 0), (0, 6, 1, 2, 9), (0, 4, 8, 5, 2, 8), (0, 4, 8, 5, 3, 3), (0, 4, 8, 5, 8, 3, 5), (0, 4, 8, 5, 7, 0, 6), (0, 1, 4, 4), (0, 1, 3, 8), (0, 4, 5, 8, 8), (0, 4, 2, 1), (0, 4, 0, 0, 3), (0, 2, 7, 0, 2), (0, 4, 8, 8, 1), (0, 4, 8, 8, 8, 6), (0, 6, 7, 2), (0, 2, 3, 8, 0), (0, 4, 0, 1, 5, 3), (0, 5, 8, 6), (0, 2, 9, 5), (0, 4, 8, 5, 2, 0), (0, 4, 2, 1), (0, 6, 0, 3, 7), (0, 4, 8, 5, 2, 0), (0, 4, 8, 5, 2, 0), (0, 9, 0, 6), (0, 0, 7, 9), (0, 4, 1, 1, 0), (0, 6, 0, 9, 2), (0, 6, 3, 3, 1), (0, 4, 1, 1, 0), (0, 4, 1, 1, 0), (0, 4, 0, 6, 3, 5), (0, 4, 1, 1, 0), (0, 1, 9, 4), (0, 4, 8, 5, 7, 7, 7), (0, 4, 1, 1, 0), (0, 4, 8, 5, 7, 4, 0), (0, 4, 8, 5, 2, 9), (0, 6, 7, 4, 6), (0, 4, 8, 5, 2, 0), (0, 1, 6, 7, 5, 8), (0, 4, 1, 1, 0), (0, 4, 1, 1, 4), (0, 4, 1, 1, 0), (0, 4, 8, 7, 2), (0, 5, 8, 6), (0, 4, 1, 1, 0), (0, 4, 1, 8), (0, 4, 1, 8), (0, 4, 1, 1, 0), (0, 8, 4, 8), (0, 4, 1, 2), (0, 4, 1, 2), (0, 4, 1, 2), (0, 4, 1, 2), (0, 4, 1, 2), (0, 4, 1, 2), (0, 4, 1, 1, 0), (0, 4, 2, 5), (0, 6, 6, 9), (0, 1, 2, 4, 7), (0, 4, 3, 4), (0, 6, 7, 5, 0), (0, 4, 8, 0, 9, 1, 1), (0, 4, 8, 7, 5), (0, 5, 7, 6), (0, 5, 3, 5), (0, 6, 7, 2), (0, 4, 8, 0, 6), (0, 4, 2, 1), (0, 0, 1, 6), (0, 0, 4, 1), (0, 9, 3, 2, 7, 6), (0, 4, 6, 2, 5), (0, 0, 6, 2), (0, 6, 6, 9), (0, 6, 5, 1), (0, 1, 4, 4), (0, 4, 4, 1, 0), (0, 4, 7, 3), (0, 9, 8, 0), (0, 4, 8, 8, 8, 1), (0, 6, 1, 5), (0, 1, 4, 6, 0), (0, 4, 2, 4, 6), (0, 6, 6, 5), (0, 4, 2, 4, 3, 0), (0, 4, 2, 5), (0, 4, 2, 0), (0, 4, 2, 6), (0, 6, 2, 8), (0, 4, 1, 1, 9), (0, 1, 6, 7, 7), (0, 6, 7, 8), (0, 9, 2, 2), (0, 9, 6, 2), (0, 4, 2, 3), (0, 6, 3, 5), (0, 9, 6, 2), (0, 1, 7, 1), (0, 4, 7, 8, 6), (0, 4, 2, 8), (0, 6, 8, 5), (0, 4, 2, 4, 6), (0, 1, 9, 5), (0, 5, 3, 5), (0, 4, 7, 5), (0, 9, 8, 4), (0, 9, 0, 5), (0, 0, 6, 2), (0, 4, 2, 2, 8), (0, 4, 2, 5), (0, 1, 2, 5, 7, 0), (0, 4, 8, 5, 7, 1, 9), (0, 6, 6, 7), (0, 9, 8, 1), (0, 6, 6, 2), (0, 4, 8, 5, 4, 7), (0, 6, 7, 7), (0, 6, 1, 2, 5), (0, 4, 8, 6, 4, 5), (0, 4, 8, 6, 0), (0, 4, 8, 5, 1, 8), (0, 9, 7, 1), (0, 5, 4, 0), (0, 4, 8, 5, 3, 3), (0, 9, 3, 4), (0, 6, 1, 9), (0, 4, 8, 5, 7, 6, 1), (0, 6, 3, 9), (0, 9, 8, 2), (0, 4, 8, 1, 2), (0, 1, 6, 1, 6), (0, 5, 7, 4), (0, 4, 0, 5, 2), (0, 1, 2, 4, 1), (0, 9, 6, 2), (0, 6, 7, 0, 2), (0, 9, 6, 2), (0, 9, 9, 7), (0, 4, 8, 6, 9), (0, 6, 6, 7), (0, 6, 5, 8, 6), (0, 4, 2, 6), (0, 4, 8, 3, 7), (0, 4, 8, 5, 8, 4, 2), (0, 2, 5, 8), (0, 8, 3, 5), (0, 9, 6, 2), (0, 4, 6, 1, 1, 1), (0, 4, 5, 8, 1), (0, 2, 5, 8), (0, 8, 3, 5), (0, 4, 8, 5, 7, 6, 6), (0, 9, 3, 2, 7, 0), (0, 9, 6, 2), (0, 4, 3, 9), (0, 4, 8, 1, 1), (0, 4, 5, 5), (0, 6, 7, 0, 7), (0, 1, 6, 7, 0, 6), (0, 9, 8, 4), (0, 8, 3, 5), (0, 6, 0, 5, 2), (0, 4, 0, 3, 8), (0, 4, 8, 5, 7, 6, 5), (0, 1, 0, 5), (0, 6, 3, 6), (0, 8, 0), (0, 6, 6, 7), (0, 2, 3, 8, 9), (0, 2, 3, 2, 1), (0, 4, 8, 5, 7, 7, 2), (0, 5, 4, 4), (0, 4, 2, 0), (0, 5, 4, 4), (0, 1, 6, 7, 3), (0, 4, 2, 4, 3, 5), (0, 4, 8, 5, 2, 0), (0, 9, 9, 6), (0, 4, 3, 6), (0, 9, 6, 9), (0, 4, 8, 5, 2, 6), (0, 6, 6, 3), (0, 9, 4, 4), (0, 4, 8, 1, 6), (0, 2, 3, 6, 0), (0, 4, 8, 5, 7, 1, 7), (0, 9, 7, 2), (0, 4, 8, 5, 1, 6), (0, 5, 7, 0), (0, 5, 8, 1), (0, 6, 0, 8, 7), (0, 9, 6, 4), (0, 9, 2, 7), (0, 8, 4, 5), (0, 4, 8, 2, 2), (0, 2, 0, 7), (0, 9, 8, 1), (0, 4, 3, 5), (0, 4, 3, 5), (0, 6, 5, 5), (0, 4, 8, 5, 3, 3), (0, 6, 1, 2, 8), (0, 1, 2, 4, 3), (0, 9, 3, 2, 0), (0, 9, 3, 9), (0, 4, 8, 6, 7), (0, 4, 7, 2), (0, 4, 8, 5, 7, 4, 8), (0, 4, 8, 0, 8, 2), (0, 4, 8, 5, 3, 4, 1), (0, 5, 0, 2, 1, 1), (0, 5, 1, 1), (0, 6, 6, 1), (0, 4, 8, 8, 8, 1), (0, 4, 8, 2, 7), (0, 6, 1, 2, 2), (0, 4, 8, 5, 6, 4), (0, 6, 0, 5, 3), (0, 1, 5, 3), (0, 9, 5, 3), (0, 6, 7, 5, 0), (0, 7, 9, 1), (0, 6, 7, 4, 7), (0, 4, 8, 4, 4, 4), (0, 4, 8, 5, 3, 1), (0, 4, 8, 5, 7, 7, 7), (0, 6, 6, 1), (0, 9, 6, 4), (0, 9, 2, 2), (0, 6, 7, 4, 7), (0, 6, 7, 2), (0, 6, 6, 2), (0, 9, 6, 8), (0, 6, 1, 1, 5), (0, 2, 3, 5), (0, 2, 3, 8, 0), (0, 6, 5, 7), (0, 9, 6, 1), (0, 9, 6, 8), (0, 9, 5, 1), (0, 4, 8, 5, 1, 8), (0, 6, 7, 3, 9), (0, 2, 3, 5), (0, 6, 6, 1), (0, 4, 1, 1, 0), (0, 2, 8, 6), (0, 4, 1, 1, 0), (0, 4, 1, 1, 0), (0, 4, 1, 1, 0), (0, 4, 1, 1, 0), (0, 4, 1, 1, 6), (0, 4, 1, 1, 6), (0, 2, 3, 5), (0, 2, 8, 6), (0, 6, 7, 2), (0, 6, 6, 2), (0, 2, 0, 5), (0, 4, 0, 1, 5, 1), (0, 9, 2, 8), (0, 1, 0, 5), (0, 1, 6, 3, 4), (0, 2, 0, 6), (0, 2, 7, 0, 6), (0, 8, 0), (0, 6, 1, 2, 2), (0, 6, 7, 4, 1), (0, 9, 3, 1), (0, 9, 3, 9), (0, 9, 2, 8), (0, 9, 6, 8), (0, 6, 8, 4), (0, 1, 6, 7, 0, 6), (0, 9, 3, 4), (0, 4, 0, 1, 0), (0, 4, 6, 1, 1, 7), (0, 6, 6, 5), (0, 2, 0, 5), (0, 2, 3, 8, 8), (0, 6, 1, 4), (0, 6, 7, 5, 0), (0, 0, 0, 4, 0), (0, 2, 0, 6), (0, 2, 3, 5), (0, 2, 0, 6), (0, 9, 5, 2), (0, 9, 6, 5), (0, 4, 8, 5, 3, 4, 1), (0, 6, 1, 2, 2), (0, 5, 7, 0), (0, 4, 5, 4, 7), (0, 2, 5, 1), (0, 2, 8, 4), (0, 1, 6, 3, 0), (0, 1, 9, 9), (0, 9, 6, 5), (0, 4, 8, 5, 7, 0, 9, 4), (0, 4, 8, 8, 2), (0, 4, 8, 8, 2), (0, 2, 5, 1), (0, 2, 3, 5), (0, 2, 8, 9), (0, 4, 0, 1, 2), (0, 1, 6, 5, 2), (0, 6, 1, 2, 8), (0, 6, 6, 4, 3), (0, 4, 8, 5, 7, 4, 5), (0, 4, 0, 1, 5, 9), (0, 4, 0, 1, 1), (0, 6, 6, 4, 3), (0, 9, 6, 1), (0, 4, 8, 0, 0, 4), (0, 1, 6, 0), (0, 6, 7, 7), (0, 4, 0, 6, 4, 1), (0, 6, 6, 2), (0, 9, 0, 1, 4), (0, 9, 2, 2), (0, 9, 7, 7), (0, 3, 4), (0, 4, 8, 5, 7, 4, 0), (0, 6, 6, 2), (0, 4, 1, 2), (0, 2, 9, 2), (0, 6, 6, 3), (0, 6, 6, 4, 8), (0, 9, 6, 9), (0, 4, 6, 5, 0), (0, 3, 8), (0, 6, 2, 5), (0, 6, 6, 3), (0, 9, 7, 0), (0, 4, 6, 7), (0, 6, 0, 6), (0, 1, 2, 8, 2), (0, 4, 0, 3, 0), (0, 6, 0, 3, 8), (0, 9, 9, 9), (0, 4, 8, 8, 8, 6), (0, 4, 6, 1, 1, 7), (0, 6, 6, 2), (0, 1, 5, 8, 8), (0, 1, 6, 3, 5), (0, 1, 6, 5, 7), (0, 4, 1, 7), (0, 4, 1, 1, 5), (0, 4, 8, 5, 4, 8, 2), (0, 2, 8, 4), (0, 1, 5, 4), (0, 5, 1, 1), (0, 6, 0, 5, 6), (0, 0, 5), (0, 4, 0, 1, 5, 2), (0, 6, 7, 8), (0, 6, 6, 1), (0, 6, 2, 3), (0, 4, 8, 5, 2, 4), (0, 4, 2, 5), (0, 4, 8, 5, 2, 7), (0, 1, 6, 7, 5, 3), (0, 9, 2, 3), (0, 9, 0, 5), (0, 4, 4, 0, 1), (0, 9, 3, 2, 3), (0, 9, 5, 1), (0, 1, 5, 8, 7), (0, 4, 6, 5, 0), (0, 9, 3, 1), (0, 4, 8, 5, 4, 8, 1), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 1, 0, 1), (0, 9, 6, 8), (0, 4, 8, 6, 5), (0, 5, 0, 2, 1, 0), (0, 4, 8, 5, 3, 4, 2), (0, 4, 8, 5, 7, 1, 1), (0, 6, 2, 8), (0, 6, 5, 2), (0, 1, 3, 9, 0), (0, 4, 8, 5, 7, 7, 9), (0, 4, 8, 0, 2), (0, 6, 7, 1), (0, 4, 2, 2, 4), (0, 4, 0, 5, 8), (0, 4, 8, 5, 9, 7), (0, 4, 8, 1, 2), (0, 1, 3, 1), (0, 1, 5, 2, 6), (0, 1, 3, 9, 0), (0, 4, 6, 3), (0, 9, 5, 6), (0, 9, 6, 8), (0, 1, 6, 8), (0, 4, 8, 5, 8, 4, 2), (0, 9, 6, 1), (0, 9, 2, 9), (0, 4, 8, 5, 3, 3), (0, 4, 4, 4, 2), (0, 6, 9, 3), (0, 4, 8, 5, 7, 6, 1), (0, 4, 8, 5, 0, 3), (0, 0, 6, 3), (0, 6, 5, 4), (0, 4, 8, 2, 7), (0, 4, 5, 6, 0), (0, 1, 2, 5, 3), (0, 4, 8, 0, 0, 4), (0, 6, 0, 3, 3), (0, 4, 4, 7), (0, 6, 1, 6), (0, 6, 5, 7), (0, 4, 7, 3), (0, 0, 8, 9), (0, 4, 8, 5, 7, 6, 9), (0, 4, 0, 1, 5, 2), (0, 4, 8, 6, 4, 2), (0, 4, 2, 6), (0, 4, 0, 6, 3, 4), (0, 4, 8, 5, 7, 5, 6), (0, 4, 8, 8, 0, 5), (0, 0, 4, 3), (0, 6, 5, 4), (0, 6, 2, 5), (0, 9, 4, 5), (0, 7, 1, 6, 3), (0, 4, 6, 1, 1, 1), (0, 1, 2, 5, 7, 5), (0, 4, 8, 5, 7, 7, 6), (0, 4, 8, 5, 8, 6, 1), (0, 6, 3, 7), (0, 6, 7, 1), (0, 4, 3, 9), (0, 1, 9, 0, 8), (0, 4, 9, 2, 2), (0, 4, 4, 1, 1, 2), (0, 6, 2, 6), (0, 4, 8, 5, 8, 5, 5), (0, 6, 5, 3), (0, 1, 9, 1), (0, 5, 3, 1), (0, 9, 8, 1), (0, 4, 8, 5, 1, 4), (0, 4, 0, 1, 5, 6), (0, 9, 0, 4), (0, 1, 6, 7, 3), (0, 5, 5), (0, 1, 6, 4, 0), (0, 4, 2, 4, 3, 9), (0, 6, 0, 5, 4), (0, 6, 8, 2), (0, 0, 9, 2, 1), (0, 4, 4, 4, 8), (0, 1, 5, 7), (0, 4, 8, 5, 5, 9), (0, 4, 0, 8, 7), (0, 4, 4, 7), (0, 4, 8, 4, 8), (0, 4, 0, 2, 1), (0, 4, 8, 5, 7, 9, 1), (0, 6, 2, 7), (0, 1, 2, 5, 8), (0, 4, 8, 5, 7, 4, 5), (0, 9, 5, 8), (0, 4, 8, 5, 1, 2), (0, 4, 5, 3), (0, 4, 8, 5, 7, 2, 9), (0, 5, 1, 0), (0, 4, 8, 8, 7), (0, 4, 8, 5, 7, 8, 2), (0, 6, 0, 7, 3), (0, 4, 3, 3), (0, 4, 0, 4, 3), (0, 6, 1, 8), (0, 4, 8, 5, 7, 5, 5), (0, 4, 8, 6, 2), (0, 4, 9, 5, 1), (0, 6, 1, 2, 0), (0, 6, 7, 0, 1), (0, 8, 0), (0, 1, 3, 4), (0, 4, 5, 4, 1, 2), (0, 1, 2, 5, 1), (0, 4, 5, 4, 1, 2), (0, 4, 0, 7, 5), (0, 4, 8, 8, 0, 3), (0, 1, 6, 9, 6), (0, 4, 0, 2, 1), (0, 6, 7, 0, 7), (0, 4, 8, 5, 1, 2), (0, 0, 1, 6), (0, 9, 0, 9), (0, 6, 1, 8), (0, 9, 9, 7), (0, 9, 3, 2, 0), (0, 6, 6, 6), (0, 4, 0, 7, 9), (0, 4, 0, 2, 1), (0, 9, 9, 2), (0, 9, 2, 7), (0, 9, 8, 2), (0, 1, 5, 1), (0, 0, 4, 4), (0, 4, 8, 5, 9, 6), (0, 4, 0, 1, 5, 2), (0, 6, 7, 6), (0, 4, 8, 5, 9, 3), (0, 4, 0, 2, 1), (0, 4, 4, 4, 1), (0, 4, 4, 4, 1), (0, 4, 8, 5, 7, 7, 3), (0, 4, 8, 7, 2), (0, 4, 8, 5, 1, 8), (0, 6, 8, 6), (0, 0, 9, 1), (0, 3, 0, 4), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 3), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 3, 0, 0), (0, 0, 9, 6), (0, 0, 6, 4), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 4, 4, 5), (0, 9, 9, 7), (0, 4, 8, 5, 0, 8), (0, 9, 8, 2), (0, 4, 8, 2, 5, 5), (0, 4, 8, 5, 7, 3, 2), (0, 2, 0, 0), (0, 4, 8, 5, 8, 9), (0, 4, 2, 1), (0, 6, 0, 9, 5), (0, 6, 7, 3, 9), (0, 6, 0, 8, 5), (0, 1, 2, 9), (0, 4, 8, 5, 7, 0, 7), (0, 1, 6, 3, 2), (0, 9, 8, 6), (0, 9, 0, 9), (0, 1, 9, 0, 0), (0, 9, 8, 9), (0, 4, 8, 5, 2, 0), (0, 4, 8, 5, 0, 2), (0, 6, 6, 7), (0, 4, 0, 6, 4, 1), (0, 4, 8, 4, 1), (0, 4, 8, 5, 7, 6, 1), (0, 4, 8, 5, 7, 8, 3), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 4, 0, 1, 7), (0, 1, 6, 7, 0, 5), (0, 6, 6, 1), (0, 9, 0, 5), (0, 4, 8, 5, 7, 0, 5), (0, 0, 9, 2, 1), (0, 9, 0, 0), (0, 9, 0, 4), (0, 1, 3, 4), (0, 9, 9, 7), (0, 4, 8, 5, 2, 4), (0, 1, 6, 3, 1, 3), (0, 6, 6, 9), (0, 4, 8, 2, 4, 1), (0, 4, 8, 9, 4, 1), (0, 4, 5, 8, 6), (0, 1, 3, 4), (0, 4, 8, 5, 2, 1), (0, 4, 8, 5, 2, 5), (0, 4, 8, 5, 7, 8, 2), (0, 4, 8, 5, 4, 7), (0, 2, 9, 3), (0, 1, 5, 8, 8), (0, 1, 6, 5, 0), (0, 4, 6, 9), (0, 4, 8, 5, 2, 5), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 2), (0, 3, 0, 2), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 3), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 2), (0, 3, 0, 2), (0, 3, 0, 2), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 3, 0, 1, 0, 0, 0, 0), (0, 9, 8, 3), (0, 4, 8, 2, 6), (0, 4, 8, 2, 6), (0, 4, 8, 5, 8, 5, 2), (0, 4, 8, 5, 7, 9, 0), (0, 4, 2, 4, 1), (0, 4, 8, 2, 6), (0, 4, 8, 2, 2), (0, 4, 8, 8, 0, 2), (0, 4, 8, 2, 4, 2), (0, 1, 3, 8), (0, 9, 9, 0), (0, 4, 8, 7, 9), (0, 0, 1, 9, 5), (0, 1, 5, 6, 0), (0, 6, 7, 3, 4), (0, 4, 8, 5, 7, 2, 7), (0, 9, 0, 1, 6), (0, 4, 5, 8, 1), (0, 4, 5, 8, 1), (0, 4, 2, 7, 9), (0, 6, 0, 0, 5), (0, 4, 8, 5, 7, 0, 2, 0), (0, 4, 8, 5, 2, 8), (0, 4, 8, 5, 7, 0, 6), (0, 4, 8, 5, 7, 8, 3), (0, 2, 3, 6, 0), (0, 4, 8, 5, 7, 0, 3, 2), (0, 4, 8, 5, 4, 4), (0, 6, 1, 1, 9), (0, 1, 6, 9, 1), (0, 6, 3, 6), (0, 4, 0, 7, 3), (0, 1, 6, 3, 5), (0, 2, 8, 0), (0, 4, 4, 9), (0, 1, 0, 6), (0, 9, 8, 2), (0, 0, 6, 1), (0, 4, 8, 5, 2, 3), (0, 6, 7, 2), (0, 6, 1, 2, 2), (0, 4, 8, 5, 7, 8, 7), (0, 1, 6, 4, 1), (0, 4, 8, 5, 7, 0, 0, 2), (0, 6, 0, 2), (0, 4, 6, 1, 1, 1), (0, 6, 1, 7), (0, 5, 1, 0), (0, 6, 5, 4), (0, 6, 5, 8, 4), (0, 4, 8, 0, 8, 2), (0, 4, 8, 0, 9, 0), (0, 9, 9, 5), (0, 4, 7, 7), (0, 6, 1, 1, 0), (0, 6, 9, 5), (0, 0, 1, 5), (0, 4, 8, 4, 4, 4), (0, 4, 8, 5, 7, 0, 0, 0), (0, 9, 2, 9), (0, 9, 3, 8), (0, 1, 5, 6, 8), (0, 4, 8, 5, 7, 0, 2, 0), (0, 9, 9, 3), (0, 4, 0, 1, 5, 8), (0, 9, 9, 3), (0, 6, 2, 5), (0, 6, 8, 6), (0, 0, 7, 1), (0, 4, 8, 5, 8, 6, 0), (0, 4, 4, 0, 1), (0, 4, 0, 3, 0), (0, 7, 3, 1), (0, 2, 3, 1), (0, 7, 3, 1), (0, 7, 3, 1), (0, 7, 9, 4), (0, 4, 8, 5, 8, 5, 4), (0, 9, 9, 5), (0, 2, 7, 7), (0, 6, 7, 6), (0, 6, 7, 1), (0, 4, 6, 3), (0, 6, 1, 2, 2), (0, 7, 3, 1), (0, 4, 7, 6, 0), (0, 5, 1, 1), (0, 6, 8, 5), (0, 4, 6, 1, 4), (0, 4, 8, 0, 9, 8), (0, 4, 6, 1, 4), (0, 1, 5, 4), (0, 2, 0, 6), (0, 4, 8, 5, 9, 1), (0, 6, 8, 3), (0, 6, 7, 1), (0, 2, 6, 4, 1), (0, 4, 8, 5, 7, 8, 2), (0, 9, 8, 7), (0, 4, 6, 5, 1), (0, 4, 8, 9, 2), (0, 6, 0, 9, 8), (0, 4, 8, 5, 7, 9, 1), (0, 4, 8, 5, 7, 8, 0), (0, 4, 8, 5, 8, 8), (0, 4, 8, 5, 8, 2), (0, 4, 7, 2), (0, 1, 2, 5, 6), (0, 4, 7, 1), (0, 1, 7, 3), (0, 9, 6, 1), (0, 0, 7, 9), (0, 4, 8, 5, 2, 8), (0, 9, 3, 2, 3), (0, 6, 6, 5), (0, 4, 8, 3, 4, 6), (0, 4, 8, 5, 1, 9), (0, 4, 5, 4, 1, 1), (0, 4, 8, 3, 0), (0, 4, 8, 5, 1, 2), (0, 4, 8, 3, 0), (0, 2, 1, 1), (0, 4, 9, 5, 0, 0, 8), (0, 4, 3, 5), (0, 6, 7, 9), (0, 4, 8, 5, 5, 1), (0, 4, 8, 5, 7, 9, 0), (0, 4, 0, 3, 3), (0, 4, 8, 0, 1), (0, 1, 6, 5, 9), (0, 1, 8, 1, 1), (0, 7, 1, 0, 1), (0, 7, 5, 6), (0, 7, 5, 6), (0, 4, 8, 5, 7, 0, 3, 7), (0, 6, 8, 3), (0, 6, 6, 8), (0, 1, 3, 2), (0, 6, 0, 1, 5), (0, 4, 0, 5, 4), (0, 6, 0, 3, 9), (0, 4, 6, 1, 1, 1), (0, 4, 8, 5, 3, 6), (0, 4, 8, 5, 1, 1, 4), (0, 4, 8, 5, 7, 0, 3, 4), (0, 9, 0, 9), (0, 4, 8, 0, 8, 0), (0, 4, 4, 7), (0, 4, 8, 5, 7, 0, 9, 3), (0, 4, 8, 8, 8, 3), (0, 4, 8, 0, 0, 5), (0, 4, 5, 4, 1, 4, 2), (0, 4, 8, 9, 3), (0, 4, 4, 1, 1, 0), (0, 4, 6, 2, 1), (0, 9, 1, 3), (0, 4, 8, 5, 7, 7, 3), (0, 4, 8, 5, 7, 0, 6), (0, 4, 8, 3, 0), (0, 4, 5, 7, 8), (0, 4, 6, 5, 0), (0, 4, 6, 6, 7), (0, 4, 8, 5, 8, 2), (0, 6, 9, 9), (0, 2, 3, 0), (0, 2, 3, 2, 1), (0, 4, 8, 5, 7, 7, 7), (0, 6, 9, 2), (0, 4, 8, 5, 9, 5), (0, 6, 6, 1), (0, 4, 8, 9, 2), (0, 6, 1, 1, 0), (0, 6, 8, 7), (0, 5, 1, 3), (0, 9, 3, 2, 3), (0, 2, 6, 3), (0, 2, 8, 4), (0, 9, 3, 2, 7, 3), (0, 0, 1, 8), (0, 1, 3, 9, 3), (0, 0, 6, 1), (0, 4, 0, 9), (0, 2, 3, 0), (0, 1, 6, 9, 5), (0, 6, 8, 6), (0, 6, 6, 2), (0, 6, 2, 6), (0, 4, 8, 5, 5, 1), (0, 4, 8, 0, 8, 3), (0, 6, 6, 1), (0, 9, 7, 5), (0, 9, 2, 2), (0, 6, 7, 3, 8), (0, 6, 6, 4, 3), (0, 9, 0, 6), (0, 4, 7, 8, 4), (0, 4, 8, 4, 5), (0, 4, 6, 2, 8), (0, 6, 1, 8), (0, 1, 4, 8), (0, 1, 4, 5), (0, 4, 4, 3), (0, 6, 1, 2, 8), (0, 2, 3, 0), (0, 4, 8, 5, 7, 7, 8), (0, 4, 0, 3, 7), (0, 6, 3, 3, 4), (0, 6, 3, 6), (0, 9, 2, 8), (0, 4, 4, 1, 5), (0, 4, 8, 1, 2), (0, 4, 6, 2, 1), (0, 4, 5, 4, 1, 9), (0, 1, 2, 2, 8), (0, 2, 3, 6, 5), (0, 4, 8, 1, 2), (0, 1, 3, 2), (0, 6, 5, 2), (0, 1, 6, 7, 6), (0, 4, 8, 5, 4, 8, 5), (0, 9, 5, 3), (0, 2, 6, 4, 3), (0, 4, 8, 4, 5), (0, 4, 8, 5, 1, 6), (0, 4, 0, 4, 4), (0, 9, 3, 2, 6), (0, 6, 0, 8, 4), (0, 1, 6, 1, 4), (0, 1, 2, 7, 1), (0, 4, 8, 5, 1, 6), (0, 4, 8, 1, 6), (0, 0, 0, 4, 0), (0, 0, 0, 4, 0), (0, 4, 9, 5, 0, 4), (0, 4, 8, 5, 3, 6), (0, 4, 0, 1, 6), (0, 4, 8, 5, 8, 7), (0, 1, 9, 2), (0, 1, 2, 4, 6), (0, 0, 9, 6), (0, 4, 0, 6, 4, 1), (0, 2, 3, 3), (0, 6, 2, 4), (0, 2, 1, 1), (0, 2, 0, 2), (0, 2, 1, 6), (0, 6, 7, 3, 6), (0, 4, 8, 5, 7, 9, 8), (0, 6, 1, 2, 5), (0, 2, 3, 6, 0), (0, 9, 6, 1), (0, 4, 8, 5, 2, 8), (0, 4, 8, 5, 9, 5), (0, 4, 0, 1, 0), (0, 0, 9, 6), (0, 4, 9, 5, 0, 6), (0, 4, 2, 1), (0, 4, 8, 5, 2, 8), (0, 9, 3, 3), (0, 4, 6, 1, 1, 0), (0, 7, 3, 2), (0, 7, 1, 0, 3), (0, 1, 2, 5, 7, 5), (0, 4, 8, 5, 7, 7, 5), (0, 4, 8, 0, 6), (0, 4, 4, 1, 1, 0), (0, 4, 4, 1, 1, 0), (0, 4, 8, 9, 0), (0, 6, 0, 8, 9), (0, 6, 2, 5), (0, 4, 8, 5, 4, 3), (0, 6, 6, 3), (0, 4, 8, 5, 2, 7), (0, 9, 8, 4), (0, 4, 8, 0, 0, 4), (0, 7, 1, 2), (0, 9, 0, 1, 1), (0, 2, 3, 1), (0, 6, 7, 6), (0, 4, 8, 0, 6), (0, 4, 8, 5, 8, 4, 8), (0, 4, 8, 0, 0, 5), (0, 4, 8, 5, 8, 1), (0, 6, 6, 2), (0, 4, 8, 0, 5, 7), (0, 6, 7, 4, 2), (0, 4, 8, 2, 4, 6), (0, 2, 6, 0), (0, 4, 8, 5, 7, 0, 0, 9), (0, 9, 6, 4), (0, 1, 2, 2, 2), (0, 6, 0, 1, 0), (0, 2, 3, 6, 8), (0, 0, 0, 5, 0), (0, 4, 6, 2, 5), (0, 0, 6, 7), (0, 6, 0, 0, 0), (0, 6, 3, 5), (0, 6, 8, 5), (0, 6, 6, 1), (0, 4, 5, 4, 1, 4, 2), (0, 6, 5, 5), (0, 5, 3, 2), (0, 6, 7, 4, 1), (0, 6, 7, 2), (0, 1, 5, 5, 0), (0, 6, 7, 3, 9), (0, 9, 0, 5), (0, 6, 2, 9), (0, 4, 8, 5, 7, 0, 8), (0, 2, 7, 6), (0, 7, 1, 1), (0, 2, 1, 4), (0, 6, 2, 5), (0, 8, 0), (0, 1, 9, 0, 0), (0, 9, 6, 8), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 8, 1), (0, 5, 0, 2, 1, 1), (0, 6, 9, 4), (0, 4, 8, 6, 4, 1), (0, 9, 8, 7), (0, 9, 3, 8), (0, 9, 7, 7), (0, 4, 9, 0), (0, 9, 3, 9), (0, 2, 3, 7, 2), (0, 2, 3, 7, 2), (0, 4, 8, 5, 7, 5, 3), (0, 1, 1, 8), (0, 4, 7, 1), (0, 6, 3, 5), (0, 4, 0, 6, 4, 9), (0, 4, 6, 2, 3), (0, 9, 0, 1, 3), (0, 6, 0, 3, 0), (0, 4, 7, 6, 1), (0, 6, 7, 1), (0, 6, 7, 1), (0, 4, 8, 5, 8, 1), (0, 4, 8, 3, 4, 2), (0, 0, 1, 9, 1), (0, 6, 7, 1), (0, 6, 8, 5), (0, 2, 1, 4), (0, 9, 0, 1, 3), (0, 4, 1, 3), (0, 6, 7, 1), (0, 6, 5, 2), (0, 2, 7, 0, 0), (0, 2, 7, 3), (0, 9, 8, 7), (0, 4, 8, 5, 7, 9, 1), (0, 4, 8, 5, 7, 9, 1), (0, 2, 0, 8), (0, 6, 7, 6), (0, 9, 3, 2, 7, 0), (0, 9, 8, 4), (0, 4, 8, 5, 7, 9, 1), (0, 0, 9, 1), (0, 5, 5), (0, 4, 8, 2, 4, 2), (0, 6, 2, 5), (0, 6, 9, 1), (0, 6, 9, 3), (0, 2, 1, 4), (0, 2, 1, 4), (0, 4, 8, 8, 6), (0, 4, 8, 5, 7, 6, 2), (0, 6, 8, 0), (0, 6, 5, 2), (0, 4, 8, 5, 4, 3), (0, 1, 6, 9, 0), (0, 4, 8, 5, 7, 0, 6), (0, 2, 3, 7, 8), (0, 1, 5, 5, 3), (0, 0, 0, 5, 5), (0, 9, 9, 9), (0, 0, 0, 5, 3), (0, 4, 0, 0, 6), (0, 2, 7, 2), (0, 4, 7, 8, 4), (0, 4, 0, 3, 0), (0, 4, 6, 8, 4), (0, 6, 0, 3, 2), (0, 1, 2, 8, 2), (0, 6, 9, 4), (0, 4, 8, 5, 8, 9), (0, 4, 0, 1, 9), (0, 6, 7, 6), (0, 4, 0, 7, 0), (0, 6, 6, 1), (0, 9, 9, 2), (0, 1, 5, 2, 3), (0, 4, 8, 5, 3, 2), (0, 4, 9, 5, 2), (0, 2, 9, 2), (0, 4, 8, 5, 7, 0, 0, 4), (0, 4, 0, 1, 2), (0, 4, 8, 5, 1, 1, 1), (0, 4, 8, 5, 7, 0, 8), (0, 6, 7, 1), (0, 4, 8, 0, 5, 6), (0, 9, 3, 9), (0, 1, 7, 7, 1), (0, 5, 4, 3), (0, 6, 1, 2, 8), (0, 2, 3, 7, 2), (0, 4, 8, 8, 1), (0, 4, 0, 1, 5, 6), (0, 2, 3, 9), (0, 4, 8, 2, 4, 5), (0, 1, 2, 5, 1), (0, 6, 7, 1), (0, 4, 7, 8, 2), (0, 4, 0, 1, 3), (0, 4, 7, 6, 2), (0, 9, 0, 3), (0, 4, 8, 5, 2, 6), (0, 9, 0, 4), (0, 6, 1, 8), (0, 1, 2, 5, 8), (0, 1, 2, 3), (0, 0, 0, 1), (0, 1, 6, 4, 8), (0, 4, 8, 6, 5), (0, 4, 8, 5, 8, 1), (0, 4, 8, 5, 8, 1), (0, 4, 8, 5, 8, 7), (0, 6, 7, 5, 4), (0, 1, 9, 1), (0, 1, 5, 0, 2), (0, 6, 0, 1, 8), (0, 4, 8, 5, 7, 0, 2, 9), (0, 7, 1, 0, 1), (0, 4, 8, 5, 3, 6), (0, 4, 2, 1), (0, 6, 0, 1, 4), (0, 5, 4, 6), (0, 1, 6, 9, 4), (0, 4, 8, 8, 8, 2), (0, 4, 7, 6, 2), (0, 4, 0, 7, 2), (0, 4, 8, 5, 8, 8), (0, 7, 9, 5), (0, 4, 8, 5, 0, 1), (0, 9, 0, 4), (0, 9, 8, 1), (0, 1, 2, 5, 7, 6), (0, 1, 6, 8), (0, 4, 8, 3, 3), (0, 2, 2, 8), (0, 2, 2, 8), (0, 6, 6, 0), (0, 6, 1, 2, 5), (0, 4, 8, 5, 7, 2, 8), (0, 4, 0, 1, 6), (0, 6, 0, 5, 2), (0, 4, 6, 6, 2), (0, 1, 2, 8, 2), (0, 4, 8, 5, 6, 6), (0, 6, 7, 5, 5), (0, 4, 7, 6, 3), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 2, 0), (0, 6, 2, 9), (0, 4, 8, 5, 5, 8), (0, 1, 6, 7, 4), (0, 4, 8, 5, 7, 5, 5), (0, 4, 8, 5, 8, 5, 6), (0, 4, 0, 4, 5), (0, 1, 8, 3), (0, 6, 7, 5, 0), (0, 1, 6, 7, 2), (0, 4, 5, 4, 1, 2), (0, 4, 6, 2, 5), (0, 1, 4, 7), (0, 6, 7, 0, 7), (0, 4, 8, 0, 0, 2), (0, 0, 4, 7), (0, 5, 3, 5), (0, 1, 6, 4, 9), (0, 1, 4, 6, 2), (0, 2, 0, 6), (0, 4, 8, 5, 7, 0, 0, 6), (0, 6, 6, 1), (0, 9, 0, 1, 3), (0, 9, 3, 2, 3), (0, 1, 9, 1), (0, 4, 8, 5, 7, 0, 3, 1), (0, 8, 0), (0, 4, 8, 5, 8, 2), (0, 9, 1, 3), (0, 9, 1, 4), (0, 4, 2, 0), (0, 9, 0, 5), (0, 9, 6, 7), (0, 4, 8, 5, 0, 5), (0, 4, 8, 5, 0, 0), (0, 9, 8, 4), (0, 4, 8, 5, 7, 5, 5), (0, 4, 1, 3), (0, 4, 1, 1, 8), (0, 6, 1, 8), (0, 0, 6, 0), (0, 4, 1, 3), (0, 9, 2, 7), (0, 9, 0, 5), (0, 4, 8, 5, 3, 5), (0, 2, 2, 3), (0, 4, 8, 8, 0, 4), (0, 4, 9, 4), (0, 4, 0, 1, 4), (0, 9, 0, 1, 0), (0, 6, 6, 5), (0, 6, 5, 8, 5), (0, 9, 9, 4), (0, 4, 8, 5, 7, 4, 2), (0, 4, 8, 5, 3, 4, 7), (0, 1, 2, 8, 0), (0, 4, 8, 5, 3, 3), (0, 7, 7, 4), (0, 7, 7, 4), (0, 7, 2, 0), (0, 7, 2, 0), (0, 7, 2, 0), (0, 7, 2, 0), (0, 7, 7, 8), (0, 4, 8, 2, 3), (0, 4, 8, 1, 4), (0, 4, 8, 1, 4), (0, 4, 8, 3, 2), (0, 9, 9, 3), (0, 9, 9, 1), (0, 4, 0, 1, 5, 3), (0, 2, 1, 0), (0, 4, 8, 0, 5, 5), (0, 6, 6, 2), (0, 9, 1, 5), (0, 6, 1, 7), (0, 4, 8, 8, 7), (0, 1, 6, 5, 4), (0, 9, 9, 3), (0, 6, 7, 3, 4), (0, 4, 8, 0, 9, 1, 8), (0, 4, 8, 0, 8, 2), (0, 4, 1, 1, 2), (0, 4, 8, 1, 6), (0, 4, 8, 2, 1, 2), (0, 1, 8, 1, 1), (0, 4, 8, 5, 1, 1, 2), (0, 9, 3, 1), (0, 4, 8, 5, 8, 6, 2), (0, 1, 6, 3, 1, 9), (0, 4, 8, 5, 7, 4, 1), (0, 4, 8, 5, 2, 9), (0, 0, 2, 9), (0, 2, 7, 6), (0, 2, 7, 7), (0, 6, 6, 1), (0, 5, 9), (0, 6, 7, 1), (0, 6, 2, 3), (0, 6, 7, 1), (0, 4, 8, 5, 7, 4, 1), (0, 6, 7, 1), (0, 4, 8, 3, 4, 6), (0, 7, 2, 1), (0, 6, 7, 1), (0, 7, 2, 1), (0, 6, 8, 5), (0, 4, 8, 5, 8, 4, 5), (0, 1, 2, 7, 8), (0, 7, 2, 3), (0, 7, 2, 3), (0, 7, 2, 1), (0, 1, 1, 8), (0, 7, 2, 4), (0, 7, 1, 0, 9), (0, 7, 7, 2), (0, 7, 8, 0), (0, 4, 8, 5, 7, 4, 0), (0, 4, 8, 9, 4, 1), (0, 4, 8, 9, 4, 1), (0, 7, 1, 6, 5), (0, 4, 0, 3, 2), (0, 4, 5, 2, 6), (0, 4, 8, 5, 7, 0, 9, 9), (0, 7, 9, 2), (0, 9, 3, 2, 7, 5), (0, 6, 6, 6), (0, 6, 7, 0, 6), (0, 4, 7, 6, 1), (0, 4, 8, 5, 0, 2), (0, 4, 8, 5, 7, 0, 3, 0), (0, 4, 8, 0, 5, 6), (0, 6, 6, 9), (0, 4, 8, 2, 6), (0, 1, 2, 5, 4), (0, 4, 8, 5, 7, 2, 4), (0, 1, 2, 3), (0, 7, 9, 6), (0, 4, 8, 9, 4, 5), (0, 4, 8, 3, 4, 6), (0, 0, 7, 4), (0, 4, 8, 5, 9, 6), (0, 7, 2, 6), (0, 7, 3, 0), (0, 7, 3, 0), (0, 7, 3, 0), (0, 4, 8, 4, 1), (0, 6, 7, 0, 0), (0, 4, 8, 3, 1), (0, 5, 7, 0), (0, 4, 8, 4, 8), (0, 4, 8, 5, 7, 6, 1), (0, 1, 6, 3, 8), (0, 7, 3, 0), (0, 2, 5, 0), (0, 7, 2, 1), (0, 4, 8, 5, 7, 9, 6), (0, 7, 2, 3), (0, 7, 2, 4), (0, 7, 2, 4), (0, 7, 2, 4), (0, 7, 2, 4), (0, 7, 2, 4), (0, 6, 3, 8), (0, 1, 2, 8, 0), (0, 7, 2, 3), (0, 7, 2, 3), (0, 7, 2, 3), (0, 7, 2, 3), (0, 7, 2, 3), (0, 2, 6, 4, 0), (0, 7, 2, 7), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 7, 2, 1), (0, 6, 0, 4), (0, 5, 7, 4), (0, 1, 2, 5, 1), (0, 4, 8, 8, 1), (0, 4, 8, 5, 8, 7), (0, 1, 2, 7, 6), (0, 0, 9, 6), (0, 1, 4, 6, 2), (0, 4, 1, 2), (0, 4, 1, 1, 7), (0, 4, 0, 7, 3), (0, 4, 0, 7, 3), (0, 4, 8, 5, 7, 3, 4), (0, 1, 6, 9, 1), (0, 4, 0, 6, 3, 9), (0, 4, 8, 5, 7, 8, 1), (0, 4, 8, 3, 3), (0, 4, 8, 5, 2, 0), (0, 3, 7), (0, 6, 5, 2), (0, 6, 0, 9, 1), (0, 6, 3, 9), (0, 6, 1, 4), (0, 4, 8, 1, 0), (0, 4, 8, 8, 0, 0), (0, 6, 0, 8, 2), (0, 4, 0, 7, 3), (0, 4, 2, 3), (0, 0, 1, 9, 1), (0, 4, 8, 5, 7, 0, 2, 0), (0, 1, 4, 1), (0, 4, 8, 5, 3, 0), (0, 6, 5, 8, 6), (0, 5, 5), (0, 9, 6, 8), (0, 6, 9, 1), (0, 9, 3, 2, 0), (0, 6, 0, 5, 0), (0, 6, 9, 2), (0, 6, 9, 1), (0, 6, 9, 6), (0, 4, 8, 8, 6), (0, 5, 8, 0), (0, 4, 0, 7, 2), (0, 2, 3, 2, 2), (0, 6, 9, 7, 1), (0, 9, 0, 2), (0, 9, 0, 5), (0, 4, 8, 5, 7, 3, 4), (0, 4, 8, 4, 4, 4), (0, 1, 5, 6, 7), (0, 4, 0, 6, 1), (0, 4, 8, 5, 1, 1, 6), (0, 4, 5, 2, 7), (0, 4, 2, 4, 4), (0, 4, 8, 4, 8), (0, 4, 8, 4, 8), (0, 1, 5, 7), (0, 4, 1, 7), (0, 4, 1, 7), (0, 7, 1, 9), (0, 4, 8, 2, 2), (0, 4, 8, 2, 4, 5), (0, 4, 8, 5, 2, 3), (0, 9, 0, 1, 5), (0, 4, 1, 7), (0, 4, 1, 7), (0, 4, 1, 1, 7), (0, 2, 3, 6, 0), (0, 9, 0, 2), (0, 4, 8, 5, 2, 4), (0, 4, 8, 5, 2, 3), (0, 1, 4, 4), (0, 6, 0, 0, 1), (0, 6, 1, 2, 0), (0, 9, 8, 7), (0, 4, 8, 6, 4, 1), (0, 1, 2, 4, 1), (0, 4, 5, 2, 5), (0, 4, 5, 8, 1), (0, 9, 0, 2), (0, 4, 0, 8, 6), (0, 6, 6, 0), (0, 1, 6, 7, 1), (0, 4, 8, 5, 7, 2, 8), (0, 9, 3, 2, 0), (0, 4, 7, 1), (0, 1, 6, 7, 5, 0), (0, 7, 2, 5), (0, 4, 6, 2, 5), (0, 4, 8, 5, 4, 1), (0, 2, 6, 0), (0, 4, 8, 0, 1), (0, 4, 8, 3, 1), (0, 4, 0, 1, 5, 0), (0, 4, 8, 2, 5, 5), (0, 4, 8, 3, 3), (0, 4, 0, 6, 4, 1), (0, 0, 8, 0), (0, 4, 4, 5), (0, 4, 8, 5, 8, 6, 5), (0, 4, 0, 6, 2), (0, 4, 7, 6, 2), (0, 9, 0, 3), (0, 4, 7, 3), (0, 4, 8, 1, 6), (0, 7, 9, 4), (0, 7, 3, 1), (0, 9, 6, 4), (0, 1, 8, 1, 1), (0, 4, 8, 2, 1, 4), (0, 4, 8, 5, 3, 0), (0, 0, 6, 3), (0, 3, 7), (0, 1, 7, 3), (0, 1, 5, 3), (0, 6, 1, 2, 2), (0, 4, 8, 5, 1, 2), (0, 1, 9, 2), (0, 4, 8, 8, 5), (0, 5, 8, 3), (0, 6, 1, 9), (0, 4, 8, 6, 3), (0, 9, 0, 2), (0, 9, 0, 2), (0, 1, 9, 8), (0, 9, 8, 7), (0, 2, 6, 4, 3), (0, 4, 8, 2, 1, 4), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 7, 0, 1), (0, 4, 0, 1, 5, 9), (0, 2, 3, 0), (0, 6, 7, 5, 1), (0, 6, 1, 2, 2), (0, 4, 5, 2, 8), (0, 5, 3, 3, 1), (0, 5, 3, 3, 1), (0, 5, 3, 7), (0, 6, 1, 6), (0, 6, 7, 0, 1), (0, 1, 9, 0, 7), (0, 6, 1, 0), (0, 1, 7, 2), (0, 9, 2, 6), (0, 5, 3, 3, 1), (0, 4, 8, 0, 9, 8), (0, 6, 8, 6), (0, 4, 4, 0, 4), (0, 4, 8, 8, 2), (0, 4, 8, 0, 8, 2), (0, 9, 9, 6), (0, 4, 4, 0, 7), (0, 4, 0, 5, 5), (0, 4, 9, 2, 7), (0, 4, 4, 0, 7), (0, 4, 8, 5, 7, 6, 8), (0, 1, 6, 5, 4), (0, 0, 8, 4), (0, 6, 9, 8, 9), (0, 1, 2, 4, 2), (0, 3, 7), (0, 5, 3, 3, 2), (0, 4, 8, 5, 9, 2), (0, 4, 8, 5, 0, 2), (0, 4, 8, 5, 7, 0, 5), (0, 4, 8, 8, 8, 6), (0, 4, 0, 1, 5, 8), (0, 1, 0, 6), (0, 4, 4, 7), (0, 9, 0, 8), (0, 0, 2, 5), (0, 4, 0, 6, 4, 1), (0, 4, 8, 5, 7, 9, 6), (0, 4, 8, 5, 2, 3), (0, 0, 0, 3), (0, 2, 7, 6), (0, 4, 9, 2, 8), (0, 5, 3, 3, 1), (0, 2, 8, 1), (0, 0, 6, 2), (0, 4, 8, 0, 5, 1), (0, 9, 9, 3), (0, 4, 8, 5, 2, 0), (0, 2, 7, 1), (0, 6, 7, 8), (0, 5, 1, 3), (0, 4, 8, 5, 7, 9, 5), (0, 1, 8, 3), (0, 6, 0, 3, 0), (0, 1, 4, 9), (0, 1, 2, 3), (0, 1, 3, 9, 3), (0, 4, 8, 3, 4, 2), (0, 2, 6, 2), (0, 2, 7, 0, 7), (0, 4, 8, 8, 2), (0, 0, 1, 5), (0, 0, 1, 9, 1), (0, 0, 2, 7), (0, 4, 8, 8, 5), (0, 7, 7, 5), (0, 0, 4, 4), (0, 9, 4, 6), (0, 7, 5, 6), (0, 7, 2, 2), (0, 7, 2, 2), (0, 7, 2, 2), (0, 7, 2, 8), (0, 5, 3, 3, 2), (0, 4, 8, 5, 2, 8), (0, 0, 6, 4), (0, 4, 8, 5, 7, 6, 1), (0, 9, 9, 4), (0, 7, 2, 2), (0, 7, 2, 9), (0, 7, 2, 2), (0, 7, 2, 2), (0, 7, 2, 2), (0, 7, 2, 2), (0, 7, 2, 2), (0, 1, 4, 5), (0, 6, 1, 7), (0, 4, 8, 5, 7, 4, 4), (0, 7, 1, 0, 3), (0, 4, 0, 1, 0), (0, 6, 1, 2, 2), (0, 7, 7, 5), (0, 9, 3, 2, 3), (0, 0, 4, 3), (0, 4, 8, 5, 5, 8), (0, 4, 8, 5, 9, 7), (0, 4, 8, 5, 7, 5, 5), (0, 6, 6, 4, 3), (0, 4, 8, 5, 4, 8, 1), (0, 4, 8, 5, 1, 1, 6), (0, 4, 8, 5, 1, 4), (0, 1, 7, 7, 1), (0, 6, 8, 2), (0, 4, 8, 5, 3, 2), (0, 1, 2, 2, 5), (0, 4, 8, 5, 7, 9, 4), (0, 0, 2, 1), (0, 1, 2, 5, 7, 6), (0, 1, 7, 7, 7), (0, 6, 8, 7), (0, 4, 8, 2, 4, 5), (0, 4, 8, 8, 5), (0, 1, 7, 7, 1), (0, 1, 2, 8, 0), (0, 4, 0, 1, 5, 9), (0, 2, 7, 1), (0, 0, 6, 2), (0, 4, 8, 5, 0, 8), (0, 2, 3, 2, 1), (0, 0, 6, 2), (0, 9, 1, 3), (0, 9, 5, 0), (0, 6, 2, 1), (0, 4, 8, 2, 5, 4), (0, 4, 8, 5, 9, 1), (0, 4, 8, 5, 7, 5, 9), (0, 4, 8, 5, 7, 9, 0), (0, 2, 7, 2), (0, 6, 0, 5, 1), (0, 1, 6, 3, 1, 5), (0, 4, 5, 9), (0, 6, 9, 6), (0, 4, 0, 1, 2), (0, 4, 8, 8, 0, 6), (0, 0, 0, 3), (0, 6, 0, 0, 5), (0, 4, 3, 3), (0, 4, 8, 3, 1), (0, 4, 8, 5, 0, 2), (0, 6, 2, 9), (0, 6, 7, 6), (0, 6, 9, 7, 1), (0, 6, 0, 3, 5), (0, 6, 3, 4), (0, 6, 3, 7), (0, 1, 2, 3), (0, 6, 9, 7, 2), (0, 6, 7, 4, 6), (0, 6, 0, 9, 6), (0, 6, 3, 3, 6), (0, 4, 4, 1, 8), (0, 6, 2, 4), (0, 6, 8, 4), (0, 6, 9, 3), (0, 6, 9, 7, 4), (0, 4, 8, 5, 7, 8, 8), (0, 6, 6, 1), (0, 4, 8, 2, 1, 4), (0, 9, 1, 5), (0, 6, 0, 0, 1), (0, 1, 2, 5, 7, 2), (0, 4, 0, 7, 8), (0, 4, 8, 5, 4, 3), (0, 4, 2, 4, 1), (0, 6, 1, 7), (0, 2, 7, 9), (0, 9, 0, 5), (0, 4, 8, 2, 8), (0, 6, 7, 3, 2), (0, 6, 0, 1, 2), (0, 5, 1, 6), (0, 0, 4, 8), (0, 9, 1, 3), (0, 4, 8, 5, 3, 6), (0, 4, 8, 5, 3, 6), (0, 9, 5, 5), (0, 4, 8, 5, 3, 6), (0, 0, 2, 0), (0, 2, 3, 2, 1), (0, 4, 0, 7, 3), (0, 6, 3, 4), (0, 0, 6, 1), (0, 4, 8, 4, 1), (0, 4, 8, 5, 1, 2), (0, 6, 1, 7), (0, 6, 7, 9), (0, 7, 5, 2), (0, 9, 1, 1), (0, 6, 0, 2), (0, 6, 6, 9), (0, 4, 9, 4), (0, 9, 2, 8), (0, 6, 8, 0), (0, 6, 3, 7), (0, 7, 1, 0, 3), (0, 1, 6, 0), (0, 9, 9, 4), (0, 9, 9, 6), (0, 7, 1, 1), (0, 4, 8, 5, 7, 9, 2), (0, 6, 0, 7, 2), (0, 6, 5, 7), (0, 1, 0, 2, 7), (0, 1, 4, 6, 2), (0, 6, 9, 4), (0, 6, 7, 8), (0, 6, 7, 0, 7), (0, 4, 7, 0), (0, 4, 8, 5, 9, 4), (0, 4, 8, 0, 4), (0, 6, 0, 0, 1), (0, 9, 4, 7), (0, 6, 9, 7, 4), (0, 4, 8, 5, 3, 8), (0, 9, 2, 4), (0, 0, 4, 7), (0, 4, 8, 8, 4), (0, 9, 1, 7), (0, 6, 9, 7, 0), (0, 6, 1, 4), (0, 4, 8, 2, 9), (0, 6, 9, 4), (0, 6, 5, 7), (0, 6, 5, 4), (0, 6, 5, 5), (0, 9, 3, 2, 7, 0), (0, 9, 1, 1), (0, 9, 1, 1), (0, 0, 9, 1), (0, 9, 9, 9), (0, 4, 0, 2, 3), (0, 1, 5, 4), (0, 1, 2, 2, 3), (0, 9, 0, 8), (0, 6, 5, 2), (0, 1, 2, 9), (0, 4, 8, 5, 7, 4, 9), (0, 0, 1, 9, 2), (0, 6, 7, 6), (0, 6, 3, 1), (0, 4, 8, 5, 8, 6, 5), (0, 9, 1, 0), (0, 4, 8, 5, 7, 6, 9), (0, 9, 9, 8), (0, 6, 0, 1, 0), (0, 0, 6, 2), (0, 6, 5, 8, 2), (0, 4, 4, 7), (0, 4, 5, 7, 9), (0, 2, 8, 8), (0, 6, 6, 4, 2), (0, 9, 3, 2, 3), (0, 4, 0, 6, 6), (0, 4, 8, 5, 3, 5), (0, 4, 5, 8, 0), (0, 0, 1, 2), (0, 4, 8, 5, 1, 4), (0, 6, 5, 7), (0, 2, 3, 2, 4), (0, 2, 3, 6, 0), (0, 4, 6, 5, 1), (0, 1, 4, 3, 3), (0, 4, 8, 8, 4), (0, 0, 2, 4), (0, 6, 6, 2), (0, 4, 2, 2, 3), (0, 1, 2, 4, 3), (0, 4, 8, 5, 7, 9, 2), (0, 2, 6, 0), (0, 4, 8, 4, 1), (0, 4, 0, 1, 8), (0, 6, 5, 7), (0, 6, 5, 7), (0, 1, 5, 2, 1), (0, 4, 6, 1, 7), (0, 4, 2, 4, 3, 3), (0, 4, 8, 5, 7, 0, 0, 8), (0, 6, 9, 7, 0), (0, 7, 9, 3), (0, 7, 9, 3), (0, 7, 9, 9), (0, 6, 2, 6), (0, 4, 8, 5, 9, 2), (0, 4, 8, 2, 2), (0, 0, 4, 1), (0, 0, 5), (0, 1, 6, 7, 4), (0, 1, 2, 5, 1), (0, 9, 3, 2, 7, 9), (0, 6, 2, 5), (0, 7, 3, 4), (0, 6, 0, 5, 7), (0, 6, 8, 0), (0, 4, 0, 7, 9), (0, 4, 8, 0, 2), (0, 4, 3, 1, 2), (0, 6, 8, 2), (0, 5, 3, 3, 1), (0, 7, 3, 4), (0, 4, 8, 5, 4, 8, 0), (0, 4, 6, 1, 9), (0, 4, 8, 5, 3, 0), (0, 4, 8, 8, 7), (0, 4, 8, 5, 8, 3, 8), (0, 4, 8, 2, 9), (0, 4, 8, 2, 9), (0, 6, 7, 5, 3), (0, 4, 8, 3, 4, 6), (0, 4, 2, 4, 1), (0, 7, 3, 0), (0, 4, 2, 4, 4), (0, 7, 3, 0), (0, 4, 0, 7, 3), (0, 2, 7, 1), (0, 4, 0, 1, 8), (0, 6, 0, 7, 8), (0, 6, 7, 1), (0, 4, 8, 5, 1, 1, 6), (0, 1, 6, 7, 3), (0, 2, 3, 8, 2), (0, 4, 8, 5, 2, 1), (0, 6, 8, 3), (0, 6, 8, 9), (0, 0, 4, 9), (0, 7, 1, 5, 1), (0, 2, 6, 0), (0, 6, 8, 6), (0, 6, 8, 4), (0, 6, 8, 0), (0, 6, 8, 0), (0, 6, 0, 0, 1), (0, 4, 1, 7), (0, 6, 8, 7), (0, 0, 0, 4, 1), (0, 2, 6, 4, 0), (0, 0, 0, 4, 1), (0, 6, 3, 5), (0, 6, 3, 4), (0, 6, 2, 9), (0, 6, 3, 3, 3), (0, 6, 3, 4), (0, 6, 9, 7, 7), (0, 6, 6, 7), (0, 6, 1, 8), (0, 6, 8, 5), (0, 6, 9, 7, 4), (0, 6, 7, 3, 4), (0, 6, 9, 7, 4), (0, 6, 9, 8, 5), (0, 6, 9, 8, 5), (0, 6, 9, 8, 5), (0, 6, 9, 8, 5), (0, 6, 8, 0), (0, 9, 5, 8), (0, 9, 5, 1), (0, 4, 8, 2, 9), (0, 6, 0, 9, 8), (0, 9, 4, 0), (0, 6, 9, 8, 5), (0, 9, 5, 7), (0, 6, 3, 2), (0, 6, 3, 3, 5), (0, 6, 3, 7), (0, 6, 0, 8, 4), (0, 6, 8, 3), (0, 7, 3, 0), (0, 7, 3, 0), (0, 4, 8, 5, 8, 4, 1), (0, 4, 8, 1, 5), (0, 7, 3, 0), (0, 7, 3, 0), (0, 4, 8, 8, 8, 0), (0, 7, 3, 0), (0, 6, 6, 9), (0, 4, 5, 4, 1, 2), (0, 7, 3, 0), (0, 4, 8, 8, 0, 3), (0, 6, 3, 4), (0, 6, 1, 6), (0, 1, 0, 2, 3), (0, 4, 8, 5, 7, 7, 5), (0, 6, 0, 8, 2), (0, 6, 1, 2, 2), (0, 6, 7, 6), (0, 5, 7, 7), (0, 6, 8, 4), (0, 9, 5, 9), (0, 6, 5, 2), (0, 2, 6, 4, 5), (0, 0, 4, 4), (0, 6, 0, 8, 4), (0, 6, 7, 6), (0, 2, 6, 4, 2), (0, 4, 4, 8), (0, 6, 5, 4), (0, 4, 5, 4, 1, 0), (0, 4, 4, 0, 3), (0, 6, 1, 2, 3), (0, 6, 0, 8, 2), (0, 0, 9, 2, 1), (0, 6, 9, 7, 4), (0, 2, 6, 8), (0, 2, 7, 7), (0, 6, 5, 8, 3), (0, 6, 0, 8, 2), (0, 4, 8, 5, 7, 0, 0, 3), (0, 6, 0, 8, 2), (0, 4, 8, 5, 4, 8, 8), (0, 9, 2, 8), (0, 0, 3, 0), (0, 6, 6, 8), (0, 6, 5, 5), (0, 4, 8, 6, 2), (0, 6, 8, 4), (0, 4, 5, 0), (0, 0, 9, 3), (0, 4, 8, 9, 4, 7), (0, 6, 1, 2, 5), (0, 4, 8, 5, 1, 1, 8), (0, 6, 0, 8, 6), (0, 0, 9, 3), (0, 9, 3, 1), (0, 9, 3, 9), (0, 6, 3, 1), (0, 6, 3, 5), (0, 4, 8, 5, 7, 8, 7), (0, 6, 7, 2), (0, 6, 3, 6), (0, 6, 3, 3, 3), (0, 9, 8, 2), (0, 6, 0, 8, 6), (0, 6, 6, 3), (0, 6, 7, 3, 2), (0, 4, 8, 5, 1, 1, 3), (0, 6, 9, 3), (0, 1, 6, 6, 5), (0, 4, 8, 5, 7, 0, 0, 2), (0, 4, 8, 4, 4, 6), (0, 6, 2, 3), (0, 4, 8, 5, 7, 6, 1), (0, 6, 7, 0, 8), (0, 4, 8, 5, 7, 4, 6), (0, 6, 2, 9), (0, 6, 2, 3), (0, 6, 2, 3), (0, 1, 6, 5, 3), (0, 6, 6, 9), (0, 6, 2, 5), (0, 9, 6, 1), (0, 4, 8, 5, 1, 1, 6), (0, 4, 4, 4, 2), (0, 6, 1, 2, 5), (0, 6, 0, 8, 2), (0, 0, 4, 1), (0, 6, 2, 9), (0, 4, 8, 2, 2), (0, 4, 5, 5), (0, 6, 2, 4), (0, 6, 0, 0, 1), (0, 6, 8, 3), (0, 9, 3, 2, 2), (0, 6, 9, 7, 2), (0, 3, 5), (0, 6, 3, 3, 5), (0, 6, 6, 4, 4), (0, 4, 8, 5, 2, 8), (0, 4, 0, 8, 3), (0, 6, 2, 3), (0, 4, 8, 0, 8, 2), (0, 6, 0, 0, 1), (0, 6, 7, 2), (0, 6, 9, 7, 9), (0, 4, 6, 1, 1, 1), (0, 2, 6, 8), (0, 4, 8, 5, 8, 8), (0, 4, 7, 3), (0, 4, 2, 7, 1, 1), (0, 6, 2, 3), (0, 6, 8, 4), (0, 6, 0, 5, 4), (0, 1, 5, 4), (0, 6, 0, 1, 0), (0, 6, 1, 1, 1), (0, 6, 3, 5), (0, 6, 2, 3), (0, 4, 8, 5, 7, 7, 4), (0, 4, 8, 5, 4, 6), (0, 4, 8, 2, 4, 5), (0, 6, 2, 3), (0, 6, 1, 2, 2), (0, 3, 7), (0, 4, 8, 7, 0), (0, 0, 6, 5), (0, 6, 7, 9), (0, 0, 2, 3), (0, 4, 8, 0, 1), (0, 4, 9, 9), (0, 1, 7, 7, 1), (0, 3, 7), (0, 6, 0, 3, 1), (0, 4, 8, 5, 1, 9), (0, 4, 9, 5, 0, 0, 6), (0, 6, 2, 4), (0, 6, 0, 6), (0, 4, 8, 5, 1, 0, 2), (0, 4, 8, 5, 8, 8), (0, 6, 2, 4), (0, 2, 3, 0), (0, 0, 4, 1), (0, 0, 1, 7), (0, 9, 0, 9), (0, 4, 8, 5, 8, 2), (0, 2, 7, 0, 1), (0, 6, 0, 9, 1), (0, 6, 9, 0), (0, 7, 3, 7), (0, 9, 0, 1, 4), (0, 4, 8, 0, 8, 8), (0, 4, 8, 5, 7, 6, 6), (0, 4, 7, 8, 5), (0, 6, 3, 3, 4), (0, 4, 8, 5, 0, 2), (0, 1, 6, 7, 3), (0, 6, 5, 2), (0, 0, 3, 4), (0, 0, 1, 2), (0, 4, 9, 5, 0, 0, 4), (0, 7, 1, 5, 0), (0, 9, 1, 4), (0, 6, 1, 2, 2), (0, 9, 9, 6), (0, 7, 1, 6, 0), (0, 4, 8, 5, 7, 7, 2), (0, 4, 8, 5, 1, 5), (0, 4, 8, 5, 0, 0), (0, 6, 0, 2), (0, 1, 7, 7, 1), (0, 6, 3, 4), (0, 6, 0, 0, 2), (0, 4, 0, 2, 0), (0, 4, 0, 1, 5, 2), (0, 4, 8, 5, 1, 5), (0, 5, 8, 8), (0, 5, 8, 8), (0, 6, 0, 9, 5), (0, 6, 0, 9, 0), (0, 9, 1, 4), (0, 2, 9, 1), (0, 9, 0, 2), (0, 4, 9, 9), (0, 6, 0, 9, 2), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 4, 8, 1), (0, 6, 1, 2, 5), (0, 7, 1, 9), (0, 2, 3, 6, 0), (0, 4, 8, 5, 7, 8, 0), (0, 9, 1, 3), (0, 6, 7, 5, 3), (0, 9, 9, 2), (0, 9, 8, 7), (0, 6, 7, 5, 3), (0, 4, 8, 5, 7, 8, 0), (0, 9, 4, 3), (0, 6, 6, 3), (0, 9, 1, 4), (0, 4, 8, 0, 3), (0, 4, 8, 0, 3), (0, 9, 5, 1), (0, 4, 8, 5, 7, 3, 5), (0, 4, 1, 5), (0, 9, 2, 0), (0, 6, 6, 3), (0, 6, 0, 5, 2), (0, 4, 0, 7, 3), (0, 4, 5, 1), (0, 9, 1, 8), (0, 9, 1, 1), (0, 9, 5, 5), (0, 4, 8, 8, 3), (0, 4, 0, 7, 0), (0, 4, 0, 3, 3), (0, 6, 2, 9), (0, 6, 1, 4), (0, 2, 0, 1), (0, 4, 8, 5, 7, 5, 5), (0, 9, 1, 4), (0, 3, 7), (0, 3, 7), (0, 2, 3, 6, 0), (0, 4, 8, 5, 8, 3, 4), (0, 4, 6, 2, 0), (0, 4, 8, 5, 6, 5), (0, 6, 3, 3, 4), (0, 6, 6, 4, 3), (0, 9, 4, 0), (0, 4, 8, 5, 6, 2), (0, 1, 2, 9), (0, 6, 6, 4, 8), (0, 6, 7, 5, 3), (0, 9, 9, 6), (0, 6, 7, 2), (0, 7, 5, 5), (0, 4, 4, 3), (0, 1, 1, 0), (0, 1, 6, 1, 2), (0, 7, 5, 2), (0, 4, 6, 5, 9), (0, 0, 2, 9), (0, 4, 4, 4, 8), (0, 2, 1, 0), (0, 4, 8, 5, 2, 0), (0, 1, 9, 0, 0), (0, 4, 8, 5, 7, 2, 0), (0, 7, 1, 9), (0, 6, 7, 5, 8), (0, 4, 8, 5, 0, 0), (0, 9, 6, 9), (0, 9, 1, 6), (0, 6, 0, 3, 4), (0, 4, 7, 8, 5), (0, 0, 6, 1), (0, 4, 8, 5, 9, 1), (0, 6, 1, 2, 2), (0, 2, 1, 4), (0, 4, 8, 0, 7), (0, 9, 6, 5), (0, 4, 6, 8, 2), (0, 4, 8, 5, 8, 6, 5), (0, 9, 9, 4), (0, 9, 9, 6), (0, 6, 2, 6), (0, 0, 0, 0), (0, 6, 6, 8), (0, 1, 6, 9, 6), (0, 7, 1, 6, 3), (0, 7, 9, 3), (0, 4, 8, 4, 3), (0, 4, 6, 0, 3), (0, 6, 6, 1), (0, 4, 7, 6, 2), (0, 5, 3, 3, 4), (0, 9, 6, 4), (0, 4, 0, 0, 8), (0, 4, 8, 5, 7, 5, 3), (0, 5, 3, 5), (0, 0, 4, 9), (0, 6, 3, 9), (0, 4, 0, 3, 3), (0, 4, 9, 3), (0, 4, 5, 4, 1, 4, 0), (0, 4, 8, 5, 7, 0, 3, 4), (0, 6, 5, 2), (0, 6, 7, 7), (0, 6, 5, 3), (0, 4, 8, 5, 7, 9, 5), (0, 6, 1, 2, 7), (0, 6, 0, 3, 0), (0, 4, 6, 1, 7), (0, 2, 8, 8), (0, 6, 6, 1), (0, 6, 0, 9, 1), (0, 1, 5, 4), (0, 6, 8, 1), (0, 7, 9, 0), (0, 4, 8, 5, 7, 6, 4), (0, 4, 8, 2, 5, 1), (0, 7, 5, 8), (0, 6, 9, 3), (0, 9, 1, 4), (0, 4, 0, 6, 2), (0, 7, 1, 0, 4), (0, 7, 6), (0, 6, 3, 1), (0, 6, 3, 3, 3), (0, 6, 3, 6), (0, 6, 3, 3, 7), (0, 0, 7, 6), (0, 9, 3, 2, 3), (0, 6, 9, 7, 3), (0, 9, 3, 2, 7, 8), (0, 4, 8, 5, 7, 0, 2, 0), (0, 0, 0, 5, 6), (0, 4, 8, 8, 0, 3), (0, 1, 7, 7, 1), (0, 6, 1, 5), (0, 6, 5, 4), (0, 7, 5, 2), (0, 1, 5, 4), (0, 4, 8, 4, 1), (0, 7, 9, 3), (0, 4, 0, 2, 2), (0, 2, 2, 8), (0, 4, 8, 5, 7, 1, 1), (0, 9, 3, 2, 7, 0), (0, 2, 9, 1), (0, 6, 0, 0, 1), (0, 6, 6, 9), (0, 4, 0, 3, 0), (0, 4, 8, 5, 7, 0, 6), (0, 4, 8, 8, 2), (0, 1, 2, 7, 3), (0, 7, 8, 2), (0, 7, 8, 2), (0, 7, 8, 2), (0, 6, 0, 9, 6), (0, 5, 3, 2), (0, 9, 3, 5), (0, 6, 3, 8), (0, 4, 5, 2, 5), (0, 4, 2, 2, 1), (0, 6, 0, 9, 6), (0, 6, 3, 4), (0, 4, 8, 5, 9, 4), (0, 2, 7, 8), (0, 2, 2, 5), (0, 6, 6, 9), (0, 6, 1, 2, 5), (0, 5, 3, 3, 6), (0, 0, 9, 2, 1), (0, 0, 5), (0, 0, 9, 2, 2), (0, 0, 9, 5), (0, 2, 5, 6), (0, 0, 9, 2, 8), (0, 0, 9, 6), (0, 5, 3, 3, 0), (0, 4, 8, 9, 4, 1), (0, 4, 8, 2, 1, 6), (0, 4, 8, 5, 7, 9, 7), (0, 4, 8, 7, 8), (0, 0, 0, 4, 0), (0, 9, 9, 0), (0, 6, 1, 2, 2), (0, 2, 0, 1), (0, 6, 0, 6), (0, 6, 6, 9), (0, 6, 6, 1), (0, 9, 9, 6), (0, 6, 3, 1), (0, 2, 9, 9), (0, 4, 7, 8, 2), (0, 6, 1, 2, 2), (0, 6, 7, 0, 6), (0, 6, 4, 6), (0, 7, 1, 0, 8), (0, 6, 9, 8, 8), (0, 9, 3, 2, 4), (0, 9, 9, 2), (0, 4, 4, 8), (0, 6, 0, 0, 1), (0, 4, 0, 3, 2), (0, 6, 6, 4, 0), (0, 0, 1, 9, 4), (0, 2, 3, 7, 7), (0, 4, 8, 5, 7, 6, 1), (0, 9, 9, 4), (0, 6, 2, 9), (0, 1, 0, 4), (0, 6, 3, 1), (0, 6, 3, 1), (0, 6, 0, 0, 9), (0, 6, 1, 8), (0, 1, 2, 8, 7), (0, 7, 1, 4), (0, 5, 0, 2, 1, 1), (0, 4, 2, 4, 3, 0), (0, 6, 0, 0, 1), (0, 7, 8, 3), (0, 9, 1, 3), (0, 4, 8, 5, 3, 5), (0, 4, 8, 5, 2, 3), (0, 4, 8, 5, 2, 3), (0, 4, 0, 6, 4, 1), (0, 4, 8, 5, 7, 0, 3, 2), (0, 4, 9, 2, 0), (0, 4, 8, 5, 7, 0, 5), (0, 9, 3, 2, 7, 0), (0, 4, 8, 5, 3, 9), (0, 4, 8, 5, 7, 0, 1), (0, 4, 0, 3, 0), (0, 4, 8, 8, 0, 6), (0, 2, 9, 3), (0, 6, 0, 8, 2), (0, 9, 8, 4), (0, 4, 8, 5, 2, 2), (0, 4, 0, 2, 7), (0, 2, 7, 0, 9), (0, 6, 3, 2), (0, 0, 0, 1), (0, 9, 7, 4), (0, 0, 4, 3), (0, 4, 8, 5, 7, 1, 2), (0, 4, 0, 4, 4), (0, 1, 2, 5, 6), (0, 0, 7, 4), (0, 6, 3, 4), (0, 6, 1, 2, 8), (0, 4, 4, 9), (0, 6, 0, 0, 3), (0, 6, 0, 7, 8), (0, 2, 0, 8), (0, 4, 6, 6, 3), (0, 2, 1, 8), (0, 6, 3, 3, 1), (0, 6, 3, 1), (0, 9, 5, 3), (0, 0, 8, 1), (0, 6, 0, 5, 9), (0, 9, 8, 1), (0, 2, 0, 1), (0, 6, 1, 7), (0, 1, 6, 5, 1), (0, 1, 6, 5, 1), (0, 4, 8, 5, 7, 0, 3, 6), (0, 9, 1, 3), (0, 4, 5, 4, 1, 4, 5), (0, 4, 5, 9), (0, 4, 5, 8, 0), (0, 6, 0, 0, 5), (0, 7, 7, 7), (0, 4, 9, 5, 0, 8), (0, 6, 0, 0, 4), (0, 2, 2, 5), (0, 2, 2, 5), (0, 6, 0, 5, 9), (0, 4, 8, 5, 7, 5, 6), (0, 6, 5, 9), (0, 1, 8, 3), (0, 9, 1, 3), (0, 6, 6, 4, 1), (0, 6, 0, 9, 3), (0, 6, 6, 8), (0, 1, 0, 3), (0, 2, 9, 0), (0, 2, 1, 4), (0, 4, 8, 5, 0, 4), (0, 9, 1, 4), (0, 4, 7, 4), (0, 2, 1, 4), (0, 6, 0, 3, 0), (0, 6, 8, 2), (0, 0, 6, 4), (0, 6, 3, 2), (0, 4, 0, 1, 5, 2), (0, 2, 6, 5), (0, 4, 8, 5, 4, 8, 4), (0, 4, 4, 3), (0, 6, 3, 8), (0, 6, 3, 4), (0, 6, 3, 4), (0, 4, 8, 6, 2), (0, 9, 9, 2), (0, 9, 3, 2, 7, 0), (0, 7, 7, 0), (0, 6, 7, 0, 6), (0, 6, 7, 5, 2), (0, 0, 0, 0), (0, 4, 4, 1, 1, 3), (0, 4, 4, 0, 2), (0, 6, 3, 1), (0, 1, 6, 7, 3), (0, 4, 8, 5, 4, 8, 1), (0, 6, 6, 1), (0, 7, 5, 2), (0, 4, 8, 5, 8, 6, 8), (0, 6, 5, 0), (0, 2, 1, 2), (0, 4, 8, 4, 4, 4), (0, 4, 8, 5, 8, 2), (0, 4, 8, 5, 5, 1), (0, 6, 3, 9), (0, 6, 0, 5, 2), (0, 4, 0, 2, 2), (0, 6, 8, 4), (0, 4, 8, 5, 7, 0, 7), (0, 2, 3, 5), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 8, 3, 3), (0, 1, 7, 7, 1), (0, 4, 4, 9), (0, 2, 9, 2), (0, 4, 8, 5, 7, 3, 0), (0, 6, 2, 1), (0, 4, 8, 8, 0, 8), (0, 1, 2, 2, 8), (0, 2, 8, 6), (0, 4, 8, 5, 7, 3, 4), (0, 4, 0, 2, 1), (0, 0, 0, 5, 5), (0, 1, 3, 5), (0, 1, 6, 8), (0, 4, 8, 5, 7, 6, 7), (0, 0, 0, 5, 5), (0, 4, 9, 2, 5), (0, 4, 8, 5, 7, 7, 5), (0, 1, 5, 1), (0, 4, 8, 2, 7), (0, 9, 1, 4), (0, 2, 8, 8), (0, 1, 6, 6, 1), (0, 9, 1, 0), (0, 4, 8, 7, 8), (0, 4, 0, 1, 4), (0, 2, 8, 7), (0, 4, 8, 5, 3, 3), (0, 4, 8, 2, 5, 9), (0, 4, 8, 5, 3, 5), (0, 4, 2, 2, 5), (0, 1, 2, 4, 3), (0, 1, 5, 8, 2), (0, 1, 2, 5, 7, 8), (0, 4, 8, 5, 2, 8), (0, 2, 2, 5), (0, 4, 8, 0, 9, 1, 1), (0, 4, 8, 5, 7, 4, 1), (0, 9, 1, 4), (0, 9, 1, 3), (0, 0, 1, 8), (0, 4, 0, 7, 3), (0, 4, 7, 1), (0, 4, 7, 6, 8), (0, 6, 0, 1, 2), (0, 1, 9, 8), (0, 2, 0, 2), (0, 4, 8, 5, 7, 0, 7), (0, 0, 1, 9, 1), (0, 1, 0, 5), (0, 2, 6, 4, 3), (0, 5, 0, 7), (0, 0, 7, 7), (0, 6, 3, 3, 1), (0, 9, 2, 7), (0, 4, 8, 5, 7, 3, 5), (0, 4, 8, 5, 7, 9, 0), (0, 0, 8, 0), (0, 2, 8, 8), (0, 7, 9, 4), (0, 4, 5, 4, 1, 5), (0, 6, 4, 4), (0, 6, 9, 4), (0, 4, 4, 0, 1), (0, 9, 9, 2), (0, 4, 8, 9, 4, 1), (0, 0, 8, 0), (0, 6, 9, 8, 4), (0, 6, 9, 6), (0, 7, 9, 8), (0, 6, 0, 4), (0, 9, 4, 5), (0, 0, 1, 7), (0, 1, 5, 0, 5), (0, 9, 0, 7), (0, 4, 8, 5, 8, 3, 4), (0, 4, 8, 5, 7, 0, 8), (0, 6, 0, 3, 6), (0, 6, 5, 6), (0, 6, 0, 8, 2), (0, 9, 7, 6), (0, 1, 2, 5, 4), (0, 9, 0, 0), (0, 1, 6, 3, 1, 6), (0, 3, 7), (0, 1, 9, 0, 0), (0, 4, 8, 8, 8, 0), (0, 4, 8, 5, 7, 2, 5), (0, 4, 3, 1, 4), (0, 0, 0, 5, 9), (0, 0, 0, 5, 0), (0, 6, 6, 4, 8), (0, 6, 3, 9), (0, 4, 6, 5, 0), (0, 6, 7, 0, 1), (0, 6, 6, 6), (0, 6, 6, 6), (0, 4, 6, 1, 2), (0, 9, 7, 6), (0, 0, 0, 5, 1), (0, 4, 8, 4, 4, 4), (0, 9, 8, 2), (0, 4, 8, 0, 6), (0, 6, 1, 2, 6), (0, 6, 6, 6), (0, 6, 6, 9), (0, 9, 3, 9), (0, 4, 8, 5, 7, 0, 4), (0, 4, 8, 5, 7, 0, 4), (0, 4, 8, 5, 7, 4, 5), (0, 4, 8, 5, 7, 6, 4), (0, 4, 8, 5, 7, 6, 4), (0, 6, 3, 9), (0, 4, 8, 8, 1), (0, 4, 4, 9), (0, 0, 9, 2, 1), (0, 6, 8, 1), (0, 4, 8, 8, 3), (0, 9, 5, 6), (0, 3, 1, 0), (0, 3, 1, 0), (0, 3, 1, 0), (0, 6, 0, 3, 4), (0, 3, 1, 1), (0, 6, 3, 7), (0, 4, 8, 2, 5, 3), (0, 4, 8, 5, 9, 3), (0, 4, 8, 5, 9, 4), (0, 0, 0, 5, 8), (0, 1, 5, 5, 5), (0, 4, 6, 4), (0, 6, 1, 7), (0, 2, 3, 8, 6), (0, 9, 6, 5), (0, 5, 3, 3, 8), (0, 1, 6, 7, 0, 2), (0, 6, 8, 6), (0, 4, 0, 6, 7), (0, 6, 5, 2), (0, 9, 6, 0), (0, 4, 5, 8, 5), (0, 4, 8, 5, 7, 0, 6), (0, 0, 1, 2), (0, 4, 0, 5, 3), (0, 4, 8, 8, 7), (0, 4, 8, 7, 5), (0, 9, 4, 3), (0, 1, 6, 7, 8), (0, 1, 3, 4), (0, 6, 0, 0, 2), (0, 9, 0, 2), (0, 6, 8, 2), (0, 6, 0, 7, 3), (0, 4, 0, 0, 7), (0, 0, 1, 8), (0, 0, 1, 1), (0, 0, 1, 8), (0, 4, 8, 5, 7, 5, 8), (0, 6, 8, 4), (0, 6, 0, 0, 6), (0, 4, 8, 5, 8, 3, 2), (0, 4, 4, 7), (0, 5, 3, 3, 7), (0, 6, 0, 0, 6), (0, 6, 2, 1), (0, 9, 9, 4), (0, 4, 8, 6, 3), (0, 9, 1, 0), (0, 9, 1, 1), (0, 4, 8, 5, 0, 2), (0, 1, 5, 1), (0, 1, 6, 7, 0, 0), (0, 9, 5, 3), (0, 6, 0, 1, 6), (0, 4, 8, 5, 7, 0, 1), (0, 6, 0, 9, 7), (0, 6, 0, 0, 6), (0, 9, 5, 5), (0, 6, 9, 2), (0, 4, 2, 4, 3, 0), (0, 6, 5, 3), (0, 6, 3, 8), (0, 6, 3, 3, 1), (0, 6, 3, 3, 3), (0, 6, 9, 7, 1), (0, 6, 2, 6), (0, 5, 8, 1), (0, 4, 8, 5, 7, 2, 0), (0, 0, 9, 6), (0, 4, 7, 8, 0), (0, 6, 2, 9), (0, 6, 3, 4), (0, 6, 1, 2, 2), (0, 4, 8, 3, 6), (0, 0, 1, 2), (0, 6, 7, 3, 5), (0, 7, 8, 0), (0, 4, 8, 9, 4, 1), (0, 6, 6, 4, 5), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 5, 3, 2), (0, 6, 6, 3), (0, 9, 8, 1), (0, 4, 5, 4, 2), (0, 4, 8, 0, 5, 4), (0, 6, 5, 3), (0, 6, 3, 4), (0, 4, 0, 7, 2), (0, 4, 0, 7, 3), (0, 6, 5, 7), (0, 6, 5, 8, 0), (0, 6, 5, 9), (0, 1, 6, 3, 1, 0), (0, 4, 8, 9, 4, 1), (0, 4, 8, 9, 4, 1), (0, 6, 7, 7), (0, 4, 5, 4, 1, 2), (0, 1, 6, 4, 1), (0, 4, 8, 9, 4, 1), (0, 4, 8, 5, 8, 3, 5), (0, 6, 5, 8, 6), (0, 7, 8, 1, 4, 0, 0, 0), (0, 4, 8, 5, 7, 4, 7), (0, 7, 8, 1, 4, 0, 0, 0), (0, 6, 9, 4), (0, 6, 9, 7, 2), (0, 4, 6, 1, 1, 2), (0, 9, 1, 2), (0, 7, 8, 8), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 2), (0, 4, 0, 6, 4, 2), (0, 6, 5, 3), (0, 6, 5, 8, 9), (0, 9, 5, 8), (0, 4, 8, 9, 1), (0, 0, 0, 5, 2), (0, 4, 8, 6, 4, 6), (0, 9, 3, 2, 1), (0, 4, 8, 9, 2), (0, 9, 3, 2, 9), (0, 7, 5, 4), (0, 6, 7, 3, 3), (0, 6, 7, 4, 1), (0, 6, 7, 4, 9), (0, 4, 8, 3, 5), (0, 9, 0, 5), (0, 6, 7, 5, 3), (0, 0, 7, 0), (0, 2, 6, 4, 1), (0, 4, 0, 0, 9), (0, 6, 3, 9), (0, 6, 5, 8, 1), (0, 6, 5, 7), (0, 4, 7, 4), (0, 4, 7, 8, 4), (0, 4, 5, 4, 1, 4, 2), (0, 4, 8, 6, 3), (0, 6, 7, 7), (0, 4, 8, 1, 6), (0, 9, 0, 2), (0, 4, 8, 5, 9, 3), (0, 4, 8, 8, 3), (0, 9, 2, 0), (0, 6, 3, 9), (0, 4, 8, 0, 0, 7), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 8), (0, 6, 5, 7), (0, 5, 3, 3, 6), (0, 4, 4, 5), (0, 7, 8, 4), (0, 7, 8, 1, 4, 0, 0, 0), (0, 1, 3, 1), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 8), (0, 7, 8, 9), (0, 6, 5, 6), (0, 1, 5, 8, 3), (0, 4, 8, 5, 7, 6, 1), (0, 4, 2, 7, 8), (0, 9, 0, 5), (0, 4, 4, 0, 2), (0, 0, 7, 1), (0, 4, 4, 0, 1), (0, 2, 2, 3), (0, 4, 8, 2, 8), (0, 4, 2, 2, 6), (0, 4, 8, 9, 4, 1), (0, 0, 4, 4), (0, 7, 8, 1, 4, 0, 0, 0), (0, 4, 8, 5, 3, 6), (0, 4, 8, 5, 3, 6), (0, 7, 8, 6), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 5), (0, 2, 3, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 2, 3, 5), (0, 6, 3, 4), (0, 1, 4, 3, 4), (0, 7, 8, 1, 4, 0, 0, 0), (0, 4, 8, 3, 4, 9), (0, 6, 5, 3), (0, 6, 5, 2), (0, 1, 6, 6, 8), (0, 4, 8, 2, 2), (0, 7, 3, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 6, 8, 0), (0, 4, 8, 5, 2, 1), (0, 6, 5, 4), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 1, 0, 3), (0, 7, 8, 1, 1), (0, 7, 8, 1, 4, 0, 0, 0), (0, 5, 7, 9), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 8), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 8), (0, 7, 8, 7), (0, 7, 8, 1, 4, 0, 0, 0), (0, 7, 8, 1, 4, 0, 0, 0), (0, 6, 3, 4), (0, 6, 7, 5, 0), (0, 4, 6, 0, 6), (0, 2, 8, 1), (0, 6, 9, 3), (0, 7, 8, 1, 4, 0, 0, 0), (0, 6, 8, 0), (0, 2, 6, 2), (0, 4, 8, 5, 1, 1, 6), (0, 1, 2, 8, 3), (0, 2, 2, 1), (0, 6, 8, 6), (0, 4, 8, 5, 1, 2), (0, 4, 8, 4, 1), (0, 2, 2, 8), (0, 4, 0, 6, 1), (0, 2, 5, 0), (0, 2, 5, 0), (0, 6, 6, 7), (0, 6, 1, 2, 6), (0, 5, 4, 2), (0, 6, 7, 0, 3), (0, 4, 8, 0, 2), (0, 7, 8, 1, 4, 0, 0, 0), (0, 4, 8, 4, 4, 6), (0, 6, 7, 4, 2), (0, 6, 7, 2), (0, 4, 8, 5, 3, 1), (0, 6, 6, 3), (0, 6, 7, 6), (0, 6, 6, 4, 1), (0, 6, 8, 5), (0, 2, 0, 2), (0, 0, 0, 4, 0), (0, 4, 8, 5, 1, 1, 6), (0, 2, 6, 2), (0, 2, 6, 2), (0, 1, 5, 3), (0, 2, 2, 1), (0, 0, 0, 5, 0), (0, 0, 6, 4), (0, 1, 3, 9, 9), (0, 0, 0, 4, 0), (0, 6, 8, 2), (0, 4, 8, 5, 7, 7, 1), (0, 1, 2, 8, 2), (0, 4, 4, 4, 2), (0, 4, 8, 3, 9), (0, 4, 0, 7, 3), (0, 4, 8, 5, 5, 8), (0, 9, 0, 8), (0, 9, 0, 2), (0, 9, 0, 1, 4), (0, 4, 8, 5, 7, 9, 4), (0, 6, 1, 2, 2), (0, 4, 8, 5, 7, 9, 8), (0, 4, 8, 6, 4, 1), (0, 0, 0, 5, 5), (0, 4, 8, 5, 2, 5), (0, 4, 8, 5, 7, 3, 1), (0, 4, 0, 5, 2), (0, 4, 8, 5, 8, 7), (0, 4, 2, 2, 3), (0, 1, 0, 5), (0, 0, 1, 9, 1), (0, 6, 0, 4), (0, 4, 6, 0, 5), (0, 4, 6, 2, 7), (0, 6, 6, 0), (0, 6, 9, 7, 3), (0, 9, 6, 1), (0, 4, 8, 8, 7), (0, 2, 9, 2), (0, 9, 4, 3), (0, 9, 0, 7), (0, 6, 0, 7, 3), (0, 4, 7, 6, 3), (0, 4, 7, 8, 0), (0, 4, 7, 8, 4), (0, 6, 1, 3), (0, 4, 2, 8), (0, 6, 9, 3), (0, 0, 1, 9, 9), (0, 4, 8, 7, 5), (0, 4, 8, 0, 0, 5), (0, 4, 8, 8, 2), (0, 6, 3, 7), (0, 5, 3, 5), (0, 6, 0, 8, 3), (0, 4, 8, 0, 0, 5), (0, 0, 6, 9), (0, 4, 4, 7), (0, 4, 8, 5, 1, 6), (0, 6, 6, 4, 6), (0, 6, 6, 7), (0, 4, 2, 2, 4), (0, 4, 8, 5, 2, 4), (0, 6, 2, 6), (0, 6, 6, 5), (0, 4, 4, 1, 1, 0), (0, 4, 8, 6, 5), (0, 4, 6, 1, 1, 1), (0, 9, 4, 2), (0, 7, 1, 1), (0, 6, 2, 3), (0, 4, 2, 0), (0, 4, 8, 9, 4, 1), (0, 5, 1, 8), (0, 6, 2, 6), (0, 6, 2, 6), (0, 4, 8, 5, 8, 7), (0, 1, 9, 9), (0, 6, 2, 6), (0, 4, 8, 5, 5, 0), (0, 4, 5, 5), (0, 7, 7, 3), (0, 7, 7, 6), (0, 7, 1, 6, 0), (0, 6, 2, 5), (0, 1, 5, 9), (0, 4, 8, 5, 7, 4, 0), (0, 4, 0, 6, 4, 5), (0, 4, 0, 3, 4), (0, 6, 2, 5), (0, 4, 8, 5, 1, 9), (0, 2, 8, 7), (0, 3, 4), (0, 6, 2, 6), (0, 6, 3, 7), (0, 0, 1, 9, 1), (0, 6, 7, 3, 9), (0, 6, 2, 5), (0, 1, 2, 5, 7, 5), (0, 6, 2, 3), (0, 6, 2, 6), (0, 6, 2, 3), (0, 6, 2, 9), (0, 6, 2, 3), (0, 2, 0, 9), (0, 6, 0, 3, 2), (0, 0, 1, 9, 1), (0, 6, 0, 8, 5), (0, 5, 3, 3, 2), (0, 8, 0), (0, 6, 0, 3, 4), (0, 0, 0, 3), (0, 6, 0, 0, 6), (0, 9, 5, 1), (0, 6, 0, 9, 1), (0, 6, 3, 1), (0, 4, 8, 0, 7), (0, 4, 8, 8, 0, 1), (0, 6, 0, 1, 0), (0, 6, 7, 3, 9), (0, 4, 8, 5, 7, 8, 5), (0, 6, 8, 5), (0, 4, 8, 5, 2, 3), (0, 6, 1, 4), (0, 6, 0, 8, 3), (0, 6, 1, 2, 8), (0, 1, 9, 3), (0, 4, 8, 5, 2, 8), (0, 1, 6, 7, 5, 1), (0, 4, 2, 3), (0, 6, 3, 4), (0, 2, 6, 9), (0, 4, 8, 5, 7, 5, 0), (0, 1, 5, 6, 2), (0, 6, 2, 0), (0, 6, 8, 5), (0, 4, 2, 2, 7), (0, 4, 8, 5, 2, 4), (0, 2, 7, 1), (0, 9, 3, 2, 7, 7), (0, 0, 0, 1), (0, 4, 2, 0), (0, 6, 0, 3, 9), (0, 6, 0, 8, 4), (0, 6, 2, 6), (0, 4, 0, 1, 4), (0, 6, 0, 8, 6), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 0, 0, 2), (0, 6, 0, 3, 2), (0, 4, 0, 1, 9), (0, 4, 8, 0, 5, 3), (0, 4, 6, 7), (0, 6, 0, 8, 8), (0, 4, 8, 9, 2), (0, 4, 8, 9, 2), (0, 4, 8, 9, 2), (0, 1, 2, 6), (0, 4, 4, 3), (0, 4, 8, 8, 8, 1), (0, 6, 3, 6), (0, 4, 8, 0, 6), (0, 4, 0, 6, 4, 8), (0, 6, 5, 4), (0, 4, 8, 5, 1, 1, 1), (0, 6, 7, 9), (0, 6, 2, 9), (0, 9, 3, 1), (0, 6, 6, 3), (0, 6, 0, 4), (0, 4, 8, 5, 3, 3), (0, 5, 8, 0), (0, 5, 8, 2), (0, 5, 8, 1), (0, 4, 8, 6, 3), (0, 6, 1, 0), (0, 1, 8, 1, 6), (0, 4, 5, 9), (0, 4, 4, 6), (0, 6, 3, 3, 5), (0, 6, 2, 1), (0, 4, 8, 5, 7, 0, 0, 1), (0, 4, 8, 5, 8, 3, 9), (0, 9, 2, 3), (0, 4, 0, 1, 5, 5), (0, 6, 0, 4), (0, 6, 0, 4), (0, 6, 0, 1, 8), (0, 2, 1, 8), (0, 4, 8, 0, 3), (0, 1, 6, 7, 0, 4), (0, 2, 5, 0), (0, 4, 6, 0, 6), (0, 4, 6, 0, 6), (0, 1, 5, 8, 1), (0, 2, 0, 3), (0, 6, 5, 8, 2), (0, 4, 8, 8, 0, 3), (0, 4, 6, 6, 0), (0, 4, 5, 4, 1, 2), (0, 4, 5, 4, 1, 4, 4), (0, 6, 2, 7), (0, 6, 1, 1, 4), (0, 4, 8, 5, 7, 5, 6), (0, 4, 0, 6, 1), (0, 4, 0, 6, 1), (0, 2, 2, 4), (0, 4, 8, 0, 8, 8), (0, 4, 7, 2), (0, 1, 9, 0, 0), (0, 6, 8, 5), (0, 6, 3, 8), (0, 4, 4, 1, 7), (0, 4, 0, 1, 2), (0, 4, 8, 5, 3, 8), (0, 2, 3, 1), (0, 6, 0, 3, 2), (0, 9, 4, 7), (0, 0, 8, 4), (0, 4, 2, 2, 1), (0, 6, 5, 4), (0, 6, 3, 5), (0, 4, 8, 5, 7, 2, 5), (0, 9, 3, 2, 3), (0, 4, 2, 4, 5), (0, 6, 2, 6), (0, 6, 0, 6), (0, 1, 5, 5, 4), (0, 4, 6, 4), (0, 6, 3, 5), (0, 6, 3, 5), (0, 6, 7, 8), (0, 6, 3, 5), (0, 4, 8, 5, 7, 0, 8), (0, 2, 5, 0), (0, 8, 0), (0, 2, 3, 8, 1), (0, 4, 0, 4, 5), (0, 4, 8, 5, 9, 8), (0, 4, 6, 0, 3), (0, 4, 8, 0, 6), (0, 7, 5, 0), (0, 4, 8, 0, 3), (0, 5, 1, 1), (0, 6, 3, 5), (0, 4, 0, 2, 4), (0, 2, 0, 7), (0, 9, 8, 1), (0, 4, 9, 8), (0, 4, 8, 4, 1), (0, 9, 3, 2, 5), (0, 6, 3, 4), (0, 5, 0, 2, 1, 0), (0, 2, 8, 3), (0, 5, 8, 1), (0, 9, 5, 6), (0, 1, 9, 6), (0, 1, 6, 5, 2), (0, 1, 9, 0, 4), (0, 6, 5, 4), (0, 6, 7, 5, 6), (0, 3, 7), (0, 5, 0, 2, 4), (0, 1, 5, 6, 2), (0, 6, 3, 9), (0, 6, 7, 9), (0, 6, 8, 0), (0, 4, 8, 5, 9, 6), (0, 6, 0, 0, 1), (0, 6, 7, 9), (0, 4, 8, 5, 7, 9, 0), (0, 7, 5, 2), (0, 4, 9, 3), (0, 4, 8, 0, 2), (0, 6, 1, 4), (0, 5, 0, 9), (0, 4, 8, 8, 7), (0, 4, 4, 7), (0, 4, 0, 2, 1), (0, 6, 9, 7, 6), (0, 5, 8, 9), (0, 4, 5, 5), (0, 4, 4, 0, 2), (0, 4, 8, 5, 7, 6, 4), (0, 6, 0, 0, 1), (0, 9, 7, 6), (0, 0, 0, 7), (0, 4, 8, 5, 7, 4, 9), (0, 9, 9, 4), (0, 4, 6, 8, 2), (0, 5, 1, 6), (0, 0, 0, 3), (0, 4, 8, 5, 5, 8), (0, 6, 0, 0, 1), (0, 4, 8, 2, 8), (0, 4, 8, 4, 4, 1), (0, 7, 1, 6, 6), (0, 4, 0, 2, 4), (0, 9, 7, 6), (0, 9, 6, 4), (0, 5, 0, 2, 1, 1), (0, 5, 0, 3), (0, 6, 7, 9), (0, 0, 0, 0), (0, 6, 7, 5, 5), (0, 2, 1, 7), (0, 0, 0, 3), (0, 1, 9, 5), (0, 4, 8, 5, 7, 0, 3, 1), (0, 9, 6, 5), (0, 6, 3, 3, 9), (0, 6, 0, 2), (0, 6, 1, 4), (0, 6, 7, 3, 4), (0, 6, 0, 9, 6), (0, 4, 8, 5, 7, 1, 1), (0, 5, 0, 2, 1, 1), (0, 6, 1, 7), (0, 6, 8, 8), (0, 5, 0, 2, 1, 1), (0, 4, 7, 7), (0, 4, 8, 5, 7, 9, 4), (0, 6, 0, 2), (0, 4, 4, 9), (0, 4, 9, 5, 7), (0, 4, 0, 0, 8), (0, 6, 3, 8), (0, 6, 5, 3), (0, 6, 9, 1), (0, 0, 8, 0), (0, 4, 8, 0, 9, 9), (0, 6, 2, 5), (0, 4, 8, 5, 7, 8, 4), (0, 4, 8, 3, 4, 5), (0, 9, 6, 5), (0, 6, 7, 6), (0, 6, 2, 5), (0, 0, 1, 9, 1), (0, 4, 8, 5, 2, 2), (0, 6, 7, 4, 4), (0, 6, 6, 8), (0, 4, 8, 5, 1, 5), (0, 4, 4, 0, 2), (0, 4, 0, 6, 1), (0, 4, 8, 0, 1), (0, 6, 6, 8), (0, 2, 1, 4), (0, 6, 6, 8), (0, 4, 8, 2, 2), (0, 6, 0, 1, 0), (0, 6, 9, 2), (0, 5, 0, 2, 7), (0, 0, 0, 5, 5), (0, 0, 7, 0), (0, 7, 1, 0, 6), (0, 5, 3, 3, 4), (0, 4, 8, 2, 4, 4), (0, 0, 7, 9), (0, 6, 3, 1), (0, 4, 6, 1, 1, 5), (0, 9, 3, 1), (0, 0, 0, 3), (0, 6, 3, 1), (0, 6, 1, 1, 2), (0, 1, 6, 5, 7), (0, 0, 0, 7), (0, 1, 6, 9, 6), (0, 4, 8, 5, 7, 6, 5), (0, 6, 3, 9), (0, 4, 8, 5, 7, 4, 3), (0, 4, 6, 1, 1, 1), (0, 4, 9, 9), (0, 4, 8, 5, 1, 1, 2), (0, 6, 7, 5, 5), (0, 4, 8, 5, 7, 4, 9), (0, 6, 7, 7), (0, 6, 3, 7), (0, 4, 8, 5, 7, 1, 8), (0, 4, 4, 0, 5), (0, 6, 5, 3), (0, 2, 0, 4), (0, 5, 1, 1), (0, 6, 6, 9), (0, 6, 8, 2), (0, 4, 0, 8, 1), (0, 4, 8, 5, 3, 2), (0, 9, 6, 1), (0, 4, 8, 5, 7, 9, 3), (0, 2, 2, 3), (0, 7, 8, 1, 4, 0, 0, 0), (0, 4, 8, 9, 4, 4), (0, 4, 8, 5, 1, 9), (0, 6, 0, 1, 0), (0, 4, 8, 0, 9, 1, 5), (0, 4, 4, 0, 2), (0, 4, 8, 5, 7, 9, 0), (0, 4, 0, 6, 5), (0, 1, 6, 5, 7), (0, 1, 2, 4, 4), (0, 6, 0, 0, 8), (0, 1, 6, 1, 0), (0, 6, 0, 5, 5), (0, 0, 0, 1), (0, 2, 3, 7, 3), (0, 6, 7, 4, 1), (0, 4, 8, 5, 7, 5, 6), (0, 6, 3, 7), (0, 4, 8, 1, 2), (0, 6, 7, 1), (0, 4, 5, 8, 7), (0, 4, 8, 8, 5), (0, 4, 8, 5, 7, 1, 4), (0, 4, 8, 5, 7, 4, 1), (0, 2, 7, 1), (0, 6, 6, 4, 8), (0, 6, 3, 3, 2), (0, 4, 8, 0, 8, 2), (0, 6, 1, 2, 2), (0, 0, 2, 8), (0, 4, 0, 2, 3), (0, 0, 2, 8), (0, 6, 5, 4), (0, 6, 3, 1), (0, 6, 7, 8), (0, 0, 2, 8), (0, 4, 0, 2, 3), (0, 6, 8, 5), (0, 9, 8, 1), (0, 1, 2, 5, 7, 5), (0, 4, 8, 5, 4, 0), (0, 6, 0, 0, 1), (0, 2, 8, 8), (0, 6, 3, 3, 3), (0, 6, 5, 3), (0, 6, 5, 3), (0, 9, 5, 1), (0, 4, 8, 5, 7, 5, 5), (0, 5, 8, 9), (0, 6, 0, 0, 6), (0, 1, 3, 9, 0), (0, 2, 0, 1), (0, 4, 2, 4, 3, 0), (0, 2, 1, 4), (0, 4, 8, 5, 0, 9), (0, 6, 9, 9), (0, 7, 1, 1), (0, 9, 4, 2), (0, 6, 0, 0, 6), (0, 6, 9, 9), (0, 6, 5, 3), (0, 4, 0, 6, 5), (0, 4, 4, 0, 6), (0, 4, 8, 2, 5, 1), (0, 6, 3, 6), (0, 4, 4, 0, 2), (0, 4, 8, 5, 0, 5), (0, 1, 5, 0, 2), (0, 1, 5, 5, 0), (0, 4, 8, 5, 3, 4, 1), (0, 4, 8, 5, 7, 6, 8), (0, 4, 0, 1, 5, 3), (0, 4, 8, 5, 7, 5, 2), (0, 9, 9, 4), (0, 4, 8, 5, 7, 2, 0), (0, 4, 8, 5, 8, 4, 6), (0, 1, 2, 5, 8), (0, 6, 0, 5, 5), (0, 6, 8, 2), (0, 4, 8, 5, 6, 0), (0, 4, 8, 2, 5, 5), (0, 1, 6, 3, 9), (0, 4, 8, 5, 7, 1, 4), (0, 6, 6, 9), (0, 6, 0, 2), (0, 6, 1, 2, 4), (0, 9, 0, 0), (0, 2, 1, 8), (0, 6, 6, 0), (0, 4, 8, 5, 4, 8, 1), (0, 4, 8, 2, 4, 4), (0, 1, 3, 1), (0, 4, 6, 1, 6), (0, 6, 8, 5), (0, 4, 8, 5, 8, 6, 4), (0, 4, 8, 0, 8, 7), (0, 1, 6, 3, 3), (0, 1, 6, 6, 1), (0, 4, 8, 5, 7, 8, 7), (0, 6, 5, 7), (0, 4, 7, 4), (0, 2, 8, 8), (0, 2, 3, 8, 8), (0, 4, 8, 1, 9), (0, 6, 5, 8, 2), (0, 6, 6, 4, 2), (0, 4, 4, 9), (0, 6, 3, 5), (0, 6, 5, 5), (0, 1, 6, 0), (0, 6, 6, 5), (0, 2, 8, 8), (0, 6, 5, 3), (0, 2, 8, 8), (0, 2, 8, 8), (0, 6, 5, 8, 5), (0, 4, 6, 2, 1), (0, 4, 8, 0, 1), (0, 4, 8, 5, 7, 0, 4), (0, 6, 5, 5), (0, 6, 7, 2), (0, 1, 2, 8, 5), (0, 4, 4, 0, 2), (0, 1, 3, 5), (0, 2, 2, 3), (0, 4, 0, 4, 9), (0, 2, 8, 8), (0, 6, 5, 3), (0, 4, 8, 5, 7, 8, 7), (0, 4, 8, 5, 7, 0, 3, 1), (0, 6, 5, 8, 6), (0, 9, 2, 3), (0, 6, 5, 3), (0, 2, 1, 4), (0, 4, 0, 1, 5, 5), (0, 4, 0, 6, 4, 0), (0, 4, 8, 5, 7, 4, 0), (0, 1, 4, 3, 2), (0, 6, 7, 2), (0, 6, 9, 0), (0, 4, 0, 4, 7), (0, 4, 8, 5, 8, 3, 5), (0, 7, 1, 0, 3), (0, 7, 5, 5), (0, 1, 6, 7, 0, 3), (0, 4, 0, 3, 0), (0, 7, 5, 5), (0, 7, 1, 7), (0, 4, 1, 1, 1), (0, 4, 4, 0, 2), (0, 4, 8, 3, 4, 6), (0, 6, 7, 6), (0, 4, 8, 0, 9, 5), (0, 4, 8, 2, 2), (0, 4, 2, 2, 2), (0, 1, 6, 7, 8), (0, 6, 8, 3), (0, 6, 6, 8), (0, 6, 8, 0), (0, 9, 9, 2), (0, 1, 9, 0, 0), (0, 4, 4, 4, 2), (0, 6, 8, 0), (0, 6, 5, 4), (0, 1, 6, 9, 4), (0, 4, 8, 5, 1, 9), (0, 4, 8, 5, 7, 6, 9), (0, 2, 8, 4), (0, 4, 8, 5, 8, 5, 3), (0, 4, 8, 5, 8, 3, 0), (0, 2, 3, 2, 8), (0, 2, 0, 8), (0, 6, 7, 2), (0, 4, 7, 6, 3), (0, 6, 6, 7), (0, 7, 5, 0), (0, 6, 9, 7, 2), (0, 7, 7, 0), (0, 6, 5, 7), (0, 4, 4, 0, 1), (0, 4, 0, 1, 7), (0, 1, 3, 9, 2), (0, 4, 0, 6, 4, 7), (0, 6, 9, 9), (0, 2, 0, 5), (0, 1, 5, 0, 2), (0, 6, 3, 4), (0, 4, 0, 3, 0), (0, 1, 9, 5), (0, 4, 8, 5, 8, 6, 3), (0, 4, 0, 3, 6), (0, 4, 8, 5, 8, 2), (0, 4, 7, 8, 7), (0, 7, 9, 8), (0, 1, 7, 2), (0, 4, 8, 3, 8), (0, 5, 8, 8), (0, 4, 1, 4), (0, 6, 8, 5), (0, 5, 9), (0, 9, 2, 0), (0, 6, 8, 5), (0, 2, 8, 5), (0, 1, 3, 9, 0), (0, 6, 8, 9), (0, 4, 0, 8, 7), (0, 6, 8, 0), (0, 2, 6, 8), (0, 4, 8, 5, 7, 0, 3, 8), (0, 1, 4, 6, 6), (0, 4, 8, 5, 7, 4, 3), (0, 4, 5, 4, 2), (0, 6, 1, 9), (0, 6, 0, 8, 6), (0, 4, 4, 3), (0, 4, 0, 1, 5, 0), (0, 0, 0, 4, 0), (0, 4, 2, 2, 3), (0, 6, 0, 8, 4), (0, 1, 2, 5, 3), (0, 4, 8, 5, 7, 0, 3, 3), (0, 4, 8, 5, 7, 2, 7), (0, 6, 1, 4), (0, 6, 6, 3), (0, 6, 3, 1), (0, 6, 1, 1, 8), (0, 4, 5, 8, 1), (0, 6, 6, 4, 4), (0, 6, 6, 9), (0, 4, 8, 5, 8, 8), (0, 6, 1, 1, 1), (0, 2, 8, 0), (0, 1, 2, 5, 7, 5), (0, 6, 0, 8, 6), (0, 6, 3, 5), (0, 6, 3, 5), (0, 6, 3, 1), (0, 4, 0, 8, 4), (0, 4, 6, 0, 2), (0, 1, 2, 5, 7, 5), (0, 6, 0, 0, 3), (0, 6, 1, 2, 3), (0, 4, 4, 1, 1, 5), (0, 1, 6, 7, 9), (0, 4, 8, 6, 3), (0, 4, 0, 2, 1), (0, 4, 0, 8, 1), (0, 6, 6, 3), (0, 6, 3, 7), (0, 6, 0, 8, 6), (0, 4, 8, 0, 3), (0, 4, 0, 2, 8), (0, 4, 8, 5, 7, 9, 1), (0, 4, 0, 8, 1), (0, 4, 8, 5, 3, 4, 1), (0, 4, 8, 5, 7, 2, 6), (0, 6, 6, 6), (0, 4, 8, 5, 6, 9), (0, 4, 9, 5, 0, 0, 0), (0, 5, 3, 2), (0, 6, 0, 9, 9), (0, 4, 8, 2, 4, 8), (0, 2, 6, 7), (0, 4, 5, 4, 1, 2), (0, 6, 3, 3, 5), (0, 6, 3, 9), (0, 4, 8, 5, 6, 9), (0, 6, 3, 1), (0, 6, 2, 9), (0, 2, 1, 9), (0, 9, 7, 8), (0, 6, 2, 9), (0, 6, 2, 5), (0, 6, 2, 6), (0, 9, 4, 7), (0, 6, 2, 6), (0, 4, 0, 1, 5, 7), (0, 9, 4, 7), (0, 4, 8, 5, 8, 9), (0, 4, 6, 6, 5), (0, 2, 3, 6, 5), (0, 1, 6, 3, 1, 9), (0, 4, 8, 0, 4), (0, 4, 8, 5, 6, 9), (0, 1, 6, 1, 2), (0, 2, 6, 4, 7), (0, 4, 8, 5, 4, 7), (0, 6, 0, 0, 2), (0, 6, 9, 8, 2), (0, 6, 1, 5), (0, 6, 1, 1, 2), (0, 6, 0, 9, 5), (0, 6, 7, 4, 2), (0, 1, 2, 4, 0), (0, 2, 0, 2), (0, 9, 5, 1), (0, 6, 7, 3, 1), (0, 1, 7, 2), (0, 4, 8, 5, 3, 1), (0, 3, 7), (0, 4, 8, 5, 1, 1, 6), (0, 6, 0, 5, 4), (0, 7, 1, 0, 0), (0, 4, 8, 5, 2, 7), (0, 0, 2, 7), (0, 6, 7, 7), (0, 4, 2, 7, 1, 5), (0, 4, 8, 5, 1, 3), (0, 4, 8, 7, 5), (0, 4, 0, 6, 4, 1), (0, 4, 0, 1, 5, 0), (0, 6, 9, 7, 3), (0, 4, 8, 1, 2), (0, 6, 9, 6), (0, 4, 5, 7, 1), (0, 0, 8, 2), (0, 4, 8, 5, 9, 6), (0, 6, 7, 7), (0, 6, 7, 7), (0, 4, 8, 3, 4, 8), (0, 7, 3, 3), (0, 1, 9, 2), (0, 7, 3, 0), (0, 6, 1, 1, 2), (0, 7, 4, 1, 2), (0, 4, 0, 8, 8), (0, 4, 5, 7, 2), (0, 6, 3, 3, 4), (0, 1, 6, 9, 9), (0, 4, 0, 2, 1), (0, 4, 0, 2, 9), (0, 4, 5, 9), (0, 6, 0, 8, 6), (0, 6, 0, 0, 1), (0, 1, 2, 1), (0, 7, 4, 2), (0, 4, 0, 8, 2), (0, 5, 3, 3, 5), (0, 5, 8, 0), (0, 6, 3, 3, 8), (0, 7, 4, 1, 6, 0, 0, 0), (0, 0, 2, 9), (0, 4, 8, 5, 8, 6, 1), (0, 4, 0, 2, 4), (0, 1, 2, 2, 7), (0, 4, 8, 5, 2, 3), (0, 1, 9, 1), (0, 4, 8, 5, 2, 3), (0, 7, 4, 1, 6, 0, 0, 0), (0, 6, 3, 7), (0, 0, 4, 1), (0, 6, 0, 5, 7), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 8, 5, 7, 1, 2), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 3, 3), (0, 1, 2, 5, 4), (0, 4, 8, 5, 7, 9, 2), (0, 7, 4, 2), (0, 4, 8, 4, 4, 4), (0, 4, 5, 4, 1, 4, 3), (0, 5, 4, 3), (0, 7, 4, 1, 6, 0, 0, 0), (0, 5, 8, 1), (0, 7, 4, 1, 6, 0, 0, 0), (0, 1, 2, 4, 1), (0, 7, 4, 1, 6, 0, 0, 0), (0, 7, 4, 2), (0, 2, 2, 5), (0, 2, 0, 2), (0, 7, 4, 1, 6, 0, 0, 0), (0, 4, 4, 0, 9), (0, 7, 4, 1, 6, 0, 0, 0), (0, 6, 0, 9, 0), (0, 4, 8, 8, 8, 1), (0, 4, 8, 0, 8, 0), (0, 4, 8, 5, 7, 0, 7), (0, 4, 8, 5, 9, 8), (0, 7, 4, 1, 6, 0, 0, 0), (0, 2, 3, 8, 8)]


In [23]:
Iter_KM.get_cluster_titles((0,4,8,1), labels)

['Northern Ireland',
 'List of languages',
 'List of countries by official languages',
 'Second language',
 'Kölsch language',
 'Indo-European',
 'Cornish',
 'Lozi',
 'Dari (Eastern Persian)',
 'Dravidian languages',
 'Occitan language',
 'Karelia',
 'Scottish',
 'Nynorsk',
 'Yupik',
 'Iuridictum',
 'Interpreter (computing)',
 'Anatolian hypothesis',
 'Kabyles people',
 'Deutsche Welle',
 'Italian language in Croatia',
 'Sindhi language',
 'Parkari Koli',
 'Sarikoli',
 'Tibeto-Burman languages',
 'Vithoba',
 'Vietnamese Wikipedia',
 'Korean Wikipedia',
 'Kikuyu',
 'Estonian Wikipedia',
 'Albanian Wikipedia',
 'Hakha-Chin language',
 'Mycenaean culture',
 'Swahili',
 'Belizean Kriol language',
 'Scots-Irish',
 '-hou',
 'Herero language',
 'Nguni languages',
 'Tok Pisin',
 'Marashiyan',
 'Turkmen language',
 'Nuristani languages',
 'Pamir languages',
 'ISO 639-3',
 'Yanka Kupala',
 'Haitian Creole',
 "Q'eqchi' people",
 'Mikhail Lomonosov',
 'Nkroful',
 'Google Translate',
 'Tracy D. Ter

In [24]:
Iter_KM.get_cluster_titles((0,4,8,1,5), labels)

['List of countries by official languages',
 'Kölsch language',
 'Cornish',
 'Karelia',
 'Scottish',
 'Yupik',
 'Hakha-Chin language',
 'Nostratic']

Can also find nearest cluster to given feature vector. 

In [25]:
import random
vec=[random.random() for i in range(len(tfidf.get_feature_names()))]

In [26]:
cm_dict=Iter_KM.get_cm_dict(labels)

cm_dict stores the coordinates of the center of mass of each cluster

In [27]:
cm_dict[(0,4,5,3)]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.03922884,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.02440339,  0.        ,  0.        ,
         0.        ,  0.        ,  0.02098871,  0.        ,  0.        ,
         0.02171699,  0.        ,  0.        ,  0.        ,  0.        ,
         0.02714582,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [28]:
print(Iter_KM.get_nearest_cm(vec, cm_dict))

(0, 5, 7, 1)
